In [1]:
from skmultilearn.model_selection import iterative_train_test_split
import os
import pandas as pd
from ast import literal_eval
import numpy as np

from collections import Counter
import random

from tqdm import tqdm

In [1]:
import mlflow

model = mlflow.pyfunc.load_model(
    's3://deep-mlflow-artifact/16/8f1730fb49f04d7dade7cfea778840e7/artifacts/pyfunc_models_all',
)


2021/10/23 19:33:28 WARNING mlflow.pyfunc: The version of CloudPickle that was used to save the model, `CloudPickle 2.0.0`, differs from the version of CloudPickle that is currently running, `CloudPickle 1.6.0`, and may be incompatible


[2021-10-23 19:33:34.867 selim-laptop:15380 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-23 19:33:34.919 selim-laptop:15380 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [4]:
import pandas as pd
import os
test_data = pd.read_excel(os.path.join('..', '..', '..', '..', 'feedback_output.xlsx'))
test_data = test_data[['Entry']].rename(columns={'Entry':'excerpt'})
test_data = test_data[test_data.excerpt.apply(lambda x: 'NONE' != x.upper())]


In [6]:
preds = model.predict(test_data['excerpt'])

In [3]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.6.2','generated_entries'
)


original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
original_df.columns

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['entry_id', 'excerpt', 'analysis_framework_id', 'lead_id', 'project_id',
       'title', 'entry_type', 'verified', 'sectors', 'subpillars_2d',
       'subpillars_1d', 'geo_location', 'specific_needs_groups', 'severity',
       'dates', 'demographic_groups', 'reliability', 'affected_groups',
       'source_type', 'url', 'website'],
      dtype='object')

In [144]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def custom_stratified_train_test_split(df, train_ratio:float = 0.8):
    """
    custom function for stratified train test splitting
    1) take unique sub-tags (example: ['Health'])
    2) For each unique subtag:
        i) take all indexes that have that specific subtag
        ii) split them ransomly to train and test sets

    NB: a bit time consuming ~ 950 entries/second in average
    """
    train_ids = []
    test_ids = []
    
    unique_entries = list(np.unique(df['target'].apply(str)))
    for entry in tqdm(unique_entries):
        ids_entry = list(df[df.target.apply(str)==entry].entry_id.unique())

        train_ids_entry = random.sample(
            ids_entry, int(len(ids_entry) * train_ratio)
            )
        test_ids_entry = list(
            set(ids_entry) - set(train_ids_entry)
        )

        train_ids.append(train_ids_entry)
        test_ids.append(test_ids_entry)
        
    return flatten(train_ids), flatten(test_ids)

In [23]:
train, test = custom_stratified_train_test_split(original_df)

100%|██████████| 1523/1523 [02:57<00:00,  8.56it/s]


In [41]:
def flatten(t):
    return [item for sublist in t for item in sublist]



In [42]:
def clean_rows(row):
    """
    1) Apply litteral evaluation
    2) keep unique values
    """
    return list(set(literal_eval(row)))

def get_negative_samples(df, column_name):
    """
    return the ids of entries containng negative samples
    1) filter leads that contain at least one postive example (at least one tagged entry)
    2) keep sentences with no tags
    """
    df_bis = df[['entry_id', column_name, 'lead_id']].copy()

    df_bis['count'] = df_bis[column_name].apply(lambda x: len(x))

    max_counts = df_bis[['lead_id', 'count']].groupby('lead_id', as_index=False).max()
    tagged_leads = max_counts[max_counts['count']>0].lead_id.tolist()

    negative_ids = df_bis[
        df_bis.lead_id.isin(tagged_leads) & df_bis[column_name].apply(lambda x: len(x)==0)
    ].entry_id.unique()

    return negative_ids.tolist()

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [145]:
original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))

augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [146]:
train_data, test_data = preprocess_df(augmented_data, 'subpillars_1d', 
    proportion_negative_examples_train_df=0.25)

100%|██████████| 389/389 [00:24<00:00, 16.01it/s]
<ipython-input-134-b19494d6847a>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmented_nb_df['excerpt'] = augmented_nb_df.excerpt.apply(augment_sent)


In [147]:
def stats_train_test (rain, est, column_name:str):
    """
    Sanity check of data (proportion negative examples)
    """
    def compute_ratio_negative_positive (df):
        nb_rows_negative = df[df.target.apply(lambda x: len(x)==0)].shape[0]
        return  nb_rows_negative / df.shape[0]

    ratio_negative_positive = {
        f"ratio_negative_examples_train_{column_name}": compute_ratio_negative_positive (rain),
        f"ratio_negative_examples_test_{column_name}": compute_ratio_negative_positive (est),
    }

    return ratio_negative_positive

In [148]:
stats_train_test (train_data, test_data, 'subpillars_1d')

{'ratio_negative_examples_train_subpillars_1d': 0.2318456111710346,
 'ratio_negative_examples_test_subpillars_1d': 0.24740670084654823}

In [3]:
original_df['sectors'] = original_df['sectors'].apply(literal_eval)

In [4]:
def tagname_to_id(target):
    """
    Assign id to each tag
    """
    tag_set = set()
    for tags_i in target:
        tag_set.update(tags_i)
    tagname_to_tagid = {tag: i for i, tag in enumerate(list(sorted(tag_set)))}
    return tagname_to_tagid

tag_to_id = tagname_to_id(original_df['sectors'])

def sentence_to_list(tags):
    nb_entries = len(list(tag_to_id.keys()))
    entries = [0 for i in range (nb_entries)]
    for tag_tmp in tags:
        entries[tag_to_id[tag_tmp]] = 1
    return entries

tmp_df = original_df.sample(n=10_000)
y = tmp_df['sectors'].apply(sentence_to_list)

In [98]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [49]:
def custom_stratified_train_test_split(df, test_ratio):
    """
    custom function for stratified train test splitting
    1) take unique sub-tags (example: ['Health'])
    2) For each unique subtag:
        i) take all indexes that have that specific subtag
        ii) split them ransomly to train and test sets
    """
    train_ids = []
    test_ids = []

    unique_entries = list(np.unique(df['target']))
    for entry in tqdm(unique_entries):
        ids_entry = list(original_df[original_df.sectors.apply(str)==str(entry)].entry_id)

        train_ids_entry = random.sample(
            ids_entry, int(len(ids_entry) * 0.8)
            )
        test_ids_entry = list(
            set(ids_entry) - set(train_ids_entry)
        )

        train_ids.append(train_ids_entry)
        test_ids.append(test_ids_entry)
        
    return flatten(train_ids), flatten(test_ids)

  1%|▏         | 18/1410 [00:02<03:15,  7.13it/s]


KeyboardInterrupt: 

In [61]:
dill[`fmode`] = 'FILE_MODE'

SyntaxError: invalid syntax (<ipython-input-61-411a77378381>, line 1)

In [58]:
dill.fmode = 1

In [59]:
dill.settings

{'protocol': 4, 'byref': False, 'fmode': 0, 'recurse': False, 'ignore': False}

In [43]:
train = original_df[original_df.entry_id.isin(X_train)].sectors
test = original_df[original_df.entry_id.isin(X_test)].sectors

In [52]:
def flatten(t):
    return [item for sublist in t for item in sublist]

train_list = flatten(list(train))
test_list = flatten(list(test))

Counter(train_list)

Counter({'Health': 28460,
         'Livelihoods': 13353,
         'Shelter': 7902,
         'Education': 8298,
         'Food Security': 12174,
         'Cross': 22585,
         'Protection': 23012,
         'WASH': 8825,
         'Logistics': 2130,
         'Nutrition': 3914,
         'Agriculture': 2330})

In [53]:
Counter(test_list)

Counter({'Livelihoods': 3750,
         'Logistics': 719,
         'Education': 2469,
         'Food Security': 3606,
         'Health': 7678,
         'Cross': 6199,
         'Protection': 6202,
         'WASH': 2719,
         'Shelter': 2418,
         'Nutrition': 1263,
         'Agriculture': 786})

In [99]:
original_df['subpillars_2d'] = original_df.subpillars_2d.apply(literal_eval)
list_entries = list(np.unique(flatten(list(original_df.subpillars_2d))))

In [100]:
original_df['subpillars_1d'] = original_df.subpillars_1d.apply(literal_eval)
list_entries_1d = list(np.unique(flatten(list(original_df.subpillars_1d))))

In [101]:
list_subpillars_all = list_entries + list_entries_1d
list_subpillars_all

['At Risk->Number Of People At Risk',
 'At Risk->Risk And Vulnerabilities',
 'Capacities & Response->International Response',
 'Capacities & Response->Local Response',
 'Capacities & Response->National Response',
 'Capacities & Response->Number Of People Reached/Response Gaps',
 'Humanitarian Conditions->Coping Mechanisms',
 'Humanitarian Conditions->Living Standards',
 'Humanitarian Conditions->Number Of People In Need',
 'Humanitarian Conditions->Physical And Mental Well Being',
 'Impact->Driver/Aggravating Factors',
 'Impact->Impact On People',
 'Impact->Impact On Systems, Services And Networks',
 'Impact->Number Of People Affected',
 'Priority Interventions->Expressed By Humanitarian Staff',
 'Priority Interventions->Expressed By Population',
 'Priority Needs->Expressed By Humanitarian Staff',
 'Priority Needs->Expressed By Population',
 'Casualties->Dead',
 'Casualties->Injured',
 'Casualties->Missing',
 'Context->Demography',
 'Context->Economy',
 'Context->Environment',
 'Contex

In [103]:
entries_containing_numbers

['At Risk->Number Of People At Risk',
 'Capacities & Response->Number Of People Reached/Response Gaps',
 'Humanitarian Conditions->Number Of People In Need',
 'Impact->Number Of People Affected',
 'Displacement->Type/Numbers/Movements',
 'Humanitarian Access->Number of people facing humanitarian access constraints/Humanitarian access gaps']

In [ ]:
## subpillars 2d and 1d without numbers aug

In [ ]:
rchmetrics-0.4.1 tqdm-4.41.1 transformers-4.8.2 typing-extensions-3.7.4.3 wrapt-1.12.1 yarl-1.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.19.110 requires botocore==1.20.110, but you have botocore 1.20.106 which is incompatible.
WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv

2021-10-05 19:47:19,703 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_pytorch_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "dropout_rate": 0.3,
        "experiment_name": "pl-trials",
        "proportion_negative_examples_train_df": 0.25,
        "model_mode": "train",
        "max_len": 256,
        "training_names": "subpillars_1d,subpillars_2d",
        "model_name": "microsoft/xtremedistil-l6-h256-uncased",
        "output_length": 256,
        "tokenizer_name": "microsoft/xtremedistil-l6-h256-uncased",
        "epochs": 3,
        "learning_rate": 7e-05,
        "pred_threshold": 0.4,
        "tracking_uri": "http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/"
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "test": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        },
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "pytorch-2021-10-05-21-33-45-367-all-models",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_dir": "/opt/ml/model",
    "module_dir": "s3://sagemaker-deep-experiments-dev/training/input_data/pytorch-2021-10-05-21-33-45-367-all-models/pytorch-2021-10-05-21-33-45-367-all-models/source/sourcedir.tar.gz",
    "module_name": "train_mlflow",
    "network_interface_name": "eth0",
    "num_cpus": 8,
    "num_gpus": 1,
    "output_data_dir": "/opt/ml/output/data",
    "output_dir": "/opt/ml/output",
    "output_intermediate_dir": "/opt/ml/output/intermediate",
    "resource_config": {
        "current_host": "algo-1",
        "hosts": [
            "algo-1"
        ],
        "network_interface_name": "eth0"
    },
    "user_entry_point": "train_mlflow.py"
}

Environment variables:

SM_HOSTS=["algo-1"]
SM_NETWORK_INTERFACE_NAME=eth0
SM_HPS={"dropout_rate":0.3,"epochs":3,"experiment_name":"pl-trials","learning_rate":7e-05,"max_len":256,"model_mode":"train","model_name":"microsoft/xtremedistil-l6-h256-uncased","output_length":256,"pred_threshold":0.4,"proportion_negative_examples_train_df":0.25,"tokenizer_name":"microsoft/xtremedistil-l6-h256-uncased","tracking_uri":"http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/","training_names":"subpillars_1d,subpillars_2d"}
SM_USER_ENTRY_POINT=train_mlflow.py
SM_FRAMEWORK_PARAMS={}
SM_RESOURCE_CONFIG={"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}
SM_INPUT_DATA_CONFIG={"test":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"},"train":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"}}
SM_OUTPUT_DATA_DIR=/opt/ml/output/data
SM_CHANNELS=["test","train"]
SM_CURRENT_HOST=algo-1
SM_MODULE_NAME=train_mlflow
SM_LOG_LEVEL=20
SM_FRAMEWORK_MODULE=sagemaker_pytorch_container.training:main
SM_INPUT_DIR=/opt/ml/input
SM_INPUT_CONFIG_DIR=/opt/ml/input/config
SM_OUTPUT_DIR=/opt/ml/output
SM_NUM_CPUS=8
SM_NUM_GPUS=1
SM_MODEL_DIR=/opt/ml/model
SM_MODULE_DIR=s3://sagemaker-deep-experiments-dev/training/input_data/pytorch-2021-10-05-21-33-45-367-all-models/pytorch-2021-10-05-21-33-45-367-all-models/source/sourcedir.tar.gz
SM_TRAINING_ENV={"additional_framework_parameters":{},"channel_input_dirs":{"test":"/opt/ml/input/data/test","train":"/opt/ml/input/data/train"},"current_host":"algo-1","framework_module":"sagemaker_pytorch_container.training:main","hosts":["algo-1"],"hyperparameters":{"dropout_rate":0.3,"epochs":3,"experiment_name":"pl-trials","learning_rate":7e-05,"max_len":256,"model_mode":"train","model_name":"microsoft/xtremedistil-l6-h256-uncased","output_length":256,"pred_threshold":0.4,"proportion_negative_examples_train_df":0.25,"tokenizer_name":"microsoft/xtremedistil-l6-h256-uncased","tracking_uri":"http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/","training_names":"subpillars_1d,subpillars_2d"},"input_config_dir":"/opt/ml/input/config","input_data_config":{"test":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"},"train":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"}},"input_dir":"/opt/ml/input","is_master":true,"job_name":"pytorch-2021-10-05-21-33-45-367-all-models","log_level":20,"master_hostname":"algo-1","model_dir":"/opt/ml/model","module_dir":"s3://sagemaker-deep-experiments-dev/training/input_data/pytorch-2021-10-05-21-33-45-367-all-models/pytorch-2021-10-05-21-33-45-367-all-models/source/sourcedir.tar.gz","module_name":"train_mlflow","network_interface_name":"eth0","num_cpus":8,"num_gpus":1,"output_data_dir":"/opt/ml/output/data","output_dir":"/opt/ml/output","output_intermediate_dir":"/opt/ml/output/intermediate","resource_config":{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"},"user_entry_point":"train_mlflow.py"}
SM_USER_ARGS=["--dropout_rate","0.3","--epochs","3","--experiment_name","pl-trials","--learning_rate","7e-05","--max_len","256","--model_mode","train","--model_name","microsoft/xtremedistil-l6-h256-uncased","--output_length","256","--pred_threshold","0.4","--proportion_negative_examples_train_df","0.25","--tokenizer_name","microsoft/xtremedistil-l6-h256-uncased","--tracking_uri","http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/","--training_names","subpillars_1d,subpillars_2d"]
SM_OUTPUT_INTERMEDIATE_DIR=/opt/ml/output/intermediate
SM_CHANNEL_TEST=/opt/ml/input/data/test
SM_CHANNEL_TRAIN=/opt/ml/input/data/train
SM_HP_DROPOUT_RATE=0.3
SM_HP_EXPERIMENT_NAME=pl-trials
SM_HP_PROPORTION_NEGATIVE_EXAMPLES_TRAIN_DF=0.25
SM_HP_MODEL_MODE=train
SM_HP_MAX_LEN=256
SM_HP_TRAINING_NAMES=subpillars_1d,subpillars_2d
SM_HP_MODEL_NAME=microsoft/xtremedistil-l6-h256-uncased
SM_HP_OUTPUT_LENGTH=256
SM_HP_TOKENIZER_NAME=microsoft/xtremedistil-l6-h256-uncased
SM_HP_EPOCHS=3
SM_HP_LEARNING_RATE=7e-05
SM_HP_PRED_THRESHOLD=0.4
SM_HP_TRACKING_URI=http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/
PYTHONPATH=/opt/ml/code:/opt/conda/bin:/opt/conda/lib/python36.zip:/opt/conda/lib/python3.6:/opt/conda/lib/python3.6/lib-dynload:/opt/conda/lib/python3.6/site-packages

Invoking script with the following command:

/opt/conda/bin/python3.6 train_mlflow.py --dropout_rate 0.3 --epochs 3 --experiment_name pl-trials --learning_rate 7e-05 --max_len 256 --model_mode train --model_name microsoft/xtremedistil-l6-h256-uncased --output_length 256 --pred_threshold 0.4 --proportion_negative_examples_train_df 0.25 --tokenizer_name microsoft/xtremedistil-l6-h256-uncased --tracking_uri http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/ --training_names subpillars_1d,subpillars_2d


#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-10-05 19:51:48.073 algo-1:85 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-05 19:51:48.142 algo-1:85 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-10-05 19:51:48.143 algo-1:85 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-10-05 19:51:48.143 algo-1:85 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-10-05 19:51:48.144 algo-1:85 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-10-05 19:51:48.144 algo-1:85 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-10-05 19:51:48.349 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-10-05 19:51:48.349 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-10-05 19:51:48.349 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.token_type_embeddings.weight count_params:512
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.350 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.output.dense.weight count_params:262144
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.output.dense.bias count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.0.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.351 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.output.dense.weight count_params:262144
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.output.dense.bias count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.1.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.352 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.output.dense.weight count_params:262144
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.output.dense.bias count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.353 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.2.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.output.dense.weight count_params:262144
[2021-10-05 19:51:48.354 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.output.dense.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.3.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.355 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.output.dense.weight count_params:262144
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.output.dense.bias count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.4.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.query.weight count_params:65536
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.query.bias count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.key.weight count_params:65536
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.key.bias count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.value.weight count_params:65536
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.self.value.bias count_params:256
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.output.dense.weight count_params:65536
[2021-10-05 19:51:48.356 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.output.dense.bias count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.attention.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.intermediate.dense.weight count_params:262144
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.intermediate.dense.bias count_params:1024
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.output.dense.weight count_params:262144
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.output.dense.bias count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.output.LayerNorm.weight count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.encoder.layer.5.output.LayerNorm.bias count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.pooler.dense.weight count_params:65536
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l0.pooler.dense.bias count_params:256
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l1.weight count_params:38400
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l1.bias count_params:150
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l2.weight count_params:150
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l2.bias count_params:150
[2021-10-05 19:51:48.357 algo-1:85 INFO hook.py:594] name:model.l4.weight count_params:3450
[2021-10-05 19:51:48.358 algo-1:85 INFO hook.py:594] name:model.l4.bias count_params:23
[2021-10-05 19:51:48.358 algo-1:85 INFO hook.py:596] Total Trainable Params: 12792403
[2021-10-05 19:51:48.358 algo-1:85 INFO hook.py:423] Monitoring the collections: losses
[2021-10-05 19:51:48.361 algo-1:85 INFO hook.py:486] Hook is writing from the hook with pid: 85

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/3421 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/3421 [00:00<?, ?it/s] #015Epoch 0:   1%|          | 30/3421 [00:02<04:55, 11.49it/s]#015Epoch 0:   1%|          | 30/3421 [00:02<04:55, 11.49it/s, loss=25.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.411]#015Epoch 0:   2%|▏         | 60/3421 [00:04<04:10, 13.41it/s, loss=25.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.411]#015Epoch 0:   2%|▏         | 60/3421 [00:04<04:10, 13.41it/s, loss=8.92, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.546]#015Epoch 0:   3%|▎         | 90/3421 [00:06<03:51, 14.42it/s, loss=8.92, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.546]#015Epoch 0:   3%|▎         | 90/3421 [00:06<03:51, 14.42it/s, loss=6.43, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.487]#015Epoch 0:   4%|▎         | 120/3421 [00:08<03:44, 14.70it/s, loss=6.43, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.487]#015Epoch 0:   4%|▎         | 120/3421 [00:08<03:44, 14.70it/s, loss=5.59, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:   4%|▍         | 150/3421 [00:09<03:37, 15.05it/s, loss=5.59, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:   4%|▍         | 150/3421 [00:09<03:37, 15.05it/s, loss=4.52, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.491]#015Epoch 0:   5%|▌         | 180/3421 [00:12<03:36, 14.99it/s, loss=4.52, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.491]#015Epoch 0:   5%|▌         | 180/3421 [00:12<03:36, 14.99it/s, loss=3.64, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   6%|▌         | 210/3421 [00:13<03:32, 15.15it/s, loss=3.64, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   6%|▌         | 210/3421 [00:13<03:32, 15.15it/s, loss=3.17, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   7%|▋         | 240/3421 [00:15<03:27, 15.33it/s, loss=3.17, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   7%|▋         | 240/3421 [00:15<03:27, 15.33it/s, loss=3.15, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   8%|▊         | 270/3421 [00:17<03:24, 15.38it/s, loss=3.15, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   8%|▊         | 270/3421 [00:17<03:24, 15.38it/s, loss=2.75, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   9%|▉         | 300/3421 [00:19<03:21, 15.52it/s, loss=2.75, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:   9%|▉         | 300/3421 [00:19<03:21, 15.51it/s, loss=2.64, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  10%|▉         | 330/3421 [00:21<03:18, 15.56it/s, loss=2.64, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  10%|▉         | 330/3421 [00:21<03:18, 15.56it/s, loss=2.56, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:  11%|█         | 360/3421 [00:23<03:16, 15.60it/s, loss=2.56, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:  11%|█         | 360/3421 [00:23<03:16, 15.60it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.497]#015Epoch 0:  11%|█▏        | 390/3421 [00:24<03:13, 15.68it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.497]#015Epoch 0:  11%|█▏        | 390/3421 [00:24<03:13, 15.68it/s, loss=2.33, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  12%|█▏        | 420/3421 [00:26<03:11, 15.70it/s, loss=2.33, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  12%|█▏        | 420/3421 [00:26<03:11, 15.70it/s, loss=2.17, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  13%|█▎        | 450/3421 [00:28<03:08, 15.77it/s, loss=2.17, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  13%|█▎        | 450/3421 [00:28<03:08, 15.77it/s, loss=2.35, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  14%|█▍        | 480/3421 [00:30<03:06, 15.78it/s, loss=2.35, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  14%|█▍        | 480/3421 [00:30<03:06, 15.78it/s, loss=2.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.493]#015Epoch 0:  15%|█▍        | 510/3421 [00:32<03:04, 15.79it/s, loss=2.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.493]#015Epoch 0:  15%|█▍        | 510/3421 [00:32<03:04, 15.79it/s, loss=2.81, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  16%|█▌        | 540/3421 [00:34<03:01, 15.85it/s, loss=2.81, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  16%|█▌        | 540/3421 [00:34<03:01, 15.85it/s, loss=2.27, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  17%|█▋        | 570/3421 [00:35<02:59, 15.86it/s, loss=2.27, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  17%|█▋        | 570/3421 [00:35<02:59, 15.86it/s, loss=1.82, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:  18%|█▊        | 600/3421 [00:37<02:57, 15.91it/s, loss=1.82, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.489]#015Epoch 0:  18%|█▊        | 600/3421 [00:37<02:57, 15.91it/s, loss=2.18, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  18%|█▊        | 630/3421 [00:39<02:55, 15.91it/s, loss=2.18, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  18%|█▊        | 630/3421 [00:39<02:55, 15.91it/s, loss=2.63, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  19%|█▉        | 660/3421 [00:41<02:53, 15.92it/s, loss=2.63, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  19%|█▉        | 660/3421 [00:41<02:53, 15.92it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  20%|██        | 690/3421 [00:43<02:51, 15.95it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  20%|██        | 690/3421 [00:43<02:51, 15.95it/s, loss=2.13, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  21%|██        | 720/3421 [00:45<02:49, 15.97it/s, loss=2.13, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  21%|██        | 720/3421 [00:45<02:49, 15.97it/s, loss=2.21, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  22%|██▏       | 750/3421 [00:46<02:46, 16.01it/s, loss=2.21, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.492]#015Epoch 0:  22%|██▏       | 750/3421 [00:46<02:46, 16.01it/s, loss=3.04, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.521]#015Epoch 0:  23%|██▎       | 780/3421 [00:48<02:45, 16.00it/s, loss=3.04, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.521]#015Epoch 0:  23%|██▎       | 780/3421 [00:48<02:45, 16.00it/s, loss=2.58, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  24%|██▎       | 810/3421 [00:50<02:43, 16.00it/s, loss=2.58, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  24%|██▎       | 810/3421 [00:50<02:43, 16.00it/s, loss=2.14, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  25%|██▍       | 840/3421 [00:52<02:40, 16.03it/s, loss=2.14, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  25%|██▍       | 840/3421 [00:52<02:40, 16.03it/s, loss=1.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  25%|██▌       | 870/3421 [00:54<02:39, 16.04it/s, loss=1.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  25%|██▌       | 870/3421 [00:54<02:39, 16.04it/s, loss=2.03, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.501]#015Epoch 0:  26%|██▋       | 900/3421 [00:56<02:36, 16.06it/s, loss=2.03, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.501]#015Epoch 0:  26%|██▋       | 900/3421 [00:56<02:36, 16.06it/s, loss=2.42, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.505]#015Epoch 0:  27%|██▋       | 930/3421 [00:57<02:35, 16.06it/s, loss=2.42, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.505]#015Epoch 0:  27%|██▋       | 930/3421 [00:57<02:35, 16.06it/s, loss=2.7, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587] #015Epoch 0:  28%|██▊       | 960/3421 [00:59<02:33, 16.05it/s, loss=2.7, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587]#015Epoch 0:  28%|██▊       | 960/3421 [00:59<02:33, 16.05it/s, loss=2.55, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.503]#015Epoch 0:  29%|██▉       | 990/3421 [01:01<02:31, 16.07it/s, loss=2.55, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.503]#015Epoch 0:  29%|██▉       | 990/3421 [01:01<02:31, 16.07it/s, loss=1.89, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.534]#015Epoch 0:  30%|██▉       | 1020/3421 [01:03<02:29, 16.07it/s, loss=1.89, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.534]#015Epoch 0:  30%|██▉       | 1020/3421 [01:03<02:29, 16.07it/s, loss=1.98, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.540]#015Epoch 0:  31%|███       | 1050/3421 [01:05<02:27, 16.10it/s, loss=1.98, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.540]#015Epoch 0:  31%|███       | 1050/3421 [01:05<02:27, 16.10it/s, loss=2.45, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  32%|███▏      | 1080/3421 [01:07<02:25, 16.07it/s, loss=2.45, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.488]#015Epoch 0:  32%|███▏      | 1080/3421 [01:07<02:25, 16.07it/s, loss=2.04, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  32%|███▏      | 1110/3421 [01:09<02:24, 16.04it/s, loss=2.04, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  32%|███▏      | 1110/3421 [01:09<02:24, 16.04it/s, loss=2.16, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.516]#015Epoch 0:  33%|███▎      | 1140/3421 [01:11<02:22, 16.06it/s, loss=2.16, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.516]#015Epoch 0:  33%|███▎      | 1140/3421 [01:11<02:22, 16.06it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  34%|███▍      | 1170/3421 [01:12<02:20, 16.05it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.490]#015Epoch 0:  34%|███▍      | 1170/3421 [01:12<02:20, 16.05it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.553]#015Epoch 0:  35%|███▌      | 1200/3421 [01:14<02:18, 16.07it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.553]#015Epoch 0:  35%|███▌      | 1200/3421 [01:14<02:18, 16.07it/s, loss=1.96, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.539]#015Epoch 0:  36%|███▌      | 1230/3421 [01:16<02:16, 16.07it/s, loss=1.96, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.539]#015Epoch 0:  36%|███▌      | 1230/3421 [01:16<02:16, 16.07it/s, loss=2.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.522] #015Epoch 0:  37%|███▋      | 1260/3421 [01:18<02:14, 16.06it/s, loss=2.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.522]#015Epoch 0:  37%|███▋      | 1260/3421 [01:18<02:14, 16.06it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.524]#015Epoch 0:  38%|███▊      | 1290/3421 [01:20<02:12, 16.07it/s, loss=2.47, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.524]#015Epoch 0:  38%|███▊      | 1290/3421 [01:20<02:12, 16.07it/s, loss=2.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.536] #015Epoch 0:  39%|███▊      | 1320/3421 [01:22<02:10, 16.07it/s, loss=2.5, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.536]#015Epoch 0:  39%|███▊      | 1320/3421 [01:22<02:10, 16.07it/s, loss=2.41, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.558]#015Epoch 0:  39%|███▉      | 1350/3421 [01:23<02:08, 16.09it/s, loss=2.41, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.558]#015Epoch 0:  39%|███▉      | 1350/3421 [01:23<02:08, 16.09it/s, loss=2.37, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.536]#015Epoch 0:  40%|████      | 1380/3421 [01:25<02:06, 16.09it/s, loss=2.37, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.536]#015Epoch 0:  40%|████      | 1380/3421 [01:25<02:06, 16.09it/s, loss=2.07, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.543]#015Epoch 0:  41%|████      | 1410/3421 [01:27<02:04, 16.10it/s, loss=2.07, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.543]#015Epoch 0:  41%|████      | 1410/3421 [01:27<02:04, 16.10it/s, loss=2.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.541]#015Epoch 0:  42%|████▏     | 1440/3421 [01:29<02:02, 16.11it/s, loss=2.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.541]#015Epoch 0:  42%|████▏     | 1440/3421 [01:29<02:02, 16.11it/s, loss=2.54, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.554]#015Epoch 0:  43%|████▎     | 1470/3421 [01:31<02:01, 16.10it/s, loss=2.54, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.554]#015Epoch 0:  43%|████▎     | 1470/3421 [01:31<02:01, 16.10it/s, loss=2.16, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.551]#015Epoch 0:  44%|████▍     | 1500/3421 [01:33<01:59, 16.10it/s, loss=2.16, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.551]#015Epoch 0:  44%|████▍     | 1500/3421 [01:33<01:59, 16.10it/s, loss=2.13, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.531]#015Epoch 0:  45%|████▍     | 1530/3421 [01:35<01:57, 16.10it/s, loss=2.13, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.531]#015Epoch 0:  45%|████▍     | 1530/3421 [01:35<01:57, 16.10it/s, loss=2.4, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.497] #015Epoch 0:  46%|████▌     | 1560/3421 [01:36<01:55, 16.10it/s, loss=2.4, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.497]#015Epoch 0:  46%|████▌     | 1560/3421 [01:36<01:55, 16.10it/s, loss=1.73, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.529]#015Epoch 0:  46%|████▋     | 1590/3421 [01:38<01:53, 16.10it/s, loss=1.73, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.529]#015Epoch 0:  46%|████▋     | 1590/3421 [01:38<01:53, 16.10it/s, loss=1.79, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.606]#015Epoch 0:  47%|████▋     | 1620/3421 [01:40<01:51, 16.10it/s, loss=1.79, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.606]#015Epoch 0:  47%|████▋     | 1620/3421 [01:40<01:51, 16.10it/s, loss=2.12, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.508]#015Epoch 0:  48%|████▊     | 1650/3421 [01:42<01:49, 16.12it/s, loss=2.12, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.508]#015Epoch 0:  48%|████▊     | 1650/3421 [01:42<01:49, 16.12it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.531]#015Epoch 0:  49%|████▉     | 1680/3421 [01:44<01:48, 16.11it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.531]#015Epoch 0:  49%|████▉     | 1680/3421 [01:44<01:48, 16.11it/s, loss=1.65, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.556]#015Epoch 0:  50%|█�

�██▉     | 1710/3421 [01:46<01:46, 16.11it/s, loss=1.65, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.556]#015Epoch 0:  50%|████▉     | 1710/3421 [01:46<01:46, 16.11it/s, loss=2.07, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.552]#015Epoch 0:  51%|█████     | 1740/3421 [01:47<01:44, 16.12it/s, loss=2.07, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.552]#015Epoch 0:  51%|█████     | 1740/3421 [01:47<01:44, 16.12it/s, loss=1.78, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.586]#015Epoch 0:  52%|█████▏    | 1770/3421 [01:49<01:42, 16.12it/s, loss=1.78, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.586]#015Epoch 0:  52%|█████▏    | 1770/3421 [01:49<01:42, 16.12it/s, loss=2.31, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.547]#015Epoch 0:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.13it/s, loss=2.31, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.547]#015Epoch 0:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.13it/s, loss=1.95, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.553]#015Epoch 0:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.12it/s, loss=1.95, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.553]#015Epoch 0:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.12it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.506]#015Epoch 0:  54%|█████▍    | 1860/3421 [01:55<01:36, 16.12it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.506]#015Epoch 0:  54%|█████▍    | 1860/3421 [01:55<01:36, 16.12it/s, loss=2.11, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  55%|█████▌    | 1890/3421 [01:57<01:34, 16.13it/s, loss=2.11, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  55%|█████▌    | 1890/3421 [01:57<01:34, 16.13it/s, loss=1.93, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.581]#015Epoch 0:  56%|█████▌    | 1920/3421 [01:59<01:33, 16.12it/s, loss=1.93, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.581]#015Epoch 0:  56%|█████▌    | 1920/3421 [01:59<01:33, 16.12it/s, loss=1.88, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.511]#015Epoch 0:  57%|█████▋    | 1950/3421 [02:00<01:31, 16.13it/s, loss=1.88, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.511]#015Epoch 0:  57%|█████▋    | 1950/3421 [02:00<01:31, 16.13it/s, loss=1.99, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.588]#015Epoch 0:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.13it/s, loss=1.99, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.588]#015Epoch 0:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.13it/s, loss=1.95, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.582]#015Epoch 0:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.13it/s, loss=1.95, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.582]#015Epoch 0:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.13it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.14it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.14it/s, loss=1.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.590]#015Epoch 0:  61%|██████    | 2070/3421 [02:08<01:23, 16.13it/s, loss=1.94, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.590]#015Epoch 0:  61%|██████    | 2070/3421 [02:08<01:23, 16.13it/s, loss=2.06, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587]#015Epoch 0:  61%|██████▏   | 2100/3421 [02:10<01:21, 16.13it/s, loss=2.06, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587]#015Epoch 0:  61%|██████▏   | 2100/3421 [02:10<01:21, 16.13it/s, loss=1.88, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.598]#015Epoch 0:  62%|██████▏   | 2130/3421 [02:12<01:20, 16.11it/s, loss=1.88, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.598]#015Epoch 0:  62%|██████▏   | 2130/3421 [02:12<01:20, 16.11it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.574]#015Epoch 0:  63%|██████▎   | 2160/3421 [02:14<01:18, 16.11it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.574]#015Epoch 0:  63%|██████▎   | 2160/3421 [02:14<01:18, 16.11it/s, loss=1.89, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.594]#015Epoch 0:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.12it/s, loss=1.89, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.594]#015Epoch 0:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.12it/s, loss=2.18, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.650]#015Epoch 0:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.12it/s, loss=2.18, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.650]#015Epoch 0:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.12it/s, loss=1.46, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.604]#015Epoch 0:  66%|██████▌   | 2250/3421 [02:19<01:12, 16.13it/s, loss=1.46, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.604]#015Epoch 0:  66%|██████▌   | 2250/3421 [02:19<01:12, 16.13it/s, loss=2.28, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.639]#015Epoch 0:  67%|██████▋   | 2280/3421 [02:21<01:10, 16.13it/s, loss=2.28, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.639]#015Epoch 0:  67%|██████▋   | 2280/3421 [02:21<01:10, 16.13it/s, loss=2.61, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.584]#015Epoch 0:  68%|██████▊   | 2310/3421 [02:23<01:08, 16.13it/s, loss=2.61, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.584]#015Epoch 0:  68%|██████▊   | 2310/3421 [02:23<01:08, 16.13it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  68%|██████▊   | 2340/3421 [02:24<01:06, 16.14it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.507]#015Epoch 0:  68%|██████▊   | 2340/3421 [02:24<01:06, 16.14it/s, loss=1.8, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.690] #015Epoch 0:  69%|██████▉   | 2370/3421 [02:26<01:05, 16.14it/s, loss=1.8, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.690]#015Epoch 0:  69%|██████▉   | 2370/3421 [02:26<01:05, 16.14it/s, loss=1.65, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.577]#015Epoch 0:  70%|███████   | 2400/3421 [02:28<01:03, 16.15it/s, loss=1.65, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.577]#015Epoch 0:  70%|███████   | 2400/3421 [02:28<01:03, 16.15it/s, loss=1.93, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.602]#015Epoch 0:  71%|███████   | 2430/3421 [02:30<01:01, 16.14it/s, loss=1.93, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.602]#015Epoch 0:  71%|███████   | 2430/3421 [02:30<01:01, 16.14it/s, loss=1.56, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.515]#015Epoch 0:  72%|███████▏  | 2460/3421 [02:32<00:59, 16.14it/s, loss=1.56, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.515]#015Epoch 0:  72%|███████▏  | 2460/3421 [02:32<00:59, 16.14it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.594]#015Epoch 0:  73%|███████▎  | 2490/3421 [02:34<00:57, 16.15it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.594]#015Epoch 0:  73%|███████▎  | 2490/3421 [02:34<00:57, 16.15it/s, loss=1.76, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.643]#015Epoch 0:  74%|███████▎  | 2520/3421 [02:36<00:55, 16.14it/s, loss=1.76, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.643]#015Epoch 0:  74%|███████▎  | 2520/3421 [02:36<00:55, 16.14it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  75%|███████▍  | 2550/3421 [02:37<00:53, 16.15it/s, loss=2.25, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.608]#015Epoch 0:  75%|███████▍  | 2550/3421 [02:37<00:53, 16.15it/s, loss=1.98, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587]#015Epoch 0:  75%|███████▌  | 2580/3421 [02:39<00:52, 16.15it/s, loss=1.98, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.587]#015Epoch 0:  75%|███████▌  | 2580/3421 [02:39<00:52, 16.15it/s, loss=1.68, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.601]#015Epoch 0:  76%|███████▋  | 2610/3421 [02:41<00:50, 16.15it/s, loss=1.68, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.601]#015Epoch 0:  76%|███████▋  | 2610/3421 [02:41<00:50, 16.15it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.646]#015Epoch 0:  77%|███████▋  | 2640/3421 [02:43<00:48, 16.15it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.646]#015Epoch 0:  77%|███████▋  | 2640/3421 [02:43<00:48, 16.15it/s, loss=1.55, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.597]#015Epoch 0:  78%|███████▊  | 2670/3421 [02:45<00:46, 16.15it/s, loss=1.55, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.597]#015Epoch 0:  78%|███████▊  | 2670/3421 [02:45<00:46, 16.15it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.612]#015Epoch 0:  79%|███████▉  | 2700/3421 [02:47<00:44, 16.16it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.612]#015Epoch 0:  79%|███████▉  | 2700/3421 [02:47<00:44, 16.16it/s, loss=1.92, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.598]#015Epoch 0:  80%|███████▉  | 2730/3421 [02:48<00:42, 16.16it/s, loss=1.92, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.598]#015Epoch 0:  80%|███████▉  | 2730/3421 [02:48<00:42, 16.16it/s, loss=1.96, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.572]#015Epoch 0:  81%|████████  | 2760/3421 [02:50<00:40, 16.16it/s, loss=1.96, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.572]#015Epoch 0:  81%|████████  | 2760/3421 [02:50<00:40, 16.16it/s, loss=1.52, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.577]#015Epoch 0:  82%|████████▏ | 2790/3421 [02:52<00:39, 16.17it/s, loss=1.52, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.577]#015Epoch 0:  82%|████████▏ | 2790/3421 [02:52<00:39, 16.17it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.590]#015Epoch 0:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.17it/s, loss=1.66, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.590]#015Epoch 0:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.17it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.675]#015Epoch 0:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.18it/s, loss=1.77, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.675]#015Epoch 0:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.18it/s, loss=1.6, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.640] #015Epoch 0:  84%|████████▍ | 2880/3421 [02:58<00:33, 16.17it/s, loss=1.6, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.640]#015Epoch 0:  84%|████████▍ | 2880/3421 [02:58<00:33, 16.17it/s, loss=2.01, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.685]#015Epoch 0:  85%|████████▌ | 2910/3421 [02:59<00:31, 16.17it/s, loss=2.01, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.685]#015Epoch 0:  85%|████████▌ | 2910/3421 [02:59<00:31, 16.17it/s, loss=1.84, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.623]#015Epoch 0:  86%|████████▌ | 2940/3421 [03:01<00:29, 16.18it/s, loss=1.84, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.623]#015Epoch 0:  86%|████████▌ | 2940/3421 [03:01<00:29, 16.18it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.646]#015Epoch 0:  87%|████████▋ | 2970/3421 [03:03<00:27, 16.18it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.646]#015Epoch 0:  87%|████████▋ | 2970/3421 [03:03<00:27, 16.18it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.669]#015Epoch 0:  88%|████████▊ | 3000/3421 [03:05<00:26, 16.19it/s, loss=2.05, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.669]#015Epoch 0:  88%|████████▊ | 3000/3421 [03:05<00:26, 16.19it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]#015Epoch 0:  89%|████████▊ | 3030/3421 [03:07<00:24, 16.20it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/397 [00:00<?, ?it/s]#033[A
#015Validating:   8%|▊         | 30/397 [00:00<00:11, 31.63it/s]#033[A#015Epoch 0:  89%|████████▉ | 3060/3421 [03:08<00:22, 16.27it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  15%|█▌        | 60/397 [00:01<00:10, 32.99it/s]#033[A#015Epoch 0:  90%|█████████ | 3090/3421 [03:08<00:20, 16.36it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  23%|██▎       | 90/397 [00:02<00:09, 33.63it/s]#033[A#015Epoch 0:  91%|█████████ | 3120/3421 [03:09<00:18, 16.45it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  30%|███       | 120/397 [00:03<00:07, 35.55it/s]#033[A#015Epoch 0:  92%|█████████▏| 3150/3421 [03:10<00:16, 16.54it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  38%|███▊      | 150/397 [00:04<00:06, 37.50it/s]#033[A#015Epoch 0:  93%|█████████▎| 3180/3421 [03:11<00:14, 16.64it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  45%|████▌     | 180/397 [00:04<00:05, 39.19it/s]#033[A#015Epoch 0:  94%|█████████▍| 3210/3421 [03:11<00:12, 16.73it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  53%|█████▎    | 210/397 [00:05<00:04, 40.12it/s]#033[A#015Epoch 0:  95%|█████████▍| 3240/3421 [03:12<00:10, 16.83it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  60%|██████    | 240/397 [00:06<00:03, 40.39it/s]#033[A#015Epoch 0:  96%|█████████▌| 3270/3421 [03:13<00:08, 16.92it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  68%|██████▊   | 270/397 [00:06<00:03, 41.28it/s]#033[A#015Epoch 0:  96%|█████████▋| 3300/3421 [03:13<00:07, 17.02it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  76%|███████▌  | 300/397 [00:07<00:02, 41.80it/s]#033[A#015Epoch 0:  97%|█████████▋| 3330/3421 [03:14<00:05, 17.11it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  83%|████████▎ | 330/397 [00:08<00:01, 41.84it/s]#033[A#015Epoch 0:  98%|█████████▊| 3360/3421 [03:15<00:03, 17.20it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]

#015Validating:  91%|█████████ | 360/397 [00:08<00:00, 42.22it/s]#033[A#015Epoch 0:  99%|█████████▉| 3390/3421 [03:16<00:01, 17.29it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating:  98%|█████████▊| 390/397 [00:09<00:00, 42.20it/s]#033[A#015Epoch 0: 100%|█████████▉| 3420/3421 [03:16<00:00, 17.38it/s, loss=1.67, v_num=0, val_f1_epoch=0.0359, val_loss_epoch=0.691, train_f1=0.634]
#015Validating: 100%|██████████| 397/397 [00:09<00:00, 42.93it/s]#033[A#015Epoch 0: 100%|██████████| 3421/3421 [03:17<00:00, 17.36it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.830, val_f1_step=0.496, val_loss_step=0.0406]
#015                                                             #033[A#015Epoch 0:   0%|          | 0/3421 [00:00<?, ?it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.830, val_f1_step=0.496, val_loss_step=0.0406]           #015Epoch 1:   0%|          | 0/3421 [00:00<?, ?it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.830, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   1%|          | 30/3421 [00:02<03:55, 14.42it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.830, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   1%|          | 30/3421 [00:02<03:55, 14.42it/s, loss=1.9, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:   2%|▏         | 60/3421 [00:03<03:34, 15.64it/s, loss=1.9, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   2%|▏         | 60/3421 [00:03<03:34, 15.64it/s, loss=1.84, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   3%|▎         | 90/3421 [00:05<03:30, 15.83it/s, loss=1.84, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   3%|▎         | 90/3421 [00:05<03:30, 15.83it/s, loss=1.91, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   4%|▎         | 120/3421 [00:07<03:25, 16.05it/s, loss=1.91, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   4%|▎         | 120/3421 [00:07<03:25, 16.05it/s, loss=1.61, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   4%|▍         | 150/3421 [00:09<03:23, 16.05it/s, loss=1.61, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   4%|▍         | 150/3421 [00:09<03:23, 16.05it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.612, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   5%|▌         | 180/3421 [00:11<03:22, 16.01it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.612, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   5%|▌         | 180/3421 [00:11<03:22, 16.01it/s, loss=1.6, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:   6%|▌         | 210/3421 [00:13<03:19, 16.14it/s, loss=1.6, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   6%|▌         | 210/3421 [00:13<03:19, 16.13it/s, loss=1.76, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   7%|▋         | 240/3421 [00:14<03:17, 16.09it/s, loss=1.76, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   7%|▋         | 240/3421 [00:14<03:17, 16.09it/s, loss=1.42, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   8%|▊         | 270/3421 [00:16<03:15, 16.09it/s, loss=1.42, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   8%|▊         | 270/3421 [00:16<03:15, 16.09it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   9%|▉         | 300/3421 [00:18<03:14, 16.05it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:   9%|▉         | 300/3421 [00:18<03:14, 16.05it/s, loss=1.66, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  10%|▉         | 330/3421 [00:20<03:12, 16.03it/s, loss=1.66, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  10%|▉         | 330/3421 [00:20<03:12, 16.03it/s, loss=1.3, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  11%|█         | 360/3421 [00:22<03:10, 16.10it/s, loss=1.3, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  11%|█         | 360/3421 [00:22<03:10, 16.10it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  11%|█▏        | 390/3421 [00:24<03:08, 16.12it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  11%|█▏        | 390/3421 [00:24<03:08, 16.12it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.696, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  12%|█▏        | 420/3421 [00:25<03:05, 16.19it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.696, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  12%|█▏        | 420/3421 [00:25<03:05, 16.19it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  13%|█▎        | 450/3421 [00:27<03:03, 16.19it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  13%|█▎        | 450/3421 [00:27<03:03, 16.19it/s, loss=2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]   #015Epoch 1:  14%|█▍        | 480/3421 [00:29<03:01, 16.19it/s, loss=2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  14%|█▍        | 480/3421 [00:29<03:01, 16.19it/s, loss=1.53, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  15%|█▍        | 510/3421 [00:31<02:59, 16.24it/s, loss=1.53, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  15%|█▍        | 510/3421 [00:31<02:59, 16.24it/s, loss=1.77, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  16%|█▌        | 540/3421 [00:33<02:57, 16.23it/s, loss=1.77, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  16%|█▌        | 540/3421 [00:33<02:57, 16.23it/s, loss=1.66, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.547, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  17%|█▋        | 570/3421 [00:35<02:55, 16.27it/s, loss=1.66, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.547, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  17%|█▋        | 570/3421 [00:35<02:55, 16.27it/s, loss=1.74, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  18%|█▊        | 600/3421 [00:36<02:53, 16.26it/s, loss=1.74, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  18%|█▊        | 600/3421 [00:36<02:53, 16.26it/s, loss=1.97, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.589, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  18%|█▊        | 630/3421 [00:38<02:51, 16.25it/s, loss=1.97, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.589, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  18%|█▊        | 630/3421 [00:38<02:51, 16.25it/s, loss=1.58, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  19%|█▉        | 660/3421 [00:40<02:49, 16.28it/s, loss=1.58, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  19%|█▉        | 660/3421 [00:40<02:49, 16.28it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  20%|██        | 690/3421 [00:42<02:47, 16.27it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  20%|██        | 690/3421 [00:42<02:47, 16.27it/s, loss=1.92, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  21%|██        | 720/3421 [00:44<02:45, 16.29it/s, loss=1.92, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  21%|██        | 720/3421 [00:44<02:45, 16.29it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  22%|██▏       | 750/3421 [00:46<02:44, 16.26it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  22%|██▏       | 750/3421 [00:46<02:44, 16.26it/s, loss=1.67, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  23%|██▎       | 780/3421 [00:48<02:42, 16.25it/s, loss=1.67, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  23%|██▎       | 780/3421 [00:48<02:42, 16.24it/s, loss=1.73, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  24%|██▎       | 810/3421 [00:49<02:40, 16.26it/s, loss=1.73, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  24%|██▎       | 810/3421 [00:49<02:40, 16.26it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  25%|██▍       | 840/3421 [00:51<02:38, 16.23it/s, loss=1.82, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  25%|██▍       | 840/3421 [00:51<02:38, 16.23it/s, loss=1.9, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  25%|██▌       | 870/3421 [00:53<02:37, 16.21it/s, loss=1.9, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  25%|██▌       | 870/3421 [00:53<02:37, 16.21it/s, loss=1.83, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  26%|██▋       | 900/3421 [00:55<02:35, 16.21it/s, loss=1.83, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  26%|██▋       | 900/3421 [00:55<02:35, 16.21it/s, loss=1.63, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  27%|██▋       | 930/3421 [00:57<02:33, 16.20it/s, loss=1.63, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  27%|██▋       | 930/3421 [00:57<02:33, 16.20it/s, loss=1.28, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  28%|██▊       | 960/3421 [00:59<02:31, 16.22it/s, loss=1.28, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  28%|██▊       | 960/3421 [00:59<02:31, 16.22it/s, loss=1.33, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.537, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  29%|██▉       | 990/3421 [01:01<02:29, 16.21it/s, loss=1.33, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.537, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  29%|██▉       | 990/3421 [01:01<02:29, 16.21it/s, loss=1.54, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  30%|██▉       | 1020/3421 [01:02<02:27, 16.23it/s, loss=1.54, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  30%|██▉       | 1020/3421 [01:02<02:27, 16.23it/s, loss=1.53, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  31%|███       | 1050/3421 [01:04<02:26, 16.22it/s, loss=1.53, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  31%|███       | 1050/3421 [01:04<02:26, 16.22it/s, loss=1.38, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  32%|███▏      | 1080/3421 [01:06<02:24, 16.22it/s, loss=1.38, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  32%|███▏      | 1080/3421 [01:06<02:24, 16.22it/s, loss=1.63, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  32%|███▏      | 1110/3421 [01:08<02:22, 16.22it/s, loss=1.63, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  32%|███▏      | 1110/3421 [01:08<02:22, 16.22it/s, loss=2.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  33%|███▎      | 1140/3421 [01:10<02:20, 16.22it/s, loss=2.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  33%|███▎      | 1140/3421 [01:10<02:20, 16.22it/s, loss=1.64, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.587, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  34%|███▍      | 1170/3421 [01:12<02:18, 16.24it/s, loss=1.64, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.587, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  34%|███▍      | 1170/3421 [01:12<02:18, 16.24it/s, loss=1.43, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  35%|███▌      | 1200/3421 [01:13<02:16, 16.22it/s, loss=1.43, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  35%|███▌      | 1200/3421 [01:13<02:16, 16.22it/s, loss=1.57, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  36%|███▌      | 1230/3421 [01:15<02:15, 16.20it/s, loss=1.57, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  36%|███▌      | 1230/3421 [01:15<02:15, 16.20it/s, loss=1.24, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.622, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  37%|███▋      | 1260/3421 [01:17<02:13, 16.21it/s, loss=1.24, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.622, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  37%|███▋      | 1260/3421 [01:17<02:13, 16.21it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  38%|███▊      | 1290/3421 [01:19<02:11, 16.20it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  38%|███▊      | 1290/3421 [01:19<02:11, 16.20it/s, loss=1.35, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  39%|███▊      | 1320/3421 [01:21<02:09, 16.21it/s, loss=1.35, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.688, val_f1_step=0.496, va

l_loss_step=0.0406]#015Epoch 1:  39%|███▊      | 1320/3421 [01:21<02:09, 16.21it/s, loss=2.2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  39%|███▉      | 1350/3421 [01:23<02:07, 16.20it/s, loss=2.2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  39%|███▉      | 1350/3421 [01:23<02:07, 16.20it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  40%|████      | 1380/3421 [01:25<02:06, 16.20it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  40%|████      | 1380/3421 [01:25<02:06, 16.20it/s, loss=1.75, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.701, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  41%|████      | 1410/3421 [01:26<02:04, 16.21it/s, loss=1.75, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.701, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  41%|████      | 1410/3421 [01:26<02:04, 16.21it/s, loss=1.59, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  42%|████▏     | 1440/3421 [01:28<02:02, 16.20it/s, loss=1.59, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  42%|████▏     | 1440/3421 [01:28<02:02, 16.20it/s, loss=1.65, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  43%|████▎     | 1470/3421 [01:30<02:00, 16.21it/s, loss=1.65, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  43%|████▎     | 1470/3421 [01:30<02:00, 16.21it/s, loss=1.73, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  44%|████▍     | 1500/3421 [01:32<01:58, 16.20it/s, loss=1.73, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  44%|████▍     | 1500/3421 [01:32<01:58, 16.20it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  45%|████▍     | 1530/3421 [01:34<01:56, 16.20it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  45%|████▍     | 1530/3421 [01:34<01:56, 16.20it/s, loss=1.75, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.685, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  46%|████▌     | 1560/3421 [01:36<01:54, 16.21it/s, loss=1.75, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.685, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  46%|████▌     | 1560/3421 [01:36<01:54, 16.21it/s, loss=1.71, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  46%|████▋     | 1590/3421 [01:38<01:52, 16.20it/s, loss=1.71, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  46%|████▋     | 1590/3421 [01:38<01:52, 16.20it/s, loss=1.52, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  47%|████▋     | 1620/3421 [01:39<01:51, 16.21it/s, loss=1.52, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  47%|████▋     | 1620/3421 [01:39<01:51, 16.21it/s, loss=1.98, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  48%|████▊     | 1650/3421 [01:41<01:49, 16.21it/s, loss=1.98, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  48%|████▊     | 1650/3421 [01:41<01:49, 16.21it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  49%|████▉     | 1680/3421 [01:43<01:47, 16.21it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  49%|████▉     | 1680/3421 [01:43<01:47, 16.21it/s, loss=1.44, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  50%|████▉     | 1710/3421 [01:45<01:45, 16.22it/s, loss=1.44, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  50%|████▉     | 1710/3421 [01:45<01:45, 16.22it/s, loss=1.5, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  51%|█████     | 1740/3421 [01:47<01:43, 16.21it/s, loss=1.5, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  51%|█████     | 1740/3421 [01:47<01:43, 16.21it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  52%|█████▏    | 1770/3421 [01:49<01:41, 16.21it/s, loss=1.69, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  52%|█████▏    | 1770/3421 [01:49<01:41, 16.21it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.18it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.18it/s, loss=1.4, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.685, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.17it/s, loss=1.4, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.685, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.17it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.625, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  54%|█████▍    | 1860/3421 [01:54<01:36, 16.18it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.625, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  54%|█████▍    | 1860/3421 [01:54<01:36, 16.18it/s, loss=1.83, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  55%|█████▌    | 1890/3421 [01:56<01:34, 16.17it/s, loss=1.83, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  55%|█████▌    | 1890/3421 [01:56<01:34, 16.17it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  56%|█████▌    | 1920/3421 [01:58<01:32, 16.18it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  56%|█████▌    | 1920/3421 [01:58<01:32, 16.18it/s, loss=1.42, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  57%|█████▋    | 1950/3421 [02:00<01:30, 16.18it/s, loss=1.42, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  57%|█████▋    | 1950/3421 [02:00<01:30, 16.18it/s, loss=1.4, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.18it/s, loss=1.4, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.18it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.19it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.19it/s, loss=1.91, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.738, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.18it/s, loss=1.91, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.738, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.18it/s, loss=1.54, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  61%|██████    | 2070/3421 [02:07<01:23, 16.19it/s, loss=1.54, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  61%|██████    | 2070/3421 [02:07<01:23, 16.19it/s, loss=1.59, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  61%|██████▏   | 2100/3421 [02:09<01:21, 16.18it/s, loss=1.59, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  61%|██████▏   | 2100/3421 [02:09<01:21, 16.18it/s, loss=1.27, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  62%|██████▏   | 2130/3421 [02:11<01:19, 16.18it/s, loss=1.27, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  62%|██████▏   | 2130/3421 [02:11<01:19, 16.18it/s, loss=1.88, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.691, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  63%|██████▎   | 2160/3421 [02:13<01:17, 16.19it/s, loss=1.88, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.691, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  63%|██████▎   | 2160/3421 [02:13<01:17, 16.19it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.19it/s, loss=1.29, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.19it/s, loss=1.36, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.708, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.19it/s, loss=1.36, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.708, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.19it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  66%|██████▌   | 2250/3421 [02:18<01:12, 16.19it/s, loss=1.46, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  66%|██████▌   | 2250/3421 [02:18<01:12, 16.19it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  67%|██████▋   | 2280/3421 [02:20<01:10, 16.19it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  67%|██████▋   | 2280/3421 [02:20<01:10, 16.19it/s, loss=1.48, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  68%|██████▊   | 2310/3421 [02:22<01:08, 16.20it/s, loss=1.48, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  68%|██████▊   | 2310/3421 [02:22<01:08, 16.20it/s, loss=1.89, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.690, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  68%|██████▊   | 2340/3421 [02:24<01:06, 16.19it/s, loss=1.89, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.690, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  68%|██████▊   | 2340/3421 [02:24<01:06, 16.19it/s, loss=1.19, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.613, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  69%|██████▉   | 2370/3421 [02:26<01:04, 16.20it/s, loss=1.19, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.613, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  69%|██████▉   | 2370/3421 [02:26<01:04, 16.20it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  70%|███████   | 2400/3421 [02:28<01:03, 16.20it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  70%|███████   | 2400/3421 [02:28<01:03, 16.20it/s, loss=1.27, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  71%|███████   | 2430/3421 [02:30<01:01, 16.19it/s, loss=1.27, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  71%|███████   | 2430/3421 [02:30<01:01, 16.19it/s, loss=1.31, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  72%|███████▏  | 2460/3421 [02:31<00:59, 16.20it/s, loss=1.31, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  72%|███████▏  | 2460/3421 [02:31<00:59, 16.20it/s, loss=2.01, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  73%|███████▎  | 2490/3421 [02:33<00:57, 16.20it/s, loss=2.01, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  73%|███████▎  | 2490/3421 [02:33<00:57, 16.20it/s, loss=2.02, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  74%|███████▎  | 2520/3421 [02:35<00:55, 16.20it/s, loss=2.02, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  74%|███████▎  | 2520/3421 [02:35<00:55, 16.20it/s, loss=1.38, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  75%|███████▍  | 2550/3421 [02:37<00:53, 16.20it/s, loss=1.38, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  75%|███████▍  | 2550/3421 [02:37<00:53, 16.20it/s, loss=1.61, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  75%|███████▌  | 2580/3421 [02:39<00:51, 16.20it/s, loss=1.61, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  75%|███████▌  | 2580/3421 [02:39<00:51, 16.20it/s, loss=1.28, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.682, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  76%|██████

█▋  | 2610/3421 [02:41<00:50, 16.21it/s, loss=1.28, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.682, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  76%|███████▋  | 2610/3421 [02:41<00:50, 16.21it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  77%|███████▋  | 2640/3421 [02:42<00:48, 16.20it/s, loss=1.41, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  77%|███████▋  | 2640/3421 [02:42<00:48, 16.20it/s, loss=1.65, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  78%|███████▊  | 2670/3421 [02:44<00:46, 16.21it/s, loss=1.65, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  78%|███████▊  | 2670/3421 [02:44<00:46, 16.21it/s, loss=1.7, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.719, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  79%|███████▉  | 2700/3421 [02:46<00:44, 16.20it/s, loss=1.7, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.719, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  79%|███████▉  | 2700/3421 [02:46<00:44, 16.20it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  80%|███████▉  | 2730/3421 [02:48<00:42, 16.20it/s, loss=1.51, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  80%|███████▉  | 2730/3421 [02:48<00:42, 16.20it/s, loss=1.58, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  81%|████████  | 2760/3421 [02:50<00:40, 16.20it/s, loss=1.58, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  81%|████████  | 2760/3421 [02:50<00:40, 16.20it/s, loss=1.74, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  82%|████████▏ | 2790/3421 [02:52<00:38, 16.19it/s, loss=1.74, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  82%|████████▏ | 2790/3421 [02:52<00:38, 16.19it/s, loss=1.87, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.716, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.19it/s, loss=1.87, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.716, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.19it/s, loss=1.3, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.0406] #015Epoch 1:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.18it/s, loss=1.3, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.18it/s, loss=1.14, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  84%|████████▍ | 2880/3421 [02:57<00:33, 16.18it/s, loss=1.14, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  84%|████████▍ | 2880/3421 [02:57<00:33, 16.18it/s, loss=2.77, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.740, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  85%|████████▌ | 2910/3421 [02:59<00:31, 16.19it/s, loss=2.77, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.740, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  85%|████████▌ | 2910/3421 [02:59<00:31, 16.19it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.703, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  86%|████████▌ | 2940/3421 [03:01<00:29, 16.19it/s, loss=1.56, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.703, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  86%|████████▌ | 2940/3421 [03:01<00:29, 16.19it/s, loss=1.68, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  87%|████████▋ | 2970/3421 [03:03<00:27, 16.20it/s, loss=1.68, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  87%|████████▋ | 2970/3421 [03:03<00:27, 16.20it/s, loss=2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.629, val_f1_step=0.496, val_loss_step=0.0406]   #015Epoch 1:  88%|████████▊ | 3000/3421 [03:05<00:25, 16.19it/s, loss=2, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.629, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  88%|████████▊ | 3000/3421 [03:05<00:25, 16.19it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]#015Epoch 1:  89%|████████▊ | 3030/3421 [03:06<00:24, 16.22it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/397 [00:00<?, ?it/s]#033[A
#015Validating:   8%|▊         | 30/397 [00:00<00:11, 32.68it/s]#033[A#015Epoch 1:  89%|████████▉ | 3060/3421 [03:07<00:22, 16.30it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  15%|█▌        | 60/397 [00:01<00:09, 35.12it/s]#033[A#015Epoch 1:  90%|█████████ | 3090/3421 [03:08<00:20, 16.40it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  23%|██▎       | 90/397 [00:02<00:08, 37.22it/s]#033[A#015Epoch 1:  91%|█████████ | 3120/3421 [03:09<00:18, 16.50it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  30%|███       | 120/397 [00:03<00:07, 38.17it/s]#033[A#015Epoch 1:  92%|█████████▏| 3150/3421 [03:09<00:16, 16.59it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  38%|███▊      | 150/397 [00:03<00:06, 39.61it/s]#033[A#015Epoch 1:  93%|█████████▎| 3180/3421 [03:10<00:14, 16.69it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  45%|████▌     | 180/397 [00:04<00:05, 40.32it/s]#033[A#015Epoch 1:  94%|█████████▍| 3210/3421 [03:11<00:12, 16.78it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  53%|█████▎    | 210/397 [00:05<00:04, 40.82it/s]#033[A#015Epoch 1:  95%|█████████▍| 3240/3421 [03:11<00:10, 16.88it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  60%|██████    | 240/397 [00:05<00:03, 41.25it/s]#033[A#015Epoch 1:  96%|█████████▌| 3270/3421 [03:12<00:08, 16.97it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  68%|██████▊   | 270/397 [00:06<00:03, 41.66it/s]#033[A#015Epoch 1:  96%|█████████▋| 3300/3421 [03:13<00:07, 17.06it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  76%|███████▌  | 300/397 [00:07<00:02, 41.47it/s]#033[A#015Epoch 1:  97%|█████████▋| 3330/3421 [03:14<00:05, 17.15it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  83%|████████▎ | 330/397 [00:08<00:01, 41.67it/s]#033[A#015Epoch 1:  98%|█████████▊| 3360/3421 [03:14<00:03, 17.25it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  91%|█████████ | 360/397 [00:08<00:00, 42.06it/s]#033[A#015Epoch 1:  99%|█████████▉| 3390/3421 [03:15<00:01, 17.34it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating:  98%|█████████▊| 390/397 [00:09<00:00, 42.35it/s]#033[A#015Epoch 1: 100%|█████████▉| 3420/3421 [03:16<00:00, 17.43it/s, loss=1.47, v_num=0, val_f1_epoch=0.629, val_loss_epoch=0.104, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0406]
#015Validating: 100%|██████████| 397/397 [00:09<00:00, 43.00it/s]#033[A#015Epoch 1: 100%|██████████| 3421/3421 [03:16<00:00, 17.41it/s, loss=1.35, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]

#015                                                             #033[A#015Epoch 1:   0%|          | 0/3421 [00:00<?, ?it/s, loss=1.35, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]           #015Epoch 2:   0%|          | 0/3421 [00:00<?, ?it/s, loss=1.35, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   1%|          | 30/3421 [00:02<04:38, 12.16it/s, loss=1.35, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   1%|          | 30/3421 [00:02<04:38, 12.16it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   2%|▏         | 60/3421 [00:04<04:01, 13.90it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   2%|▏         | 60/3421 [00:04<04:01, 13.90it/s, loss=1.33, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.749, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   3%|▎         | 90/3421 [00:06<03:45, 14.76it/s, loss=1.33, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.749, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   3%|▎         | 90/3421 [00:06<03:45, 14.75it/s, loss=1.34, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.702, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   4%|▎         | 120/3421 [00:07<03:38, 15.10it/s, loss=1.34, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.702, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   4%|▎         | 120/3421 [00:07<03:38, 15.10it/s, loss=2.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   4%|▍         | 150/3421 [00:09<03:31, 15.45it/s, loss=2.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   4%|▍         | 150/3421 [00:09<03:31, 15.45it/s, loss=1.25, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.732, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   5%|▌         | 180/3421 [00:11<03:28, 15.51it/s, loss=1.25, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.732, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   5%|▌         | 180/3421 [00:11<03:28, 15.51it/s, loss=1.34, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   6%|▌         | 210/3421 [00:13<03:26, 15.58it/s, loss=1.34, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   6%|▌         | 210/3421 [00:13<03:26, 15.58it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   7%|▋         | 240/3421 [00:15<03:22, 15.70it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   7%|▋         | 240/3421 [00:15<03:22, 15.70it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   8%|▊         | 270/3421 [00:17<03:20, 15.72it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   8%|▊         | 270/3421 [00:17<03:20, 15.72it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   9%|▉         | 300/3421 [00:18<03:17, 15.81it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:   9%|▉         | 300/3421 [00:18<03:17, 15.80it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.747, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  10%|▉         | 330/3421 [00:20<03:15, 15.83it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.747, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  10%|▉         | 330/3421 [00:20<03:15, 15.83it/s, loss=1.74, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  11%|█         | 360/3421 [00:22<03:13, 15.83it/s, loss=1.74, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  11%|█         | 360/3421 [00:22<03:13, 15.83it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.711, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  11%|█▏        | 390/3421 [00:24<03:10, 15.91it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.711, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  11%|█▏        | 390/3421 [00:24<03:10, 15.91it/s, loss=1.36, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  12%|█▏        | 420/3421 [00:26<03:08, 15.92it/s, loss=1.36, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  12%|█▏        | 420/3421 [00:26<03:08, 15.92it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  13%|█▎        | 450/3421 [00:28<03:06, 15.95it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  13%|█▎        | 450/3421 [00:28<03:06, 15.95it/s, loss=1.55, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  14%|█▍        | 480/3421 [00:30<03:04, 15.94it/s, loss=1.55, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  14%|█▍        | 480/3421 [00:30<03:04, 15.94it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  15%|█▍        | 510/3421 [00:32<03:03, 15.90it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  15%|█▍        | 510/3421 [00:32<03:03, 15.90it/s, loss=1.47, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.717, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  16%|█▌        | 540/3421 [00:34<03:01, 15.86it/s, loss=1.47, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.717, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  16%|█▌        | 540/3421 [00:34<03:01, 15.86it/s, loss=1.57, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  17%|█▋        | 570/3421 [00:35<02:59, 15.85it/s, loss=1.57, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  17%|█▋        | 570/3421 [00:35<02:59, 15.85it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  18%|█▊        | 600/3421 [00:37<02:57, 15.88it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  18%|█▊        | 600/3421 [00:37<02:57, 15.88it/s, loss=1.19, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.735, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  18%|█▊        | 630/3421 [00:39<02:55, 15.89it/s, loss=1.19, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.735, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  18%|█▊        | 630/3421 [00:39<02:55, 15.89it/s, loss=1.51, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  19%|█▉        | 660/3421 [00:41<02:53, 15.90it/s, loss=1.51, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  19%|█▉        | 660/3421 [00:41<02:53, 15.90it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.744, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  20%|██        | 690/3421 [00:43<02:51, 15.94it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.744, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  20%|██        | 690/3421 [00:43<02:51, 15.94it/s, loss=1.17, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  21%|██        | 720/3421 [00:45<02:49, 15.94it/s, loss=1.17, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  21%|██        | 720/3421 [00:45<02:49, 15.94it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  22%|██▏       | 750/3421 [00:46<02:47, 15.98it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  22%|██▏       | 750/3421 [00:46<02:47, 15.98it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  23%|██▎       | 780/3421 [00:48<02:45, 15.98it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  23%|██▎       | 780/3421 [00:48<02:45, 15.98it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  24%|██▎       | 810/3421 [00:50<02:43, 15.98it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  24%|██▎       | 810/3421 [00:50<02:43, 15.98it/s, loss=1.09, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  25%|██▍       | 840/3421 [00:52<02:41, 16.02it/s, loss=1.09, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  25%|██▍       | 840/3421 [00:52<02:41, 16.02it/s, loss=1.49, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  25%|██▌       | 870/3421 [00:54<02:39, 16.02it/s, loss=1.49, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  25%|██▌       | 870/3421 [00:54<02:39, 16.02it/s, loss=1.63, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  26%|██▋       | 900/3421 [00:56<02:37, 16.03it/s, loss=1.63, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  26%|██▋       | 900/3421 [00:56<02:37, 16.03it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.665, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  27%|██▋       | 930/3421 [00:58<02:35, 16.02it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.665, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  27%|██▋       | 930/3421 [00:58<02:35, 16.02it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  28%|██▊       | 960/3421 [00:59<02:33, 16.01it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  28%|██▊       | 960/3421 [00:59<02:33, 16.01it/s, loss=1.87, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  29%|██▉       | 990/3421 [01:01<02:31, 16.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  29%|██▉       | 990/3421 [01:01<02:31, 16.03it/s, loss=1.65, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  30%|██▉       | 1020/3421 [01:03<02:29, 16.03it/s, loss=1.65, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  30%|██▉       | 1020/3421 [01:03<02:29, 16.03it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  31%|███       | 1050/3421 [01:05<02:27, 16.05it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  31%|███       | 1050/3421 [01:05<02:27, 16.05it/s, loss=1.22, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  32%|███▏      | 1080/3421 [01:07<02:25, 16.04it/s, loss=1.22, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  32%|███▏      | 1080/3421 [01:07<02:25, 16.04it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  32%|███▏      | 1110/3421 [01:09<02:24, 16.05it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  32%|███▏      | 1110/3421 [01:09<02:24, 16.05it/s, loss=1.36, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  33%|███▎      | 1140/3421 [01:10<02:21, 16.07it/s, loss=1.36, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  33%|███▎      | 1140/3421 [01:10<02:21, 16.07it/s, loss=1.77, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  34%|███▍      | 1170/3421 [01:12<02:20, 16.07it/s, loss=1.77, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  34%|███▍      | 1170/3421 [01:12<02:20, 16.07it/s, loss=1.61, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  35%|███▌      | 1200/3421 [01:14<02:18, 16.09it/s, loss=1.61, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  35%|███▌      | 1200/3421 [01:14<02:18, 16.09it/s, loss=1.41, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  36%|███▌      | 1230/3421 [01:16<02:16, 16.08it/s, loss=1.41, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  36%|███▌      | 1230/3421 [01:16<02:16, 16.08it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  37%|███▋      | 1260/3421 [01:18<02:14, 16.08it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  37%|███▋      | 1260/3421 [01:18<02:14, 16.08it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  38%|███▊      | 1290/3421 [01:20<02:12, 16.09it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  38%|███▊      | 1290/3421 [01:20<02:12, 16.09it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.619, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  39%|███▊      | 1320/3421 [01:22<02:10, 16.09it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.619, val_f1_step=0

.496, val_loss_step=0.041]#015Epoch 2:  39%|███▊      | 1320/3421 [01:22<02:10, 16.09it/s, loss=1.06, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  39%|███▉      | 1350/3421 [01:23<02:08, 16.11it/s, loss=1.06, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  39%|███▉      | 1350/3421 [01:23<02:08, 16.11it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  40%|████      | 1380/3421 [01:25<02:06, 16.11it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  40%|████      | 1380/3421 [01:25<02:06, 16.11it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.714, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  41%|████      | 1410/3421 [01:27<02:04, 16.10it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.714, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  41%|████      | 1410/3421 [01:27<02:04, 16.10it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  42%|████▏     | 1440/3421 [01:29<02:02, 16.12it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  42%|████▏     | 1440/3421 [01:29<02:02, 16.12it/s, loss=1.41, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  43%|████▎     | 1470/3421 [01:31<02:01, 16.10it/s, loss=1.41, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  43%|████▎     | 1470/3421 [01:31<02:01, 16.10it/s, loss=1.21, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  44%|████▍     | 1500/3421 [01:33<01:59, 16.11it/s, loss=1.21, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  44%|████▍     | 1500/3421 [01:33<01:59, 16.11it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  45%|████▍     | 1530/3421 [01:35<01:57, 16.09it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  45%|████▍     | 1530/3421 [01:35<01:57, 16.09it/s, loss=1.15, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.755, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  46%|████▌     | 1560/3421 [01:36<01:55, 16.08it/s, loss=1.15, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.755, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  46%|████▌     | 1560/3421 [01:36<01:55, 16.08it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.718, val_f1_step=0.496, val_loss_step=0.041] #015Epoch 2:  46%|████▋     | 1590/3421 [01:38<01:53, 16.10it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.718, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  46%|████▋     | 1590/3421 [01:38<01:53, 16.10it/s, loss=1.13, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  47%|████▋     | 1620/3421 [01:40<01:51, 16.09it/s, loss=1.13, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  47%|████▋     | 1620/3421 [01:40<01:51, 16.09it/s, loss=1.45, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.718, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  48%|████▊     | 1650/3421 [01:42<01:49, 16.10it/s, loss=1.45, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.718, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  48%|████▊     | 1650/3421 [01:42<01:49, 16.10it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  49%|████▉     | 1680/3421 [01:44<01:48, 16.10it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  49%|████▉     | 1680/3421 [01:44<01:48, 16.10it/s, loss=1.48, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  50%|████▉     | 1710/3421 [01:46<01:46, 16.10it/s, loss=1.48, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  50%|████▉     | 1710/3421 [01:46<01:46, 16.10it/s, loss=1.44, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.723, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  51%|█████     | 1740/3421 [01:48<01:44, 16.11it/s, loss=1.44, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.723, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  51%|█████     | 1740/3421 [01:48<01:44, 16.11it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041] #015Epoch 2:  52%|█████▏    | 1770/3421 [01:49<01:42, 16.11it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  52%|█████▏    | 1770/3421 [01:49<01:42, 16.11it/s, loss=1.58, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.12it/s, loss=1.58, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  53%|█████▎    | 1800/3421 [01:51<01:40, 16.12it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.11it/s, loss=1.38, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.673, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  53%|█████▎    | 1830/3421 [01:53<01:38, 16.11it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  54%|█████▍    | 1860/3421 [01:55<01:36, 16.11it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  54%|█████▍    | 1860/3421 [01:55<01:36, 16.11it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  55%|█████▌    | 1890/3421 [01:57<01:34, 16.12it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  55%|█████▌    | 1890/3421 [01:57<01:34, 16.12it/s, loss=1.2, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.041] #015Epoch 2:  56%|█████▌    | 1920/3421 [01:59<01:33, 16.11it/s, loss=1.2, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  56%|█████▌    | 1920/3421 [01:59<01:33, 16.11it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.750, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  57%|█████▋    | 1950/3421 [02:00<01:31, 16.13it/s, loss=1.31, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.750, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  57%|█████▋    | 1950/3421 [02:00<01:31, 16.13it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.12it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  58%|█████▊    | 1980/3421 [02:02<01:29, 16.12it/s, loss=1.62, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.12it/s, loss=1.62, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  59%|█████▉    | 2010/3421 [02:04<01:27, 16.12it/s, loss=1.15, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.13it/s, loss=1.15, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  60%|█████▉    | 2040/3421 [02:06<01:25, 16.13it/s, loss=1.2, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.041] #015Epoch 2:  61%|██████    | 2070/3421 [02:08<01:23, 16.12it/s, loss=1.2, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  61%|██████    | 2070/3421 [02:08<01:23, 16.12it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  61%|██████▏   | 2100/3421 [02:10<01:21, 16.13it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  61%|██████▏   | 2100/3421 [02:10<01:21, 16.13it/s, loss=1.88, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  62%|██████▏   | 2130/3421 [02:12<01:20, 16.13it/s, loss=1.88, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.671, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  62%|██████▏   | 2130/3421 [02:12<01:20, 16.13it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.578, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  63%|██████▎   | 2160/3421 [02:14<01:18, 16.12it/s, loss=1.67, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.578, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  63%|██████▎   | 2160/3421 [02:14<01:18, 16.12it/s, loss=1.59, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.696, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.12it/s, loss=1.59, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.696, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  64%|██████▍   | 2190/3421 [02:15<01:16, 16.12it/s, loss=1.22, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.12it/s, loss=1.22, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  65%|██████▍   | 2220/3421 [02:17<01:14, 16.12it/s, loss=1.48, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  66%|██████▌   | 2250/3421 [02:19<01:12, 16.12it/s, loss=1.48, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  66%|██████▌   | 2250/3421 [02:19<01:12, 16.12it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.041] #015Epoch 2:  67%|██████▋   | 2280/3421 [02:21<01:10, 16.12it/s, loss=1.3, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  67%|██████▋   | 2280/3421 [02:21<01:10, 16.12it/s, loss=1.28, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.658, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  68%|██████▊   | 2310/3421 [02:23<01:08, 16.12it/s, loss=1.28, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.658, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  68%|██████▊   | 2310/3421 [02:23<01:08, 16.12it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  68%|██████▊   | 2340/3421 [02:25<01:07, 16.13it/s, loss=1.43, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  68%|██████▊   | 2340/3421 [02:25<01:07, 16.13it/s, loss=1.66, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.783, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  69%|██████▉   | 2370/3421 [02:26<01:05, 16.13it/s, loss=1.66, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.783, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  69%|██████▉   | 2370/3421 [02:26<01:05, 16.13it/s, loss=1.51, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  70%|███████   | 2400/3421 [02:28<01:03, 16.14it/s, loss=1.51, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  70%|███████   | 2400/3421 [02:28<01:03, 16.14it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  71%|███████   | 2430/3421 [02:30<01:01, 16.13it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  71%|███████   | 2430/3421 [02:30<01:01, 16.13it/s, loss=1.52, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  72%|███████▏  | 2460/3421 [02:32<00:59, 16.12it/s, loss=1.52, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  72%|███████▏  | 2460/3421 [02:32<00:59, 16.12it/s, loss=1.62, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  73%|███████▎  | 2490/3421 [02:34<00:57, 16.11it/s, loss=1.62, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  73%|███████▎  | 2490/3421 [02:34<00:57, 16.11it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  74%|███████▎  | 2520/3421 [02:36<00:55, 16.11it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  74%|███████▎  | 2520/3421 [02:36<00:55, 16.11it/s, loss=1.65, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  75%|███████▍  | 2550/3421 [02:38<00:54, 16.12it/s, loss=1.65, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  75%|███████▍  | 2550/3421 [02:38<00:54, 16.12it/s, loss=1.68, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  75%|███████▌  | 2580/3421 [02:40<00:52, 16.12it/s, loss=1.68, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  75%|███████▌  | 2580/3421 [02:40<00:52, 16.12it/s, loss=1.39, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  76%|████

███▋  | 2610/3421 [02:41<00:50, 16.11it/s, loss=1.39, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  76%|███████▋  | 2610/3421 [02:41<00:50, 16.11it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.728, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  77%|███████▋  | 2640/3421 [02:43<00:48, 16.12it/s, loss=1.56, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.728, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  77%|███████▋  | 2640/3421 [02:43<00:48, 16.12it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  78%|███████▊  | 2670/3421 [02:45<00:46, 16.12it/s, loss=1.29, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  78%|███████▊  | 2670/3421 [02:45<00:46, 16.12it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  79%|███████▉  | 2700/3421 [02:47<00:44, 16.13it/s, loss=1.32, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  79%|███████▉  | 2700/3421 [02:47<00:44, 16.13it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.758, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  80%|███████▉  | 2730/3421 [02:49<00:42, 16.12it/s, loss=1.24, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.758, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  80%|███████▉  | 2730/3421 [02:49<00:42, 16.12it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.732, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  81%|████████  | 2760/3421 [02:51<00:41, 16.12it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.732, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  81%|████████  | 2760/3421 [02:51<00:41, 16.12it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  82%|████████▏ | 2790/3421 [02:53<00:39, 16.12it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  82%|████████▏ | 2790/3421 [02:53<00:39, 16.12it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.12it/s, loss=1.23, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  82%|████████▏ | 2820/3421 [02:54<00:37, 16.12it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.12it/s, loss=1.26, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  83%|████████▎ | 2850/3421 [02:56<00:35, 16.12it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.760, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  84%|████████▍ | 2880/3421 [02:58<00:33, 16.12it/s, loss=1.27, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.760, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  84%|████████▍ | 2880/3421 [02:58<00:33, 16.12it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  85%|████████▌ | 2910/3421 [03:00<00:31, 16.12it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  85%|████████▌ | 2910/3421 [03:00<00:31, 16.12it/s, loss=1.55, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  86%|████████▌ | 2940/3421 [03:02<00:29, 16.12it/s, loss=1.55, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.654, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  86%|████████▌ | 2940/3421 [03:02<00:29, 16.12it/s, loss=1.44, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.710, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  87%|████████▋ | 2970/3421 [03:04<00:27, 16.12it/s, loss=1.44, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.710, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  87%|████████▋ | 2970/3421 [03:04<00:27, 16.12it/s, loss=1.09, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  88%|████████▊ | 3000/3421 [03:06<00:26, 16.12it/s, loss=1.09, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  88%|████████▊ | 3000/3421 [03:06<00:26, 16.12it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]#015Epoch 2:  89%|████████▊ | 3030/3421 [03:07<00:24, 16.14it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/397 [00:00<?, ?it/s]#033[A
#015Validating:   8%|▊         | 30/397 [00:00<00:11, 31.58it/s]#033[A#015Epoch 2:  89%|████████▉ | 3060/3421 [03:08<00:22, 16.22it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  15%|█▌        | 60/397 [00:01<00:09, 34.35it/s]#033[A#015Epoch 2:  90%|█████████ | 3090/3421 [03:09<00:20, 16.31it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  23%|██▎       | 90/397 [00:02<00:08, 35.91it/s]#033[A#015Epoch 2:  91%|█████████ | 3120/3421 [03:10<00:18, 16.41it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  30%|███       | 120/397 [00:03<00:07, 37.71it/s]#033[A#015Epoch 2:  92%|█████████▏| 3150/3421 [03:10<00:16, 16.50it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  38%|███▊      | 150/397 [00:03<00:06, 38.86it/s]#033[A#015Epoch 2:  93%|█████████▎| 3180/3421 [03:11<00:14, 16.60it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  45%|████▌     | 180/397 [00:04<00:05, 40.14it/s]#033[A#015Epoch 2:  94%|█████████▍| 3210/3421 [03:12<00:12, 16.70it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  53%|█████▎    | 210/397 [00:05<00:04, 40.87it/s]#033[A#015Epoch 2:  95%|█████████▍| 3240/3421 [03:12<00:10, 16.79it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  60%|██████    | 240/397 [00:05<00:03, 41.15it/s]#033[A#015Epoch 2:  96%|█████████▌| 3270/3421 [03:13<00:08, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  68%|██████▊   | 270/397 [00:06<00:03, 41.08it/s]#033[A#015Epoch 2:  96%|█████████▋| 3300/3421 [03:14<00:07, 16.97it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  76%|███████▌  | 300/397 [00:07<00:02, 40.46it/s]#033[A#015Epoch 2:  97%|█████████▋| 3330/3421 [03:15<00:05, 17.06it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  83%|████████▎ | 330/397 [00:08<00:01, 41.14it/s]#033[A#015Epoch 2:  98%|█████████▊| 3360/3421 [03:15<00:03, 17.15it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  91%|█████████ | 360/397 [00:08<00:00, 41.69it/s]#033[A#015Epoch 2:  99%|█████████▉| 3390/3421 [03:16<00:01, 17.25it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating:  98%|█████████▊| 390/397 [00:09<00:00, 42.24it/s]#033[A#015Epoch 2: 100%|█████████▉| 3420/3421 [03:17<00:00, 17.34it/s, loss=1.37, v_num=0, val_f1_epoch=0.674, val_loss_epoch=0.0919, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.041]
#015Validating: 100%|██████████| 397/397 [00:09<00:00, 42.95it/s]#033[A#015Epoch 2: 100%|██████████| 3421/3421 [03:17<00:00, 17.32it/s, loss=1.73, v_num=0, val_f1_epoch=0.689, val_loss_epoch=0.0885, train_f1=0.826, val_f1_step=0.494, val_loss_step=0.0385]
#015                                                             #033[A#015Epoch 2: 100%|██████████| 3421/3421 [03:18<00:00, 17.22it/s, loss=1.73, v_num=0, val_f1_epoch=0.689, val_loss_epoch=0.0885, train_f1=0.826, val_f1_step=0.494, val_loss_step=0.0385]

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/11174 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/11174 [00:00<?, ?it/s] #015Epoch 0:   0%|          | 30/11174 [00:02<12:26, 14.93it/s]#015Epoch 0:   0%|          | 30/11174 [00:02<12:26, 14.93it/s, loss=20.6, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.435]#015Epoch 0:   1%|          | 60/11174 [00:03<11:46, 15.72it/s, loss=20.6, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.435]#015Epoch 0:   1%|          | 60/11174 [00:03<11:47, 15.72it/s, loss=6.19, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   1%|          | 90/11174 [00:05<11:19, 16.31it/s, loss=6.19, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   1%|          | 90/11174 [00:05<11:19, 16.31it/s, loss=4.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.531]#015Epoch 0:   1%|          | 120/11174 [00:07<11:14, 16.39it/s, loss=4.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.531]#015Epoch 0:   1%|          | 120/11174 [00:07<11:14, 16.39it/s, loss=3.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:   1%|▏         | 150/11174 [00:09<11:03, 16.61it/s, loss=3.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:   1%|▏         | 150/11174 [00:09<11:03, 16.61it/s, loss=3.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:   2%|▏         | 180/11174 [00:10<10:59, 16.66it/s, loss=3.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:   2%|▏         | 180/11174 [00:10<10:59, 16.66it/s, loss=3.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536] #015Epoch 0:   2%|▏         | 210/11174 [00:12<10:59, 16.63it/s, loss=3.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536]#015Epoch 0:   2%|▏         | 210/11174 [00:12<10:59, 16.63it/s, loss=2.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.483]#015Epoch 0:   2%|▏         | 240/11174 [00:14<10:53, 16.74it/s, loss=2.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.483]#015Epoch 0:   2%|▏         | 240/11174 [00:14<10:53, 16.74it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.496] #015Epoch 0:   2%|▏         | 270/11174 [00:16<10:51, 16.73it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.496]#015Epoch 0:   2%|▏         | 270/11174 [00:16<10:51, 16.73it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   3%|▎         | 300/11174 [00:17<10:46, 16.82it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   3%|▎         | 300/11174 [00:17<10:46, 16.82it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:   3%|▎         | 330/11174 [00:19<10:45, 16.80it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:   3%|▎         | 330/11174 [00:19<10:45, 16.80it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   3%|▎         | 360/11174 [00:21<10:43, 16.80it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   3%|▎         | 360/11174 [00:21<10:43, 16.80it/s, loss=2.31, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   3%|▎         | 390/11174 [00:23<10:39, 16.87it/s, loss=2.31, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   3%|▎         | 390/11174 [00:23<10:39, 16.87it/s, loss=2.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.483]#015Epoch 0:   4%|▍         | 420/11174 [00:24<10:38, 16.85it/s, loss=2.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.483]#015Epoch 0:   4%|▍         | 420/11174 [00:24<10:38, 16.85it/s, loss=2.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   4%|▍         | 450/11174 [00:26<10:34, 16.91it/s, loss=2.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   4%|▍         | 450/11174 [00:26<10:34, 16.91it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   4%|▍         | 480/11174 [00:28<10:32, 16.90it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   4%|▍         | 480/11174 [00:28<10:32, 16.90it/s, loss=2.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▍         | 510/11174 [00:30<10:31, 16.89it/s, loss=2.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▍         | 510/11174 [00:30<10:31, 16.88it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▍         | 540/11174 [00:31<10:28, 16.93it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▍         | 540/11174 [00:31<10:28, 16.93it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.485]#015Epoch 0:   5%|▌         | 570/11174 [00:33<10:26, 16.93it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.485]#015Epoch 0:   5%|▌         | 570/11174 [00:33<10:26, 16.93it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▌         | 600/11174 [00:35<10:23, 16.96it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.484]#015Epoch 0:   5%|▌         | 600/11174 [00:35<10:23, 16.96it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.481] #015Epoch 0:   6%|▌         | 630/11174 [00:37<10:21, 16.96it/s, loss=2.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.481]#015Epoch 0:   6%|▌         | 630/11174 [00:37<10:21, 16.96it/s, loss=2.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   6%|▌         | 660/11174 [00:38<10:20, 16.95it/s, loss=2.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   6%|▌         | 660/11174 [00:38<10:20, 16.95it/s, loss=2.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.500]#015Epoch 0:   6%|▌         | 690/11174 [00:40<10:17, 16.98it/s, loss=2.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.500]#015Epoch 0:   6%|▌         | 690/11174 [00:40<10:17, 16.98it/s, loss=1.76, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:   6%|▋         | 720/11174 [00:42<10:16, 16.95it/s, loss=1.76, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:   6%|▋         | 720/11174 [00:42<10:16, 16.95it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   7%|▋         | 750/11174 [00:44<10:13, 16.98it/s, loss=2.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   7%|▋         | 750/11174 [00:44<10:13, 16.98it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502] #015Epoch 0:   7%|▋         | 780/11174 [00:45<10:12, 16.96it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502]#015Epoch 0:   7%|▋         | 780/11174 [00:45<10:12, 16.96it/s, loss=2.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   7%|▋         | 810/11174 [00:47<10:11, 16.95it/s, loss=2.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.486]#015Epoch 0:   7%|▋         | 810/11174 [00:47<10:11, 16.95it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.499] #015Epoch 0:   8%|▊         | 840/11174 [00:49<10:08, 16.97it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.499]#015Epoch 0:   8%|▊         | 840/11174 [00:49<10:08, 16.97it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:   8%|▊         | 870/11174 [00:51<10:07, 16.98it/s, loss=2.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:   8%|▊         | 870/11174 [00:51<10:07, 16.97it/s, loss=2.17, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   8%|▊         | 900/11174 [00:52<10:04, 17.00it/s, loss=2.17, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:   8%|▊         | 900/11174 [00:52<10:04, 17.00it/s, loss=1.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:   8%|▊         | 930/11174 [00:54<10:03, 16.98it/s, loss=1.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:   8%|▊         | 930/11174 [00:54<10:03, 16.98it/s, loss=2.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:   9%|▊         | 960/11174 [00:56<10:02, 16.97it/s, loss=2.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:   9%|▊         | 960/11174 [00:56<10:02, 16.97it/s, loss=2.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   9%|▉         | 990/11174 [00:58<10:00, 16.96it/s, loss=2.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:   9%|▉         | 990/11174 [00:58<10:00, 16.96it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.504]#015Epoch 0:   9%|▉         | 1020/11174 [01:00<09:59, 16.94it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.504]#015Epoch 0:   9%|▉         | 1020/11174 [01:00<09:59, 16.94it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.520]#015Epoch 0:   9%|▉         | 1050/11174 [01:01<09:56, 16.97it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.520]#015Epoch 0:   9%|▉         | 1050/11174 [01:01<09:56, 16.97it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502]#015Epoch 0:  10%|▉         | 1080/11174 [01:03<09:55, 16.96it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502]#015Epoch 0:  10%|▉         | 1080/11174 [01:03<09:55, 16.96it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.533]#015Epoch 0:  10%|▉         | 1110/11174 [01:05<09:53, 16.95it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.533]#015Epoch 0:  10%|▉         | 1110/11174 [01:05<09:53, 16.95it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.530]#015Epoch 0:  10%|█         | 1140/11174 [01:07<09:51, 16.97it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.530]#015Epoch 0:  10%|█         | 1140/11174 [01:07<09:51, 16.97it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.515]#015Epoch 0:  10%|█         | 1170/11174 [01:08<09:49, 16.96it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.515]#015Epoch 0:  10%|█         | 1170/11174 [01:08<09:49, 16.96it/s, loss=2.29, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  11%|█         | 1200/11174 [01:10<09:47, 16.98it/s, loss=2.29, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  11%|█         | 1200/11174 [01:10<09:47, 16.98it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:  11%|█         | 1230/11174 [01:12<09:45, 16.98it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.503]#015Epoch 0:  11%|█         | 1230/11174 [01:12<09:45, 16.98it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:  11%|█▏        | 1260/11174 [01:14<09:43, 16.98it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.516]#015Epoch 0:  11%|█▏        | 1260/11174 [01:14<09:43, 16.98it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  12%|█▏        | 1290/11174 [01:15<09:41, 16.99it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  12%|█▏        | 1290/11174 [01:15<09:41, 16.99it/s, loss=1.88, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  12%|█▏        | 1320/11174 [01:17<09:40, 16.98it/s, loss=1.88, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  12%|█▏        | 1320/11174 [01:17<09:40, 16.98it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.529]#015Epoch 0:  12%|█▏        | 1350/11174 [01:19<09:38, 16.99it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.529]#015Epoch 0:  12%|█▏        | 1350/11174 [01:19<09:38, 16.99it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542] #015Epoch 0:  12%|█▏        | 1380/11174 [01:21<09:36, 16.98it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542]#015Epoch 0:  12%|█▏        | 1380/11174 [01:21<09:36, 16.98it/s, loss=2.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  13%|█▎        | 1410/11174 [01:23<09:35, 16.98it/s, loss=2.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  13%|█▎        | 1410/11174 [01:23<09:35, 16.98it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:  13%|█▎        | 1440/11174 [01:24<09:32, 17.00it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:  13%|█▎        | 1440/11174 [01:24<09:32, 17.00it/s, loss=2.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502]#015Epoch 0:  13%|█▎        | 1470/11174 [01:26<09:31, 16.99it/s, loss=2.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.502]#015Epoch 0:  13%|█▎        | 1470/11174 [01:26<09:31, 16.99it/s, loss=1.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.501]#015Epoch 0:  13%|█▎        | 1500/11174 [01:28<09:28, 17.00it/s, loss=1.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.501]#015Epoch 0:  13%|█▎        | 1500/11174 [01:28<09:28, 17.00it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.511]#015Epoch 0:  14%|█▎        | 1530/11174 [01:29<09:27, 17.00it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.511]#015Epoch 0:  14%|█▎        | 1530/11174 [01:29<09:27, 17.00it/s, loss=2.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:  14%|█▍        | 1560/11174 [01:31<09:25, 16.99it/s, loss=2.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.498]#015Epoch 0:  14%|█▍        | 1560/11174 [01:31<09:25, 16.99it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  14%|█▍        | 1590/11174 [01:33<09:23, 17.00it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  14%|█▍        | 1590/11174 [01:33<09:23, 17.00it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  14%|█▍        | 1620/11174 [01:35<09:22, 17.00it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  14%|█▍        | 1620/11174 [01:35<09:22, 17.00it/s, loss=1.88, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.525]#015Epoch 0:  15%|█▍        | 1650/11174 [01:37<09:19, 17.01it/s, loss=1.88, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.525]#015Epoch 0:  15%|█▍        | 1650/11174 [01:37<09:19, 17.01it/s, loss=1.94, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  15%|█▌        | 1680/11174 [01:38<09:18, 17.01it/s, loss=1.94, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  15%|█▌        | 1680/11174 [01:38<09:18, 17.01it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.558]#015Epoch 0:  15%|█▌        | 1710/11174 [01:40<09:16, 17.00it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.558]#015Epoch 0:  15%|█▌        | 1710/11174 [01:40<09:16, 17.00it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015E

poch 0:  16%|█▌        | 1740/11174 [01:42<09:14, 17.02it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  16%|█▌        | 1740/11174 [01:42<09:14, 17.02it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:  16%|█▌        | 1770/11174 [01:44<09:12, 17.02it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.491]#015Epoch 0:  16%|█▌        | 1770/11174 [01:44<09:12, 17.02it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.519] #015Epoch 0:  16%|█▌        | 1800/11174 [01:45<09:10, 17.03it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.519]#015Epoch 0:  16%|█▌        | 1800/11174 [01:45<09:10, 17.03it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536]#015Epoch 0:  16%|█▋        | 1830/11174 [01:47<09:08, 17.02it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536]#015Epoch 0:  16%|█▋        | 1830/11174 [01:47<09:08, 17.02it/s, loss=1.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  17%|█▋        | 1860/11174 [01:49<09:07, 17.01it/s, loss=1.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  17%|█▋        | 1860/11174 [01:49<09:07, 17.01it/s, loss=2.19, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  17%|█▋        | 1890/11174 [01:50<09:05, 17.03it/s, loss=2.19, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  17%|█▋        | 1890/11174 [01:50<09:05, 17.03it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.529]#015Epoch 0:  17%|█▋        | 1920/11174 [01:52<09:03, 17.03it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.529]#015Epoch 0:  17%|█▋        | 1920/11174 [01:52<09:03, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.500] #015Epoch 0:  17%|█▋        | 1950/11174 [01:54<09:01, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.500]#015Epoch 0:  17%|█▋        | 1950/11174 [01:54<09:01, 17.03it/s, loss=2.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.526]#015Epoch 0:  18%|█▊        | 1980/11174 [01:56<08:59, 17.03it/s, loss=2.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.526]#015Epoch 0:  18%|█▊        | 1980/11174 [01:56<08:59, 17.03it/s, loss=2.41, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  18%|█▊        | 2010/11174 [01:58<08:58, 17.02it/s, loss=2.41, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  18%|█▊        | 2010/11174 [01:58<08:58, 17.02it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.533]#015Epoch 0:  18%|█▊        | 2040/11174 [01:59<08:56, 17.01it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.533]#015Epoch 0:  18%|█▊        | 2040/11174 [01:59<08:56, 17.01it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.479]#015Epoch 0:  19%|█▊        | 2070/11174 [02:01<08:55, 17.00it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.479]#015Epoch 0:  19%|█▊        | 2070/11174 [02:01<08:55, 17.00it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.575]#015Epoch 0:  19%|█▉        | 2100/11174 [02:03<08:53, 17.01it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.575]#015Epoch 0:  19%|█▉        | 2100/11174 [02:03<08:53, 17.01it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  19%|█▉        | 2130/11174 [02:05<08:51, 17.00it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  19%|█▉        | 2130/11174 [02:05<08:51, 17.00it/s, loss=2.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.499]#015Epoch 0:  19%|█▉        | 2160/11174 [02:07<08:50, 17.00it/s, loss=2.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.499]#015Epoch 0:  19%|█▉        | 2160/11174 [02:07<08:50, 17.00it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.526]#015Epoch 0:  20%|█▉        | 2190/11174 [02:08<08:47, 17.02it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.526]#015Epoch 0:  20%|█▉        | 2190/11174 [02:08<08:47, 17.02it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.512]#015Epoch 0:  20%|█▉        | 2220/11174 [02:10<08:46, 17.01it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.512]#015Epoch 0:  20%|█▉        | 2220/11174 [02:10<08:46, 17.01it/s, loss=2.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.547]#015Epoch 0:  20%|██        | 2250/11174 [02:12<08:44, 17.02it/s, loss=2.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.547]#015Epoch 0:  20%|██        | 2250/11174 [02:12<08:44, 17.02it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  20%|██        | 2280/11174 [02:13<08:42, 17.02it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  20%|██        | 2280/11174 [02:13<08:42, 17.02it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.577]#015Epoch 0:  21%|██        | 2310/11174 [02:15<08:40, 17.02it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.577]#015Epoch 0:  21%|██        | 2310/11174 [02:15<08:40, 17.02it/s, loss=2.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.564]#015Epoch 0:  21%|██        | 2340/11174 [02:17<08:38, 17.02it/s, loss=2.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.564]#015Epoch 0:  21%|██        | 2340/11174 [02:17<08:38, 17.02it/s, loss=1.6, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.574] #015Epoch 0:  21%|██        | 2370/11174 [02:19<08:37, 17.02it/s, loss=1.6, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.574]#015Epoch 0:  21%|██        | 2370/11174 [02:19<08:37, 17.02it/s, loss=2.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  21%|██▏       | 2400/11174 [02:20<08:35, 17.02it/s, loss=2.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  21%|██▏       | 2400/11174 [02:20<08:35, 17.02it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.563]#015Epoch 0:  22%|██▏       | 2430/11174 [02:22<08:33, 17.02it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.563]#015Epoch 0:  22%|██▏       | 2430/11174 [02:22<08:33, 17.02it/s, loss=1.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.553]#015Epoch 0:  22%|██▏       | 2460/11174 [02:24<08:32, 17.02it/s, loss=1.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.553]#015Epoch 0:  22%|██▏       | 2460/11174 [02:24<08:32, 17.02it/s, loss=1.97, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.617]#015Epoch 0:  22%|██▏       | 2490/11174 [02:26<08:29, 17.03it/s, loss=1.97, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.617]#015Epoch 0:  22%|██▏       | 2490/11174 [02:26<08:29, 17.03it/s, loss=2.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.539]#015Epoch 0:  23%|██▎       | 2520/11174 [02:28<08:28, 17.02it/s, loss=2.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.539]#015Epoch 0:  23%|██▎       | 2520/11174 [02:28<08:28, 17.02it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  23%|██▎       | 2550/11174 [02:29<08:26, 17.03it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  23%|██▎       | 2550/11174 [02:29<08:26, 17.03it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  23%|██▎       | 2580/11174 [02:31<08:24, 17.03it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  23%|██▎       | 2580/11174 [02:31<08:24, 17.03it/s, loss=2.14, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  23%|██▎       | 2610/11174 [02:33<08:22, 17.03it/s, loss=2.14, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  23%|██▎       | 2610/11174 [02:33<08:22, 17.03it/s, loss=2.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  24%|██▎       | 2640/11174 [02:34<08:20, 17.04it/s, loss=2.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  24%|██▎       | 2640/11174 [02:34<08:20, 17.04it/s, loss=2.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.642]#015Epoch 0:  24%|██▍       | 2670/11174 [02:36<08:19, 17.03it/s, loss=2.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.642]#015Epoch 0:  24%|██▍       | 2670/11174 [02:36<08:19, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  24%|██▍       | 2700/11174 [02:38<08:17, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  24%|██▍       | 2700/11174 [02:38<08:17, 17.03it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.596]#015Epoch 0:  24%|██▍       | 2730/11174 [02:40<08:15, 17.03it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.596]#015Epoch 0:  24%|██▍       | 2730/11174 [02:40<08:15, 17.03it/s, loss=1.86, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.564]#015Epoch 0:  25%|██▍       | 2760/11174 [02:42<08:14, 17.03it/s, loss=1.86, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.564]#015Epoch 0:  25%|██▍       | 2760/11174 [02:42<08:14, 17.03it/s, loss=1.94, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.556]#015Epoch 0:  25%|██▍       | 2790/11174 [02:43<08:12, 17.04it/s, loss=1.94, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.556]#015Epoch 0:  25%|██▍       | 2790/11174 [02:43<08:12, 17.04it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  25%|██▌       | 2820/11174 [02:45<08:10, 17.03it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  25%|██▌       | 2820/11174 [02:45<08:10, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.586]#015Epoch 0:  26%|██▌       | 2850/11174 [02:47<08:08, 17.04it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.586]#015Epoch 0:  26%|██▌       | 2850/11174 [02:47<08:08, 17.04it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  26%|██▌       | 2880/11174 [02:49<08:06, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  26%|██▌       | 2880/11174 [02:49<08:06, 17.03it/s, loss=2.44, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  26%|██▌       | 2910/11174 [02:50<08:05, 17.03it/s, loss=2.44, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  26%|██▌       | 2910/11174 [02:50<08:05, 17.03it/s, loss=1.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:  26%|██▋       | 2940/11174 [02:52<08:03, 17.03it/s, loss=1.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.482]#015Epoch 0:  26%|██▋       | 2940/11174 [02:52<08:03, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.585]#015Epoch 0:  27%|██▋       | 2970/11174 [02:54<08:01, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.585]#015Epoch 0:  27%|██▋       | 2970/11174 [02:54<08:01, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  27%|██▋       | 3000/11174 [02:56<08:00, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  27%|██▋       | 3000/11174 [02:56<08:00, 17.03it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.524]#015Epoch 0:  27%|██▋       | 3030/11174 [02:58<07:58, 17.01it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.524]#015Epoch 0:  27%|██▋       | 3030/11174 [02:58<07:58, 17.01it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  27%|██▋       | 3060/11174 [02:59<07:56, 17.01it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  27%|██▋       | 3060/11174 [02:59<07:56, 17.01it/s, loss=1.66, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  28%|██▊       | 3090/11174 [03:01<07:54, 17.02it/s, loss=1.66, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  28%|██▊       | 3090/11174 [03:01<07:54, 17.02it/s, loss=2.17, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  28%|██▊       | 3120/11174 [03:03<07:53, 17.02it/s, loss=2.17, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  28%|██▊       | 3120/11174 [03:03<07:53, 17.02it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  28%|██▊       | 3150/11174 [03:04<07:51, 17.03it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  28%|██▊       | 3150/11174 [03:04<07:51, 17.03it/s, loss=1.67, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  28%|██▊       | 3180/11174 [03:06<07:49, 17.02it/s, loss=1.67, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  28%|██▊       | 3180/11174 [03:06<07:49, 17.02it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]   #015Epoch 0:  29%|██▊       | 3210/11174 [03:08<07:48, 17.02it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]#015Epoch 0:  29%|██▊       | 3210/11174 [03:08<07:48, 17.02it/s, loss=1.61, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  29%|██▉       | 3240/11174 [03:10<07:46, 17.02it/s, loss=1.61, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  29%|██▉       | 3240/11174 [03:10<07:46, 17.02it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  29%|██▉       | 3270/11174 [03:12<07:44, 17.02it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  29%|██▉       | 3270/11174 [03:12<07:44, 17.02it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  30%|██▉       | 3300/11174 [03:13<07:42, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  30%|██▉       | 3300/11174 [03:13<07:42, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  30%|██▉       | 3330/11174 [03:15<07:40, 17.02it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  30%|██▉       | 3330/11174 [03:15<07:40, 17.02it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  30%|███       | 3360/11174 [03:17<07:39, 17.02it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  30%|███       | 3360/11174 [03:17<07:39, 17.02it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.577]#015Epoch 0:  30%|███       | 3390/11174 [03:19<07:37, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.577]#015Epoch 0:  30%|███       | 3390/11174 [03:19<07:37, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.596]#015Epoch 0:  31%|███       | 3420/11174 [03:20<07:35, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, tra

in_f1=0.596]#015Epoch 0:  31%|███       | 3420/11174 [03:20<07:35, 17.03it/s, loss=2.05, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  31%|███       | 3450/11174 [03:22<07:33, 17.03it/s, loss=2.05, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  31%|███       | 3450/11174 [03:22<07:33, 17.03it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:  31%|███       | 3480/11174 [03:24<07:31, 17.03it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:  31%|███       | 3480/11174 [03:24<07:31, 17.03it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]#015Epoch 0:  31%|███▏      | 3510/11174 [03:26<07:30, 17.02it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]#015Epoch 0:  31%|███▏      | 3510/11174 [03:26<07:30, 17.02it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  32%|███▏      | 3540/11174 [03:27<07:28, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  32%|███▏      | 3540/11174 [03:27<07:28, 17.03it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.605]#015Epoch 0:  32%|███▏      | 3570/11174 [03:29<07:26, 17.02it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.605]#015Epoch 0:  32%|███▏      | 3570/11174 [03:29<07:26, 17.02it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  32%|███▏      | 3600/11174 [03:31<07:24, 17.03it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  32%|███▏      | 3600/11174 [03:31<07:24, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  32%|███▏      | 3630/11174 [03:33<07:23, 17.02it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  32%|███▏      | 3630/11174 [03:33<07:23, 17.02it/s, loss=2.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.608]#015Epoch 0:  33%|███▎      | 3660/11174 [03:35<07:21, 17.02it/s, loss=2.48, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.608]#015Epoch 0:  33%|███▎      | 3660/11174 [03:35<07:21, 17.02it/s, loss=1.98, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542]#015Epoch 0:  33%|███▎      | 3690/11174 [03:36<07:19, 17.03it/s, loss=1.98, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542]#015Epoch 0:  33%|███▎      | 3690/11174 [03:36<07:19, 17.03it/s, loss=1.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.585]#015Epoch 0:  33%|███▎      | 3720/11174 [03:38<07:17, 17.03it/s, loss=1.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.585]#015Epoch 0:  33%|███▎      | 3720/11174 [03:38<07:17, 17.03it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  34%|███▎      | 3750/11174 [03:40<07:15, 17.03it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  34%|███▎      | 3750/11174 [03:40<07:15, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  34%|███▍      | 3780/11174 [03:41<07:14, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  34%|███▍      | 3780/11174 [03:41<07:14, 17.03it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  34%|███▍      | 3810/11174 [03:43<07:12, 17.03it/s, loss=2.22, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  34%|███▍      | 3810/11174 [03:43<07:12, 17.03it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.525]#015Epoch 0:  34%|███▍      | 3840/11174 [03:45<07:10, 17.03it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.525]#015Epoch 0:  34%|███▍      | 3840/11174 [03:45<07:10, 17.03it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536]   #015Epoch 0:  35%|███▍      | 3870/11174 [03:47<07:08, 17.03it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.536]#015Epoch 0:  35%|███▍      | 3870/11174 [03:47<07:08, 17.03it/s, loss=1.63, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  35%|███▍      | 3900/11174 [03:48<07:07, 17.03it/s, loss=1.63, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  35%|███▍      | 3900/11174 [03:48<07:07, 17.03it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  35%|███▌      | 3930/11174 [03:50<07:05, 17.03it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  35%|███▌      | 3930/11174 [03:50<07:05, 17.03it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.523]#015Epoch 0:  35%|███▌      | 3960/11174 [03:52<07:03, 17.03it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.523]#015Epoch 0:  35%|███▌      | 3960/11174 [03:52<07:03, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  36%|███▌      | 3990/11174 [03:54<07:01, 17.04it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  36%|███▌      | 3990/11174 [03:54<07:01, 17.04it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  36%|███▌      | 4020/11174 [03:56<07:00, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  36%|███▌      | 4020/11174 [03:56<07:00, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]#015Epoch 0:  36%|███▌      | 4050/11174 [03:57<06:58, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]#015Epoch 0:  36%|███▌      | 4050/11174 [03:57<06:58, 17.03it/s, loss=2.02, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  37%|███▋      | 4080/11174 [03:59<06:56, 17.02it/s, loss=2.02, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  37%|███▋      | 4080/11174 [03:59<06:56, 17.02it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.528]#015Epoch 0:  37%|███▋      | 4110/11174 [04:01<06:55, 17.02it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.528]#015Epoch 0:  37%|███▋      | 4110/11174 [04:01<06:55, 17.02it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  37%|███▋      | 4140/11174 [04:03<06:53, 17.02it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  37%|███▋      | 4140/11174 [04:03<06:53, 17.02it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  37%|███▋      | 4170/11174 [04:04<06:51, 17.02it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  37%|███▋      | 4170/11174 [04:04<06:51, 17.02it/s, loss=1.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.653] #015Epoch 0:  38%|███▊      | 4200/11174 [04:06<06:49, 17.03it/s, loss=1.5, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.653]#015Epoch 0:  38%|███▊      | 4200/11174 [04:06<06:49, 17.03it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  38%|███▊      | 4230/11174 [04:08<06:47, 17.02it/s, loss=2.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.616]#015Epoch 0:  38%|███▊      | 4230/11174 [04:08<06:47, 17.02it/s, loss=1.38, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.637]#015Epoch 0:  38%|███▊      | 4260/11174 [04:10<06:46, 17.02it/s, loss=1.38, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.637]#015Epoch 0:  38%|███▊      | 4260/11174 [04:10<06:46, 17.02it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  38%|███▊      | 4290/11174 [04:11<06:44, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.537]#015Epoch 0:  38%|███▊      | 4290/11174 [04:11<06:44, 17.03it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  39%|███▊      | 4320/11174 [04:13<06:42, 17.03it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  39%|███▊      | 4320/11174 [04:13<06:42, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  39%|███▉      | 4350/11174 [04:15<06:40, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  39%|███▉      | 4350/11174 [04:15<06:40, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573] #015Epoch 0:  39%|███▉      | 4380/11174 [04:17<06:38, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  39%|███▉      | 4380/11174 [04:17<06:38, 17.03it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  39%|███▉      | 4410/11174 [04:18<06:37, 17.03it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  39%|███▉      | 4410/11174 [04:18<06:37, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  40%|███▉      | 4440/11174 [04:20<06:35, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  40%|███▉      | 4440/11174 [04:20<06:35, 17.03it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  40%|████      | 4470/11174 [04:22<06:33, 17.03it/s, loss=1.96, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  40%|████      | 4470/11174 [04:22<06:33, 17.03it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  40%|████      | 4500/11174 [04:24<06:31, 17.04it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  40%|████      | 4500/11174 [04:24<06:31, 17.04it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]#015Epoch 0:  41%|████      | 4530/11174 [04:25<06:29, 17.04it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.589]#015Epoch 0:  41%|████      | 4530/11174 [04:25<06:29, 17.04it/s, loss=2.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  41%|████      | 4560/11174 [04:27<06:28, 17.04it/s, loss=2.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  41%|████      | 4560/11174 [04:27<06:28, 17.04it/s, loss=1.63, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.677]#015Epoch 0:  41%|████      | 4590/11174 [04:29<06:26, 17.04it/s, loss=1.63, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.677]#015Epoch 0:  41%|████      | 4590/11174 [04:29<06:26, 17.04it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  41%|████▏     | 4620/11174 [04:31<06:24, 17.04it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  41%|████▏     | 4620/11174 [04:31<06:24, 17.04it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  42%|████▏     | 4650/11174 [04:32<06:22, 17.04it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  42%|████▏     | 4650/11174 [04:32<06:22, 17.04it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.602]#015Epoch 0:  42%|████▏     | 4680/11174 [04:34<06:21, 17.04it/s, loss=2.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.602]#015Epoch 0:  42%|████▏     | 4680/11174 [04:34<06:21, 17.04it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.612] #015Epoch 0:  42%|████▏     | 4710/11174 [04:36<06:19, 17.04it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.612]#015Epoch 0:  42%|████▏     | 4710/11174 [04:36<06:19, 17.04it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.635]#015Epoch 0:  42%|████▏     | 4740/11174 [04:38<06:17, 17.04it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.635]#015Epoch 0:  42%|████▏     | 4740/11174 [04:38<06:17, 17.04it/s, loss=1.98, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  43%|████▎     | 4770/11174 [04:39<06:15, 17.04it/s, loss=1.98, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  43%|████▎     | 4770/11174 [04:39<06:15, 17.04it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.572]#015Epoch 0:  43%|████▎     | 4800/11174 [04:41<06:13, 17.05it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.572]#015Epoch 0:  43%|████▎     | 4800/11174 [04:41<06:13, 17.05it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.515]#015Epoch 0:  43%|████▎     | 4830/11174 [04:43<06:12, 17.04it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.515]#015Epoch 0:  43%|████▎     | 4830/11174 [04:43<06:12, 17.04it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.582]#015Epoch 0:  43%|████▎     | 4860/11174 [04:45<06:10, 17.04it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.582]#015Epoch 0:  43%|████▎     | 4860/11174 [04:45<06:10, 17.04it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  44%|████▍     | 4890/11174 [04:46<06:08, 17.04it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.595]#015Epoch 0:  44%|████▍     | 4890/11174 [04:46<06:08, 17.04it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578] #015Epoch 0:  44%|████▍     | 4920/11174 [04:48<06:06, 17.04it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  44%|████▍     | 4920/11174 [04:48<06:06, 17.04it/s, loss=2.1, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  44%|████▍     | 4950/11174 [04:50<06:05, 17.04it/s, loss=2.1, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  44%|████▍     | 4950/11174 [04:50<06:05, 17.04it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.632]#015Epoch 0:  45%|████▍     | 4980/11174 [04:52<06:03, 17.04it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.632]#015Epoch 0:  45%|████▍     | 4980/11174 [04:52<06:03, 17.04it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  45%|████▍     | 5010/11174 [04:54<06:01, 17.04it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  45%|████▍     | 5010/11174 [04:54<06:01, 17.04it/s, loss=2.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  45%|████▌     | 5040/11174 [04:55<05:59, 17.04it/s, loss=2.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  45%|████▌     | 5040/11174 [04:55<05:59, 17.04it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.527]#015Epoch 0:  45%|████▌     | 5070/11174 [04:57<05:58, 17.04it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.527]#015Epoch 0:  45%|████▌     | 5070/11174 [

04:57<05:58, 17.04it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  46%|████▌     | 5100/11174 [04:59<05:56, 17.04it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.569]#015Epoch 0:  46%|████▌     | 5100/11174 [04:59<05:56, 17.04it/s, loss=2.16, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  46%|████▌     | 5130/11174 [05:01<05:54, 17.04it/s, loss=2.16, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.532]#015Epoch 0:  46%|████▌     | 5130/11174 [05:01<05:54, 17.04it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  46%|████▌     | 5160/11174 [05:03<05:53, 17.03it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  46%|████▌     | 5160/11174 [05:03<05:53, 17.03it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  46%|████▋     | 5190/11174 [05:04<05:51, 17.03it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  46%|████▋     | 5190/11174 [05:04<05:51, 17.03it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.625] #015Epoch 0:  47%|████▋     | 5220/11174 [05:06<05:49, 17.03it/s, loss=2.3, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.625]#015Epoch 0:  47%|████▋     | 5220/11174 [05:06<05:49, 17.03it/s, loss=1.86, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.538]#015Epoch 0:  47%|████▋     | 5250/11174 [05:08<05:47, 17.03it/s, loss=1.86, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.538]#015Epoch 0:  47%|████▋     | 5250/11174 [05:08<05:47, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.570]#015Epoch 0:  47%|████▋     | 5280/11174 [05:10<05:46, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.570]#015Epoch 0:  47%|████▋     | 5280/11174 [05:10<05:46, 17.03it/s, loss=2.04, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.590]#015Epoch 0:  48%|████▊     | 5310/11174 [05:11<05:44, 17.03it/s, loss=2.04, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.590]#015Epoch 0:  48%|████▊     | 5310/11174 [05:11<05:44, 17.03it/s, loss=1.66, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  48%|████▊     | 5340/11174 [05:13<05:42, 17.03it/s, loss=1.66, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  48%|████▊     | 5340/11174 [05:13<05:42, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  48%|████▊     | 5370/11174 [05:15<05:40, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  48%|████▊     | 5370/11174 [05:15<05:40, 17.03it/s, loss=2.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.628]#015Epoch 0:  48%|████▊     | 5400/11174 [05:17<05:38, 17.03it/s, loss=2.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.628]#015Epoch 0:  48%|████▊     | 5400/11174 [05:17<05:38, 17.03it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  49%|████▊     | 5430/11174 [05:18<05:37, 17.03it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.568]#015Epoch 0:  49%|████▊     | 5430/11174 [05:18<05:37, 17.03it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.576] #015Epoch 0:  49%|████▉     | 5460/11174 [05:20<05:35, 17.03it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.576]#015Epoch 0:  49%|████▉     | 5460/11174 [05:20<05:35, 17.03it/s, loss=2.31, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542]#015Epoch 0:  49%|████▉     | 5490/11174 [05:22<05:33, 17.03it/s, loss=2.31, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.542]#015Epoch 0:  49%|████▉     | 5490/11174 [05:22<05:33, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.590]#015Epoch 0:  49%|████▉     | 5520/11174 [05:24<05:32, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.590]#015Epoch 0:  49%|████▉     | 5520/11174 [05:24<05:32, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.611]#015Epoch 0:  50%|████▉     | 5550/11174 [05:25<05:30, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.611]#015Epoch 0:  50%|████▉     | 5550/11174 [05:25<05:30, 17.03it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  50%|████▉     | 5580/11174 [05:27<05:28, 17.03it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  50%|████▉     | 5580/11174 [05:27<05:28, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  50%|█████     | 5610/11174 [05:29<05:26, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  50%|█████     | 5610/11174 [05:29<05:26, 17.03it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  50%|█████     | 5640/11174 [05:31<05:24, 17.03it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  50%|█████     | 5640/11174 [05:31<05:24, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  51%|█████     | 5670/11174 [05:32<05:23, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  51%|█████     | 5670/11174 [05:32<05:23, 17.03it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.574]#015Epoch 0:  51%|█████     | 5700/11174 [05:34<05:21, 17.03it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.574]#015Epoch 0:  51%|█████     | 5700/11174 [05:34<05:21, 17.03it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  51%|█████▏    | 5730/11174 [05:36<05:19, 17.03it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  51%|█████▏    | 5730/11174 [05:36<05:19, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  52%|█████▏    | 5760/11174 [05:38<05:17, 17.03it/s, loss=1.95, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  52%|█████▏    | 5760/11174 [05:38<05:17, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.596]#015Epoch 0:  52%|█████▏    | 5790/11174 [05:39<05:16, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.596]#015Epoch 0:  52%|█████▏    | 5790/11174 [05:39<05:16, 17.03it/s, loss=1.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  52%|█████▏    | 5820/11174 [05:41<05:14, 17.03it/s, loss=1.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  52%|█████▏    | 5820/11174 [05:41<05:14, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  52%|█████▏    | 5850/11174 [05:43<05:12, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  52%|█████▏    | 5850/11174 [05:43<05:12, 17.03it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  53%|█████▎    | 5880/11174 [05:45<05:10, 17.03it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  53%|█████▎    | 5880/11174 [05:45<05:10, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  53%|█████▎    | 5910/11174 [05:47<05:09, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.551]#015Epoch 0:  53%|█████▎    | 5910/11174 [05:47<05:09, 17.03it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  53%|█████▎    | 5940/11174 [05:48<05:07, 17.03it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  53%|█████▎    | 5940/11174 [05:48<05:07, 17.03it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  53%|█████▎    | 5970/11174 [05:50<05:05, 17.03it/s, loss=2.07, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  53%|█████▎    | 5970/11174 [05:50<05:05, 17.03it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  54%|█████▎    | 6000/11174 [05:52<05:03, 17.03it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  54%|█████▎    | 6000/11174 [05:52<05:03, 17.03it/s, loss=2.05, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  54%|█████▍    | 6030/11174 [05:54<05:02, 17.03it/s, loss=2.05, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  54%|█████▍    | 6030/11174 [05:54<05:02, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.543]#015Epoch 0:  54%|█████▍    | 6060/11174 [05:55<05:00, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.543]#015Epoch 0:  54%|█████▍    | 6060/11174 [05:55<05:00, 17.03it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.570]#015Epoch 0:  55%|█████▍    | 6090/11174 [05:57<04:58, 17.02it/s, loss=1.79, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.570]#015Epoch 0:  55%|█████▍    | 6090/11174 [05:57<04:58, 17.02it/s, loss=1.97, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.625]#015Epoch 0:  55%|█████▍    | 6120/11174 [05:59<04:56, 17.02it/s, loss=1.97, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.625]#015Epoch 0:  55%|█████▍    | 6120/11174 [05:59<04:56, 17.02it/s, loss=2.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  55%|█████▌    | 6150/11174 [06:01<04:55, 17.02it/s, loss=2.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  55%|█████▌    | 6150/11174 [06:01<04:55, 17.02it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  55%|█████▌    | 6180/11174 [06:03<04:53, 17.02it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  55%|█████▌    | 6180/11174 [06:03<04:53, 17.02it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.552]#015Epoch 0:  56%|█████▌    | 6210/11174 [06:04<04:51, 17.02it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.552]#015Epoch 0:  56%|█████▌    | 6210/11174 [06:04<04:51, 17.02it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.647]#015Epoch 0:  56%|█████▌    | 6240/11174 [06:06<04:49, 17.02it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.647]#015Epoch 0:  56%|█████▌    | 6240/11174 [06:06<04:49, 17.02it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.523]#015Epoch 0:  56%|█████▌    | 6270/11174 [06:08<04:48, 17.02it/s, loss=1.62, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.523]#015Epoch 0:  56%|█████▌    | 6270/11174 [06:08<04:48, 17.02it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  56%|█████▋    | 6300/11174 [06:09<04:46, 17.03it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.604]#015Epoch 0:  56%|█████▋    | 6300/11174 [06:09<04:46, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.608]#015Epoch 0:  57%|█████▋    | 6330/11174 [06:11<04:44, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.608]#015Epoch 0:  57%|█████▋    | 6330/11174 [06:11<04:44, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  57%|█████▋    | 6360/11174 [06:13<04:42, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  57%|█████▋    | 6360/11174 [06:13<04:42, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.605]#015Epoch 0:  57%|█████▋    | 6390/11174 [06:15<04:40, 17.03it/s, loss=1.83, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.605]#015Epoch 0:  57%|█████▋    | 6390/11174 [06:15<04:40, 17.03it/s, loss=1.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.558]#015Epoch 0:  57%|█████▋    | 6420/11174 [06:17<04:39, 17.03it/s, loss=1.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.558]#015Epoch 0:  57%|█████▋    | 6420/11174 [06:17<04:39, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.552]#015Epoch 0:  58%|█████▊    | 6450/11174 [06:18<04:37, 17.03it/s, loss=1.84, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.552]#015Epoch 0:  58%|█████▊    | 6450/11174 [06:18<04:37, 17.03it/s, loss=2.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  58%|█████▊    | 6480/11174 [06:20<04:35, 17.03it/s, loss=2.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.566]#015Epoch 0:  58%|█████▊    | 6480/11174 [06:20<04:35, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]#015Epoch 0:  58%|█████▊    | 6510/11174 [06:22<04:33, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]#015Epoch 0:  58%|█████▊    | 6510/11174 [06:22<04:33, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  59%|█████▊    | 6540/11174 [06:24<04:32, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  59%|█████▊    | 6540/11174 [06:24<04:32, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.634]#015Epoch 0:  59%|█████▉    | 6570/11174 [06:25<04:30, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.634]#015Epoch 0:  59%|█████▉    | 6570/11174 [06:25<04:30, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.610] #015Epoch 0:  59%|█████▉    | 6600/11174 [06:27<04:28, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.610]#015Epoch 0:  59%|█████▉    | 6600/11174 [06:27<04:28, 17.03it/s, loss=2.16, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  59%|█████▉    | 6630/11174 [06:29<04:26, 17.03it/s, loss=2.16, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.583]#015Epoch 0:  59%|█████▉    | 6630/11174 [06:29<04:26, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  60%|█████▉    | 6660/11174 [06:31<04:25, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  60%|█████▉    | 6660/11174 [06:31<04:25, 17.03it/s, loss=1.67, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588]#015Epoch 0:  60%|█████▉    | 6690/11174 [06:32<04:23, 17.03it/s, loss=1.67, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588]#015Epoch 0:  60%|█████▉    | 6690/11174 [06:32<04:23, 17.03it/s, loss=1.57, v_num=0, val_f1_epoc

h=0.058, val_loss_epoch=0.679, train_f1=0.620]#015Epoch 0:  60%|██████    | 6720/11174 [06:34<04:21, 17.03it/s, loss=1.57, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.620]#015Epoch 0:  60%|██████    | 6720/11174 [06:34<04:21, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.637]#015Epoch 0:  60%|██████    | 6750/11174 [06:36<04:19, 17.03it/s, loss=1.81, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.637]#015Epoch 0:  60%|██████    | 6750/11174 [06:36<04:19, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.610]#015Epoch 0:  61%|██████    | 6780/11174 [06:38<04:17, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.610]#015Epoch 0:  61%|██████    | 6780/11174 [06:38<04:17, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.563]#015Epoch 0:  61%|██████    | 6810/11174 [06:39<04:16, 17.03it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.563]#015Epoch 0:  61%|██████    | 6810/11174 [06:39<04:16, 17.03it/s, loss=1.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.582]#015Epoch 0:  61%|██████    | 6840/11174 [06:41<04:14, 17.03it/s, loss=1.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.582]#015Epoch 0:  61%|██████    | 6840/11174 [06:41<04:14, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.678]#015Epoch 0:  61%|██████▏   | 6870/11174 [06:43<04:12, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.678]#015Epoch 0:  61%|██████▏   | 6870/11174 [06:43<04:12, 17.03it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  62%|██████▏   | 6900/11174 [06:44<04:10, 17.04it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  62%|██████▏   | 6900/11174 [06:44<04:10, 17.04it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  62%|██████▏   | 6930/11174 [06:46<04:09, 17.04it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.546]#015Epoch 0:  62%|██████▏   | 6930/11174 [06:46<04:09, 17.04it/s, loss=1.36, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  62%|██████▏   | 6960/11174 [06:48<04:07, 17.03it/s, loss=1.36, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  62%|██████▏   | 6960/11174 [06:48<04:07, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.539]#015Epoch 0:  63%|██████▎   | 6990/11174 [06:50<04:05, 17.04it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.539]#015Epoch 0:  63%|██████▎   | 6990/11174 [06:50<04:05, 17.04it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.664]#015Epoch 0:  63%|██████▎   | 7020/11174 [06:52<04:03, 17.04it/s, loss=2.03, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.664]#015Epoch 0:  63%|██████▎   | 7020/11174 [06:52<04:03, 17.04it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.691]#015Epoch 0:  63%|██████▎   | 7050/11174 [06:53<04:02, 17.04it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.691]#015Epoch 0:  63%|██████▎   | 7050/11174 [06:53<04:02, 17.04it/s, loss=1.28, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  63%|██████▎   | 7080/11174 [06:55<04:00, 17.03it/s, loss=1.28, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  63%|██████▎   | 7080/11174 [06:55<04:00, 17.03it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  64%|██████▎   | 7110/11174 [06:57<03:58, 17.03it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  64%|██████▎   | 7110/11174 [06:57<03:58, 17.03it/s, loss=1.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.545]#015Epoch 0:  64%|██████▍   | 7140/11174 [06:59<03:56, 17.03it/s, loss=1.77, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.545]#015Epoch 0:  64%|██████▍   | 7140/11174 [06:59<03:56, 17.03it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  64%|██████▍   | 7170/11174 [07:01<03:55, 17.03it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  64%|██████▍   | 7170/11174 [07:01<03:55, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  64%|██████▍   | 7200/11174 [07:02<03:53, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  64%|██████▍   | 7200/11174 [07:02<03:53, 17.03it/s, loss=1.4, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588] #015Epoch 0:  65%|██████▍   | 7230/11174 [07:04<03:51, 17.03it/s, loss=1.4, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588]#015Epoch 0:  65%|██████▍   | 7230/11174 [07:04<03:51, 17.03it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  65%|██████▍   | 7260/11174 [07:06<03:49, 17.03it/s, loss=2.12, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  65%|██████▍   | 7260/11174 [07:06<03:49, 17.03it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  65%|██████▌   | 7290/11174 [07:08<03:48, 17.03it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  65%|██████▌   | 7290/11174 [07:08<03:48, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  66%|██████▌   | 7320/11174 [07:09<03:46, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  66%|██████▌   | 7320/11174 [07:09<03:46, 17.03it/s, loss=1.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  66%|██████▌   | 7350/11174 [07:11<03:44, 17.03it/s, loss=1.25, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.573]#015Epoch 0:  66%|██████▌   | 7350/11174 [07:11<03:44, 17.03it/s, loss=1.53, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  66%|██████▌   | 7380/11174 [07:13<03:42, 17.03it/s, loss=1.53, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  66%|██████▌   | 7380/11174 [07:13<03:42, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.567] #015Epoch 0:  66%|██████▋   | 7410/11174 [07:15<03:41, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.567]#015Epoch 0:  66%|██████▋   | 7410/11174 [07:15<03:41, 17.03it/s, loss=1.34, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.657]#015Epoch 0:  67%|██████▋   | 7440/11174 [07:16<03:39, 17.03it/s, loss=1.34, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.657]#015Epoch 0:  67%|██████▋   | 7440/11174 [07:16<03:39, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  67%|██████▋   | 7470/11174 [07:18<03:37, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.584]#015Epoch 0:  67%|██████▋   | 7470/11174 [07:18<03:37, 17.03it/s, loss=2.36, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  67%|██████▋   | 7500/11174 [07:20<03:35, 17.04it/s, loss=2.36, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  67%|██████▋   | 7500/11174 [07:20<03:35, 17.04it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  67%|██████▋   | 7530/11174 [07:22<03:33, 17.03it/s, loss=1.82, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.561]#015Epoch 0:  67%|██████▋   | 7530/11174 [07:22<03:33, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  68%|██████▊   | 7560/11174 [07:23<03:32, 17.03it/s, loss=2.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  68%|██████▊   | 7560/11174 [07:23<03:32, 17.03it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  68%|██████▊   | 7590/11174 [07:25<03:30, 17.04it/s, loss=1.87, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.593]#015Epoch 0:  68%|██████▊   | 7590/11174 [07:25<03:30, 17.04it/s, loss=2.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.624]#015Epoch 0:  68%|██████▊   | 7620/11174 [07:27<03:28, 17.04it/s, loss=2.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.624]#015Epoch 0:  68%|██████▊   | 7620/11174 [07:27<03:28, 17.04it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  68%|██████▊   | 7650/11174 [07:29<03:26, 17.04it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  68%|██████▊   | 7650/11174 [07:29<03:26, 17.04it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:  69%|██████▊   | 7680/11174 [07:30<03:25, 17.04it/s, loss=2.11, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.548]#015Epoch 0:  69%|██████▊   | 7680/11174 [07:30<03:25, 17.04it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  69%|██████▉   | 7710/11174 [07:32<03:23, 17.03it/s, loss=2.08, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.560]#015Epoch 0:  69%|██████▉   | 7710/11174 [07:32<03:23, 17.03it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  69%|██████▉   | 7740/11174 [07:34<03:21, 17.04it/s, loss=1.78, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  69%|██████▉   | 7740/11174 [07:34<03:21, 17.04it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.591]#015Epoch 0:  70%|██████▉   | 7770/11174 [07:36<03:19, 17.03it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.591]#015Epoch 0:  70%|██████▉   | 7770/11174 [07:36<03:19, 17.03it/s, loss=1.76, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.571]#015Epoch 0:  70%|██████▉   | 7800/11174 [07:37<03:18, 17.04it/s, loss=1.76, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.571]#015Epoch 0:  70%|██████▉   | 7800/11174 [07:37<03:18, 17.04it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.597]#015Epoch 0:  70%|███████   | 7830/11174 [07:39<03:16, 17.04it/s, loss=1.93, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.597]#015Epoch 0:  70%|███████   | 7830/11174 [07:39<03:16, 17.04it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514] #015Epoch 0:  70%|███████   | 7860/11174 [07:41<03:14, 17.03it/s, loss=2.2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.514]#015Epoch 0:  70%|███████   | 7860/11174 [07:41<03:14, 17.03it/s, loss=1.15, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.587]#015Epoch 0:  71%|███████   | 7890/11174 [07:43<03:12, 17.04it/s, loss=1.15, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.587]#015Epoch 0:  71%|███████   | 7890/11174 [07:43<03:12, 17.04it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.572]#015Epoch 0:  71%|███████   | 7920/11174 [07:44<03:11, 17.03it/s, loss=1.74, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.572]#015Epoch 0:  71%|███████   | 7920/11174 [07:44<03:11, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.643]#015Epoch 0:  71%|███████   | 7950/11174 [07:46<03:09, 17.04it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.643]#015Epoch 0:  71%|███████   | 7950/11174 [07:46<03:09, 17.04it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.628]#015Epoch 0:  71%|███████▏  | 7980/11174 [07:48<03:07, 17.03it/s, loss=2.01, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.628]#015Epoch 0:  71%|███████▏  | 7980/11174 [07:48<03:07, 17.03it/s, loss=1.18, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.656]#015Epoch 0:  72%|███████▏  | 8010/11174 [07:50<03:05, 17.03it/s, loss=1.18, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.656]#015Epoch 0:  72%|███████▏  | 8010/11174 [07:50<03:05, 17.03it/s, loss=1.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  72%|███████▏  | 8040/11174 [07:51<03:03, 17.04it/s, loss=1.21, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  72%|███████▏  | 8040/11174 [07:51<03:03, 17.04it/s, loss=1.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.638]#015Epoch 0:  72%|███████▏  | 8070/11174 [07:53<03:02, 17.03it/s, loss=1.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.638]#015Epoch 0:  72%|███████▏  | 8070/11174 [07:53<03:02, 17.03it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.674] #015Epoch 0:  72%|███████▏  | 8100/11174 [07:55<03:00, 17.04it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.674]#015Epoch 0:  72%|███████▏  | 8100/11174 [07:55<03:00, 17.04it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  73%|███████▎  | 8130/11174 [07:57<02:58, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  73%|███████▎  | 8130/11174 [07:57<02:58, 17.03it/s, loss=1.45, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  73%|███████▎  | 8160/11174 [07:59<02:57, 17.03it/s, loss=1.45, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.557]#015Epoch 0:  73%|███████▎  | 8160/11174 [07:59<02:57, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.597] #015Epoch 0:  73%|███████▎  | 8190/11174 [08:00<02:55, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.597]#015Epoch 0:  73%|███████▎  | 8190/11174 [08:00<02:55, 17.03it/s, loss=1.61, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.675]#015Epoch 0:  74%|███████▎  | 8220/11174 [08:02<02:53, 17.03it/s, loss=1.61, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.675]#015Epoch 0:  74%|███████▎  | 8220/11174 [08:02<02:53, 17.03it/s, loss=1.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.690]#015Epoch 0:  74%|███████▍  | 8250/11174 [08:04<02:51, 17.03it/s, loss=1.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.690]#015Epoch 0:  74%|███████▍  | 8250/11174 [08:04<02:51, 17.03it/s, loss=2.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.575]#015Epoch 0:  74%|███████▍  | 8280/11174 [08:06<02:49, 17.03it/s, loss=2.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.575]#015Epoch 0:  74%|███████▍  | 8280/11174 [08:06<02:49, 17.03it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0

.601]#015Epoch 0:  74%|███████▍  | 8310/11174 [08:08<02:48, 17.03it/s, loss=1.52, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.601]#015Epoch 0:  74%|███████▍  | 8310/11174 [08:08<02:48, 17.03it/s, loss=1.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.643]#015Epoch 0:  75%|███████▍  | 8340/11174 [08:09<02:46, 17.03it/s, loss=1.42, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.643]#015Epoch 0:  75%|███████▍  | 8340/11174 [08:09<02:46, 17.03it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  75%|███████▍  | 8370/11174 [08:11<02:44, 17.03it/s, loss=1.46, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  75%|███████▍  | 8370/11174 [08:11<02:44, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.576]#015Epoch 0:  75%|███████▌  | 8400/11174 [08:13<02:42, 17.03it/s, loss=1.69, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.576]#015Epoch 0:  75%|███████▌  | 8400/11174 [08:13<02:42, 17.03it/s, loss=1.44, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.559]#015Epoch 0:  75%|███████▌  | 8430/11174 [08:14<02:41, 17.03it/s, loss=1.44, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.559]#015Epoch 0:  75%|███████▌  | 8430/11174 [08:14<02:41, 17.03it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  76%|███████▌  | 8460/11174 [08:16<02:39, 17.03it/s, loss=1.99, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  76%|███████▌  | 8460/11174 [08:16<02:39, 17.03it/s, loss=1.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  76%|███████▌  | 8490/11174 [08:18<02:37, 17.03it/s, loss=1.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.579]#015Epoch 0:  76%|███████▌  | 8490/11174 [08:18<02:37, 17.03it/s, loss=2.04, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  76%|███████▌  | 8520/11174 [08:20<02:35, 17.03it/s, loss=2.04, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  76%|███████▌  | 8520/11174 [08:20<02:35, 17.03it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.663]#015Epoch 0:  77%|███████▋  | 8550/11174 [08:21<02:34, 17.03it/s, loss=2.33, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.663]#015Epoch 0:  77%|███████▋  | 8550/11174 [08:21<02:34, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.611]#015Epoch 0:  77%|███████▋  | 8580/11174 [08:23<02:32, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.611]#015Epoch 0:  77%|███████▋  | 8580/11174 [08:23<02:32, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  77%|███████▋  | 8610/11174 [08:25<02:30, 17.03it/s, loss=1.59, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.592]#015Epoch 0:  77%|███████▋  | 8610/11174 [08:25<02:30, 17.03it/s, loss=1.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  77%|███████▋  | 8640/11174 [08:27<02:28, 17.03it/s, loss=1.27, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.630]#015Epoch 0:  77%|███████▋  | 8640/11174 [08:27<02:28, 17.03it/s, loss=1.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  78%|███████▊  | 8670/11174 [08:28<02:26, 17.03it/s, loss=1.85, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  78%|███████▊  | 8670/11174 [08:28<02:26, 17.03it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.657] #015Epoch 0:  78%|███████▊  | 8700/11174 [08:30<02:25, 17.04it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.657]#015Epoch 0:  78%|███████▊  | 8700/11174 [08:30<02:25, 17.04it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588]#015Epoch 0:  78%|███████▊  | 8730/11174 [08:32<02:23, 17.03it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.588]#015Epoch 0:  78%|███████▊  | 8730/11174 [08:32<02:23, 17.03it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  78%|███████▊  | 8760/11174 [08:34<02:21, 17.03it/s, loss=1.73, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  78%|███████▊  | 8760/11174 [08:34<02:21, 17.03it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.599]#015Epoch 0:  79%|███████▊  | 8790/11174 [08:35<02:19, 17.04it/s, loss=1.65, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.599]#015Epoch 0:  79%|███████▊  | 8790/11174 [08:35<02:19, 17.04it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  79%|███████▉  | 8820/11174 [08:37<02:18, 17.03it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  79%|███████▉  | 8820/11174 [08:37<02:18, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  79%|███████▉  | 8850/11174 [08:39<02:16, 17.04it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.594]#015Epoch 0:  79%|███████▉  | 8850/11174 [08:39<02:16, 17.04it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.661]#015Epoch 0:  79%|███████▉  | 8880/11174 [08:41<02:14, 17.04it/s, loss=1.91, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.661]#015Epoch 0:  79%|███████▉  | 8880/11174 [08:41<02:14, 17.04it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.640]#015Epoch 0:  80%|███████▉  | 8910/11174 [08:43<02:12, 17.03it/s, loss=1.89, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.640]#015Epoch 0:  80%|███████▉  | 8910/11174 [08:43<02:12, 17.03it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  80%|████████  | 8940/11174 [08:44<02:11, 17.04it/s, loss=1.92, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  80%|████████  | 8940/11174 [08:44<02:11, 17.04it/s, loss=1.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  80%|████████  | 8970/11174 [08:46<02:09, 17.03it/s, loss=1.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  80%|████████  | 8970/11174 [08:46<02:09, 17.03it/s, loss=2.15, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  81%|████████  | 9000/11174 [08:48<02:07, 17.04it/s, loss=2.15, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.631]#015Epoch 0:  81%|████████  | 9000/11174 [08:48<02:07, 17.04it/s, loss=1.55, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.634]#015Epoch 0:  81%|████████  | 9030/11174 [08:50<02:05, 17.03it/s, loss=1.55, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.634]#015Epoch 0:  81%|████████  | 9030/11174 [08:50<02:05, 17.03it/s, loss=1.53, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.635]#015Epoch 0:  81%|████████  | 9060/11174 [08:51<02:04, 17.03it/s, loss=1.53, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.635]#015Epoch 0:  81%|████████  | 9060/11174 [08:51<02:04, 17.03it/s, loss=1.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.609]#015Epoch 0:  81%|████████▏ | 9090/11174 [08:53<02:02, 17.04it/s, loss=1.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.609]#015Epoch 0:  81%|████████▏ | 9090/11174 [08:53<02:02, 17.04it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.646] #015Epoch 0:  82%|████████▏ | 9120/11174 [08:55<02:00, 17.03it/s, loss=1.7, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.646]#015Epoch 0:  82%|████████▏ | 9120/11174 [08:55<02:00, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  82%|████████▏ | 9150/11174 [08:57<01:58, 17.03it/s, loss=2.09, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.549]#015Epoch 0:  82%|████████▏ | 9150/11174 [08:57<01:58, 17.03it/s, loss=2.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  82%|████████▏ | 9180/11174 [08:59<01:57, 17.02it/s, loss=2.06, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  82%|████████▏ | 9180/11174 [08:59<01:57, 17.02it/s, loss=1.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.614]#015Epoch 0:  82%|████████▏ | 9210/11174 [09:01<01:55, 17.02it/s, loss=1.23, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.614]#015Epoch 0:  82%|████████▏ | 9210/11174 [09:01<01:55, 17.02it/s, loss=1.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  83%|████████▎ | 9240/11174 [09:02<01:53, 17.02it/s, loss=1.64, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.565]#015Epoch 0:  83%|████████▎ | 9240/11174 [09:02<01:53, 17.02it/s, loss=2.1, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618] #015Epoch 0:  83%|████████▎ | 9270/11174 [09:04<01:51, 17.02it/s, loss=2.1, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.618]#015Epoch 0:  83%|████████▎ | 9270/11174 [09:04<01:51, 17.02it/s, loss=1.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  83%|████████▎ | 9300/11174 [09:06<01:50, 17.03it/s, loss=1.26, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.603]#015Epoch 0:  83%|████████▎ | 9300/11174 [09:06<01:50, 17.03it/s, loss=1.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.646]#015Epoch 0:  83%|████████▎ | 9330/11174 [09:08<01:48, 17.02it/s, loss=1.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.646]#015Epoch 0:  83%|████████▎ | 9330/11174 [09:08<01:48, 17.02it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.553]#015Epoch 0:  84%|████████▍ | 9360/11174 [09:09<01:46, 17.02it/s, loss=1.47, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.553]#015Epoch 0:  84%|████████▍ | 9360/11174 [09:09<01:46, 17.02it/s, loss=1.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.586]#015Epoch 0:  84%|████████▍ | 9390/11174 [09:11<01:44, 17.03it/s, loss=1.39, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.586]#015Epoch 0:  84%|████████▍ | 9390/11174 [09:11<01:44, 17.03it/s, loss=1.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.626]#015Epoch 0:  84%|████████▍ | 9420/11174 [09:13<01:43, 17.03it/s, loss=1.24, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.626]#015Epoch 0:  84%|████████▍ | 9420/11174 [09:13<01:43, 17.03it/s, loss=1.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  85%|████████▍ | 9450/11174 [09:14<01:41, 17.03it/s, loss=1.56, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.578]#015Epoch 0:  85%|████████▍ | 9450/11174 [09:14<01:41, 17.03it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  85%|████████▍ | 9480/11174 [09:16<01:39, 17.03it/s, loss=1.68, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.607]#015Epoch 0:  85%|████████▍ | 9480/11174 [09:16<01:39, 17.03it/s, loss=1.41, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.638]#015Epoch 0:  85%|████████▌ | 9510/11174 [09:18<01:37, 17.03it/s, loss=1.41, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.638]#015Epoch 0:  85%|████████▌ | 9510/11174 [09:18<01:37, 17.03it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.703] #015Epoch 0:  85%|████████▌ | 9540/11174 [09:20<01:35, 17.03it/s, loss=1.8, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.703]#015Epoch 0:  85%|████████▌ | 9540/11174 [09:20<01:35, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.617]#015Epoch 0:  86%|████████▌ | 9570/11174 [09:22<01:34, 17.03it/s, loss=1.54, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.617]#015Epoch 0:  86%|████████▌ | 9570/11174 [09:22<01:34, 17.03it/s, loss=2.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  86%|████████▌ | 9600/11174 [09:23<01:32, 17.03it/s, loss=2.32, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.615]#015Epoch 0:  86%|████████▌ | 9600/11174 [09:23<01:32, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.613]#015Epoch 0:  86%|████████▌ | 9630/11174 [09:25<01:30, 17.03it/s, loss=1.75, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.613]#015Epoch 0:  86%|████████▌ | 9630/11174 [09:25<01:30, 17.03it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.678]   #015Epoch 0:  86%|████████▋ | 9660/11174 [09:27<01:28, 17.03it/s, loss=2, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.678]#015Epoch 0:  86%|████████▋ | 9660/11174 [09:27<01:28, 17.03it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  87%|████████▋ | 9690/11174 [09:29<01:27, 17.03it/s, loss=1.71, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.580]#015Epoch 0:  87%|████████▋ | 9690/11174 [09:29<01:27, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550] #015Epoch 0:  87%|████████▋ | 9720/11174 [09:30<01:25, 17.03it/s, loss=1.9, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.550]#015Epoch 0:  87%|████████▋ | 9720/11174 [09:30<01:25, 17.03it/s, loss=1.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.530]#015Epoch 0:  87%|████████▋ | 9750/11174 [09:32<01:23, 17.03it/s, loss=1.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.530]#015Epoch 0:  87%|████████▋ | 9750/11174 [09:32<01:23, 17.03it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.606]#015Epoch 0:  88%|████████▊ | 9780/11174 [09:34<01:21, 17.03it/s, loss=1.58, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.606]#015Epoch 0:  88%|████████▊ | 9780/11174 [09:34<01:21, 17.03it/s, loss=1.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  88%|████████▊ | 9810/11174 [09:36<01:20, 17.03it/s, loss=1.37, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.662]#015Epoch 0:  88%|████████▊ | 9810/11174 [09:36<01:20, 17.03it/s, loss=1.55, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.622]#015Epoch 0:  88%|████████▊ | 9840/11174 [09:37<01:18, 17.03it/s, loss=1.55, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.622]#015Epoch 0:  88%|████████▊ | 9840/11174 [09:37<01:18, 17.03it/s, loss=1.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.612]#015Epoch 0:  88%|████████▊ | 9870/11174 [09
:39<01:16, 17.03it/s, loss=1.49, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.612]#015Epoch 0:  88%|████████▊ | 9870/11174 [09:39<01:16, 17.03it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]#015Epoch 0:  89%|████████▊ | 9900/11174 [09:41<01:14, 17.02it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/1275 [00:00<?, ?it/s]#033[A
#015Validating:   2%|▏         | 30/1275 [00:00<00:39, 31.55it/s]#033[A#015Epoch 0:  89%|████████▉ | 9930/11174 [09:42<01:12, 17.05it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:   5%|▍         | 60/1275 [00:01<00:35, 34.57it/s]#033[A#015Epoch 0:  89%|████████▉ | 9960/11174 [09:43<01:11, 17.08it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:   7%|▋         | 90/1275 [00:02<00:32, 36.93it/s]#033[A#015Epoch 0:  89%|████████▉ | 9990/11174 [09:43<01:09, 17.11it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:   9%|▉         | 120/1275 [00:02<00:29, 38.75it/s]#033[A#015Epoch 0:  90%|████████▉ | 10020/11174 [09:44<01:07, 17.14it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  12%|█▏        | 150/1275 [00:03<00:27, 40.24it/s]#033[A#015Epoch 0:  90%|████████▉ | 10050/11174 [09:45<01:05, 17.17it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  14%|█▍        | 180/1275 [00:04<00:26, 41.18it/s]#033[A#015Epoch 0:  90%|█████████ | 10080/11174 [09:45<01:03, 17.20it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  16%|█▋        | 210/1275 [00:05<00:25, 42.05it/s]#033[A#015Epoch 0:  90%|█████████ | 10110/11174 [09:46<01:01, 17.23it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  19%|█▉        | 240/1275 [00:05<00:24, 42.62it/s]#033[A#015Epoch 0:  91%|█████████ | 10140/11174 [09:47<00:59, 17.27it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  21%|██        | 270/1275 [00:06<00:23, 43.05it/s]#033[A#015Epoch 0:  91%|█████████ | 10170/11174 [09:47<00:58, 17.30it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]

#015Validating:  24%|██▎       | 300/1275 [00:07<00:22, 43.39it/s]#033[A#015Epoch 0:  91%|█████████▏| 10200/11174 [09:48<00:56, 17.33it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  26%|██▌       | 330/1275 [00:07<00:21, 43.31it/s]#033[A#015Epoch 0:  92%|█████████▏| 10230/11174 [09:49<00:54, 17.36it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  28%|██▊       | 360/1275 [00:08<00:20, 43.66it/s]#033[A#015Epoch 0:  92%|█████████▏| 10260/11174 [09:50<00:52, 17.39it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  31%|███       | 390/1275 [00:09<00:20, 43.39it/s]#033[A#015Epoch 0:  92%|█████████▏| 10290/11174 [09:50<00:50, 17.42it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  33%|███▎      | 420/1275 [00:09<00:19, 43.69it/s]#033[A#015Epoch 0:  92%|█████████▏| 10320/11174 [09:51<00:48, 17.45it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  35%|███▌      | 450/1275 [00:10<00:18, 44.02it/s]#033[A#015Epoch 0:  93%|█████████▎| 10350/11174 [09:52<00:47, 17.48it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  38%|███▊      | 480/1275 [00:11<00:18, 43.72it/s]#033[A#015Epoch 0:  93%|█████████▎| 10380/11174 [09:52<00:45, 17.51it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  40%|████      | 510/1275 [00:11<00:17, 43.56it/s]#033[A#015Epoch 0:  93%|█████████▎| 10410/11174 [09:53<00:43, 17.54it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  42%|████▏     | 540/1275 [00:12<00:16, 43.41it/s]#033[A#015Epoch 0:  93%|█████████▎| 10440/11174 [09:54<00:41, 17.57it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  45%|████▍     | 570/1275 [00:13<00:16, 43.03it/s]#033[A#015Epoch 0:  94%|█████████▎| 10470/11174 [09:54<00:39, 17.60it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  47%|████▋     | 600/1275 [00:13<00:15, 43.34it/s]#033[A#015Epoch 0:  94%|█████████▍| 10500/11174 [09:55<00:38, 17.63it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  49%|████▉     | 630/1275 [00:14<00:14, 43.72it/s]#033[A#015Epoch 0:  94%|█████████▍| 10530/11174 [09:56<00:36, 17.66it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  52%|█████▏    | 660/1275 [00:15<00:13, 44.01it/s]#033[A#015Epoch 0:  95%|█████████▍| 10560/11174 [09:56<00:34, 17.69it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  54%|█████▍    | 690/1275 [00:16<00:13, 43.86it/s]#033[A#015Epoch 0:  95%|█████████▍| 10590/11174 [09:57<00:32, 17.72it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  56%|█████▋    | 720/1275 [00:16<00:12, 43.68it/s]#033[A#015Epoch 0:  95%|█████████▌| 10620/11174 [09:58<00:31, 17.75it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  59%|█████▉    | 750/1275 [00:17<00:12, 43.55it/s]#033[A#015Epoch 0:  95%|█████████▌| 10650/11174 [09:58<00:29, 17.78it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  61%|██████    | 780/1275 [00:18<00:11, 43.01it/s]#033[A#015Epoch 0:  96%|█████████▌| 10680/11174 [09:59<00:27, 17.81it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  64%|██████▎   | 810/1275 [00:18<00:10, 42.90it/s]#033[A#015Epoch 0:  96%|█████████▌| 10710/11174 [10:00<00:26, 17.84it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  66%|██████▌   | 840/1275 [00:19<00:10, 42.31it/s]#033[A#015Epoch 0:  96%|█████████▌| 10740/11174 [10:01<00:24, 17.87it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  68%|██████▊   | 870/1275 [00:20<00:09, 41.37it/s]#033[A#015Epoch 0:  96%|█████████▋| 10770/11174 [10:01<00:22, 17.89it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  71%|███████   | 900/1275 [00:20<00:08, 42.14it/s]#033[A#015Epoch 0:  97%|█████████▋| 10800/11174 [10:02<00:20, 17.92it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  73%|███████▎  | 930/1275 [00:21<00:08, 42.33it/s]#033[A#015Epoch 0:  97%|█████████▋| 10830/11174 [10:03<00:19, 17.95it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  75%|███████▌  | 960/1275 [00:22<00:07, 42.68it/s]#033[A#015Epoch 0:  97%|█████████▋| 10860/11174 [10:03<00:17, 17.98it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  78%|███████▊  | 990/1275 [00:23<00:06, 42.95it/s]#033[A#015Epoch 0:  97%|█████████▋| 10890/11174 [10:04<00:15, 18.01it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  80%|████████  | 1020/1275 [00:23<00:05, 43.35it/s]#033[A#015Epoch 0:  98%|█████████▊| 10920/11174 [10:05<00:14, 18.04it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  82%|████████▏ | 1050/1275 [00:24<00:05, 43.65it/s]#033[A#015Epoch 0:  98%|█████████▊| 10950/11174 [10:05<00:12, 18.07it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  85%|████████▍ | 1080/1275 [00:25<00:04, 43.67it/s]#033[A#015Epoch 0:  98%|█████████▊| 10980/11174 [10:06<00:10, 18.10it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  87%|████████▋ | 1110/1275 [00:25<00:03, 44.17it/s]#033[A#015Epoch 0:  99%|█████████▊| 11010/11174 [10:07<00:09, 18.13it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  89%|████████▉ | 1140/1275 [00:26<00:03, 44.51it/s]#033[A#015Epoch 0:  99%|█████████▉| 11040/11174 [10:07<00:07, 18.16it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  92%|█████████▏| 1170/1275 [00:27<00:02, 43.76it/s]#033[A#015Epoch 0:  99%|█████████▉| 11070/11174 [10:08<00:05, 18.19it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  94%|█████████▍| 1200/1275 [00:27<00:01, 44.10it/s]#033[A#015Epoch 0:  99%|█████████▉| 11100/11174 [10:09<00:04, 18.22it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  96%|█████████▋| 1230/1275 [00:28<00:01, 44.28it/s]#033[A#015Epoch 0: 100%|█████████▉| 11130/11174 [10:10<00:02, 18.24it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating:  99%|█████████▉| 1260/1275 [00:29<00:00, 44.55it/s]#033[A#015Epoch 0: 100%|█████████▉| 11160/11174 [10:10<00:00, 18.27it/s, loss=1.72, v_num=0, val_f1_epoch=0.058, val_loss_epoch=0.679, train_f1=0.581]
#015Validating: 100%|██████████| 1275/1275 [00:29<00:00, 44.99it/s]#033[A#015Epoch 0: 100%|██████████| 11174/11174 [10:11<00:00, 18.28it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]
#015                                                               #033[A#015Epoch 0:   0%|          | 0/11174 [00:00<?, ?it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]            #015Epoch 1:   0%|          | 0/11174 [00:00<?, ?it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   0%|          | 30/11174 [00:02<12:59, 14.30it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   0%|          | 30/11174 [00:02<12:59, 14.30it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 60/11174 [00:03<11:59, 15.45it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 60/11174 [00:03<11:59, 15.44it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 90/11174 [00:05<11:31, 16.02it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 90/11174 [00:05<11:31, 16.02it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 120/11174 [00:07<11:31, 16.00it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|          | 120/11174 [00:07<11:31, 16.00it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:   1%|▏         | 150/11174 [00:09<11:26, 16.05it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   1%|▏         | 150/11174 [00:09<11:26, 16.05it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 180/11174 [00:11<11:22, 16.10it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 180/11174 [00:11<11:22, 16.10it/s, loss=2.05, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 210/11174 [00:13<11:21, 16.10it/s, loss=2.05, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 210/11174 [00:13<11:21, 16.10it/s, loss=1.99, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 240/11174 [00:14<11:10, 16.31it/s, loss=1.99, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 240/11174 [00:14<11:10, 16.31it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.586, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 270/11174 [00:16<11:07, 16.34it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.586, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   2%|▏         | 270/11174 [00:16<11:07, 16.34it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 300/11174 [00:18<11:00, 16.46it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 300/11174 [00:18<11:00, 16.46it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 330/11174 [00:20<10:58, 16.47it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 330/11174 [00:20<10:58, 16.47it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 360/11174 [00:21<10:55, 16.49it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 360/11174 [00:21<10:55, 16.49it/s, loss=1.79, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 390/11174 [00:23<10:51, 16.56it/s, loss=1.79, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   3%|▎         | 390/11174 [00:23<10:51, 16.56it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 420/11174 [00:25<10:49, 16.57it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 420/11174 [00:25<10:49, 16.57it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.693, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 450/11174 [00:27<10:44, 16.63it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.693, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 450/11174 [00:27<10:45, 16.63it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 480/11174 [00:28<10:43, 16.62it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   4%|▍         | 480/11174 [00:28<10:43, 16.62it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▍         | 510/11174 [00:30<10:42, 16.61it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▍         | 510/11174 [00:30<10:42, 16.61it/s, loss=2.14, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▍         | 540/11174 [00:32<10:38, 16.66it/s, loss=2.14, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▍         | 540/11174 [00:32<10:38, 16.66it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▌         | 570/11174 [00:34<10:36, 16.66it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▌         | 570/11174 [00:34<10:36, 16.66it/s, loss=1.85, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▌         | 600/11174 [00:35<10:33, 16.70it/s, loss=1.85, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   5%|▌         | 600/11174 [00:35<10:33, 16.70it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.605, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 630/11174 [00:37<10:31, 16.69it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.605, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 630/11174 [00:37<10:31, 16.69it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 660/11174 [00:39<10:30, 16.66it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 660/11174 [00:39<10:30, 16.66it/s, loss=2.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 690/11174 [00:41<10:28, 16.67it/s, loss=2.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▌         | 690/11174 [00:41<10:28, 16.67it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▋         | 720/11174 [00:43<10:29, 16.61it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.616, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   6%|▋         | 720/11174 [00:43<10:29, 16.61it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:   7%|▋         | 750/11174 [00:45<10:26, 16.63it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   7%|▋         | 750/11174 [00:45<10:26, 16.63it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   7%|▋         | 780/11174 [00:46<10:25, 16.62it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   7%|▋         | 780/11174 [00:46<10:25, 16.62it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.567, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:   7%|▋         | 810/11174 [00:48<10:23, 16.63it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.567, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   7%|▋         | 810/11174 [00:48<10:23, 16.63it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 840/11174 [00:50<10:19, 16.67it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 840/11174 [00:50<10:19, 16.67it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.553, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 870/11174 [00:52<10:18, 16.66it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.553, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 870/11174 [00:52<10:18, 16.66it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.743, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 900/11174 [00:53<10:15, 16.68it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.743, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 900/11174 [00:53<10:15, 16.68it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 930/11174 [00:55<10:14, 16.67it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.580, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   8%|▊         | 930/11174 [00:55<10:14, 16.67it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.543, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▊         | 960/11174 [00:57<10:12, 16.67it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.543, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▊         | 960/11174 [00:57<10:12, 16.67it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:   9%|▉         | 990/11174 [00:59<10:10, 16.69it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▉         | 990/11174 [00:59<10:10, 16.69it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.605, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▉         | 1020/11174 [01:01<10:08, 16.69it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.605, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▉         | 1020/11174 [01:01<10:08, 16.69it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.548, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:   9%|▉         | 1050/11174 [01:02<10:05, 16.71it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.548, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:   9%|▉         | 1050/11174 [01:02<10:05, 16.71it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|▉         | 1080/11174 [01:04<10:04, 16.71it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|▉         | 1080/11174 [01:04<10:04, 16.71it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|▉         | 1110/11174 [01:06<10:02, 16.70it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|▉         | 1110/11174 [01:06<10:02, 16.70it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|█         | 1140/11174 [01:08<10:00, 16.70it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|█         | 1140/11174 [01:08<10:01, 16.70it/s, loss=2.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|█         | 1170/11174 [01:10<09:59, 16.69it/s, loss=2.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  10%|█         | 1170/11174 [01:10<09:59, 16.69it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.623, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█         | 1200/11174 [01:11<09:56, 16.71it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.623, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█         | 1200/11174 [01:11<09:56, 16.71it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█         | 1230/11174 [01:13<09:55, 16.71it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█         | 1230/11174 [01:13<09:55, 16.71it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█▏        | 1260/11174 [01:15<09:53, 16.71it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  11%|█▏        | 1260/11174 [01:15<09:53, 16.71it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  12%|█▏        | 1290/11174 [01:17<09:50, 16.73it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1290/11174 [01:17<09:50, 16.73it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.583, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1320/11174 [01:18<09:49, 16.72it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.583, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1320/11174 [01:18<09:49, 16.72it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, t

rain_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1350/11174 [01:20<09:46, 16.74it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1350/11174 [01:20<09:46, 16.74it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1380/11174 [01:22<09:45, 16.73it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  12%|█▏        | 1380/11174 [01:22<09:45, 16.73it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1410/11174 [01:24<09:43, 16.74it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1410/11174 [01:24<09:43, 16.74it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1440/11174 [01:25<09:41, 16.75it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1440/11174 [01:25<09:41, 16.75it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1470/11174 [01:27<09:39, 16.75it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.563, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1470/11174 [01:27<09:39, 16.75it/s, loss=2.3, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.571, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  13%|█▎        | 1500/11174 [01:29<09:37, 16.76it/s, loss=2.3, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.571, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  13%|█▎        | 1500/11174 [01:29<09:37, 16.76it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▎        | 1530/11174 [01:31<09:35, 16.76it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▎        | 1530/11174 [01:31<09:35, 16.76it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.693, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1560/11174 [01:33<09:33, 16.75it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.693, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1560/11174 [01:33<09:33, 16.75it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1590/11174 [01:34<09:31, 16.77it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1590/11174 [01:34<09:31, 16.77it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1620/11174 [01:36<09:30, 16.75it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  14%|█▍        | 1620/11174 [01:36<09:30, 16.75it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▍        | 1650/11174 [01:38<09:27, 16.77it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▍        | 1650/11174 [01:38<09:27, 16.77it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▌        | 1680/11174 [01:40<09:26, 16.77it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▌        | 1680/11174 [01:40<09:26, 16.77it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.572, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▌        | 1710/11174 [01:42<09:24, 16.76it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.572, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  15%|█▌        | 1710/11174 [01:42<09:24, 16.76it/s, loss=2.04, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.691, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1740/11174 [01:43<09:22, 16.77it/s, loss=2.04, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.691, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1740/11174 [01:43<09:22, 16.77it/s, loss=1.17, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1770/11174 [01:45<09:21, 16.75it/s, loss=1.17, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1770/11174 [01:45<09:21, 16.75it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1800/11174 [01:47<09:19, 16.74it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.621, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▌        | 1800/11174 [01:47<09:19, 16.74it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▋        | 1830/11174 [01:49<09:18, 16.74it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  16%|█▋        | 1830/11174 [01:49<09:18, 16.74it/s, loss=1.21, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1860/11174 [01:51<09:16, 16.72it/s, loss=1.21, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1860/11174 [01:51<09:16, 16.72it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1890/11174 [01:52<09:14, 16.74it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1890/11174 [01:52<09:14, 16.74it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1920/11174 [01:54<09:12, 16.74it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1920/11174 [01:54<09:12, 16.74it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1950/11174 [01:56<09:10, 16.75it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  17%|█▋        | 1950/11174 [01:56<09:10, 16.75it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 1980/11174 [01:58<09:08, 16.75it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 1980/11174 [01:58<09:08, 16.75it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.629, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 2010/11174 [01:59<09:07, 16.75it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.629, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 2010/11174 [01:59<09:07, 16.75it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 2040/11174 [02:01<09:04, 16.77it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  18%|█▊        | 2040/11174 [02:01<09:04, 16.77it/s, loss=1.66, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▊        | 2070/11174 [02:03<09:02, 16.77it/s, loss=1.66, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▊        | 2070/11174 [02:03<09:02, 16.77it/s, loss=1.11, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.579, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2100/11174 [02:05<09:00, 16.78it/s, loss=1.11, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.579, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2100/11174 [02:05<09:00, 16.78it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2130/11174 [02:06<08:59, 16.77it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2130/11174 [02:06<08:59, 16.77it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.731, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2160/11174 [02:08<08:57, 16.77it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.731, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  19%|█▉        | 2160/11174 [02:08<08:57, 16.77it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|█▉        | 2190/11174 [02:10<08:55, 16.79it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|█▉        | 2190/11174 [02:10<08:55, 16.79it/s, loss=1.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|█▉        | 2220/11174 [02:12<08:53, 16.78it/s, loss=1.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|█▉        | 2220/11174 [02:12<08:53, 16.78it/s, loss=1.17, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|██        | 2250/11174 [02:14<08:51, 16.79it/s, loss=1.17, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|██        | 2250/11174 [02:14<08:51, 16.79it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.564, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|██        | 2280/11174 [02:15<08:49, 16.79it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.564, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  20%|██        | 2280/11174 [02:15<08:49, 16.79it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2310/11174 [02:17<08:48, 16.79it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2310/11174 [02:17<08:48, 16.79it/s, loss=1.97, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2340/11174 [02:19<08:46, 16.79it/s, loss=1.97, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2340/11174 [02:19<08:46, 16.79it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2370/11174 [02:21<08:44, 16.79it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██        | 2370/11174 [02:21<08:44, 16.79it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██▏       | 2400/11174 [02:22<08:42, 16.81it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  21%|██▏       | 2400/11174 [02:22<08:42, 16.81it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2430/11174 [02:24<08:40, 16.81it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.568, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2430/11174 [02:24<08:40, 16.81it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2460/11174 [02:26<08:38, 16.80it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2460/11174 [02:26<08:38, 16.80it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.702, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2490/11174 [02:28<08:36, 16.81it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.702, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  22%|██▏       | 2490/11174 [02:28<08:36, 16.81it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2520/11174 [02:29<08:34, 16.81it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2520/11174 [02:29<08:34, 16.81it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2550/11174 [02:31<08:32, 16.81it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2550/11174 [02:31<08:32, 16.81it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2580/11174 [02:33<08:31, 16.81it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2580/11174 [02:33<08:31, 16.81it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2610/11174 [02:35<08:29, 16.81it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  23%|██▎       | 2610/11174 [02:35<08:29, 16.80it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.595, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▎       | 2640/11174 [02:36<08:27, 16.82it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.595, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▎       | 2640/11174 [02:36<08:27, 16.82it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%

|██▍       | 2670/11174 [02:38<08:25, 16.81it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▍       | 2670/11174 [02:38<08:25, 16.81it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▍       | 2700/11174 [02:40<08:23, 16.82it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▍       | 2700/11174 [02:40<08:23, 16.82it/s, loss=2.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.757, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▍       | 2730/11174 [02:42<08:22, 16.80it/s, loss=2.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.757, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  24%|██▍       | 2730/11174 [02:42<08:22, 16.80it/s, loss=2.15, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.701, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▍       | 2760/11174 [02:44<08:21, 16.78it/s, loss=2.15, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.701, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▍       | 2760/11174 [02:44<08:21, 16.78it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▍       | 2790/11174 [02:46<08:19, 16.79it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▍       | 2790/11174 [02:46<08:19, 16.79it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▌       | 2820/11174 [02:47<08:17, 16.79it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  25%|██▌       | 2820/11174 [02:47<08:17, 16.79it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2850/11174 [02:49<08:15, 16.80it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.729, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2850/11174 [02:49<08:15, 16.80it/s, loss=1.74, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2880/11174 [02:51<08:13, 16.80it/s, loss=1.74, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2880/11174 [02:51<08:13, 16.80it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2910/11174 [02:53<08:12, 16.79it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▌       | 2910/11174 [02:53<08:12, 16.79it/s, loss=2.04, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.601, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▋       | 2940/11174 [02:55<08:10, 16.80it/s, loss=2.04, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.601, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  26%|██▋       | 2940/11174 [02:55<08:10, 16.80it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  27%|██▋       | 2970/11174 [02:56<08:08, 16.80it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 2970/11174 [02:56<08:08, 16.80it/s, loss=1.67, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3000/11174 [02:58<08:06, 16.80it/s, loss=1.67, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3000/11174 [02:58<08:06, 16.80it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3030/11174 [03:00<08:04, 16.80it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3030/11174 [03:00<08:04, 16.80it/s, loss=2.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.704, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3060/11174 [03:02<08:03, 16.80it/s, loss=2.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.704, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  27%|██▋       | 3060/11174 [03:02<08:03, 16.80it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3090/11174 [03:03<08:01, 16.80it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3090/11174 [03:03<08:01, 16.80it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3120/11174 [03:05<07:59, 16.80it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.618, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3120/11174 [03:05<07:59, 16.80it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3150/11174 [03:07<07:57, 16.81it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3150/11174 [03:07<07:57, 16.81it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3180/11174 [03:09<07:55, 16.82it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  28%|██▊       | 3180/11174 [03:09<07:55, 16.82it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.623, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▊       | 3210/11174 [03:10<07:53, 16.81it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.623, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▊       | 3210/11174 [03:10<07:53, 16.81it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.589, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▉       | 3240/11174 [03:12<07:51, 16.82it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.589, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▉       | 3240/11174 [03:12<07:51, 16.82it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.571, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▉       | 3270/11174 [03:14<07:49, 16.82it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.571, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  29%|██▉       | 3270/11174 [03:14<07:49, 16.82it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.599, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|██▉       | 3300/11174 [03:16<07:47, 16.83it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.599, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|██▉       | 3300/11174 [03:16<07:47, 16.83it/s, loss=1.91, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|██▉       | 3330/11174 [03:17<07:46, 16.82it/s, loss=1.91, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|██▉       | 3330/11174 [03:17<07:46, 16.82it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|███       | 3360/11174 [03:19<07:44, 16.82it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.585, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|███       | 3360/11174 [03:19<07:44, 16.82it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.566, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|███       | 3390/11174 [03:21<07:42, 16.83it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.566, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  30%|███       | 3390/11174 [03:21<07:42, 16.83it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3420/11174 [03:23<07:40, 16.83it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3420/11174 [03:23<07:40, 16.83it/s, loss=2.08, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.584, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3450/11174 [03:24<07:38, 16.83it/s, loss=2.08, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.584, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3450/11174 [03:24<07:38, 16.83it/s, loss=1.29, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.576, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3480/11174 [03:26<07:37, 16.83it/s, loss=1.29, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.576, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███       | 3480/11174 [03:26<07:37, 16.83it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███▏      | 3510/11174 [03:28<07:35, 16.83it/s, loss=1.88, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  31%|███▏      | 3510/11174 [03:28<07:35, 16.83it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3540/11174 [03:30<07:33, 16.83it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3540/11174 [03:30<07:33, 16.83it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3570/11174 [03:32<07:31, 16.83it/s, loss=1.68, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3570/11174 [03:32<07:31, 16.83it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3600/11174 [03:33<07:29, 16.84it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.588, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3600/11174 [03:33<07:29, 16.84it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3630/11174 [03:35<07:27, 16.84it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  32%|███▏      | 3630/11174 [03:35<07:27, 16.84it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.658, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3660/11174 [03:37<07:26, 16.84it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.658, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3660/11174 [03:37<07:26, 16.84it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3690/11174 [03:39<07:24, 16.84it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3690/11174 [03:39<07:24, 16.84it/s, loss=1.23, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3720/11174 [03:40<07:22, 16.84it/s, loss=1.23, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  33%|███▎      | 3720/11174 [03:40<07:22, 16.84it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▎      | 3750/11174 [03:42<07:20, 16.84it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▎      | 3750/11174 [03:42<07:20, 16.84it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.727, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3780/11174 [03:44<07:19, 16.83it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.727, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3780/11174 [03:44<07:19, 16.83it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.625, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3810/11174 [03:46<07:17, 16.82it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.625, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3810/11174 [03:46<07:17, 16.82it/s, loss=1.97, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3840/11174 [03:48<07:15, 16.82it/s, loss=1.97, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  34%|███▍      | 3840/11174 [03:48<07:15, 16.82it/s, loss=2.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▍      | 3870/11174 [03:50<07:14, 16.82it/s, loss=2.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.604, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▍      | 3870/11174 [03:50<07:14, 16.82it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.736, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▍      | 3900/11174 [03:51<07:12, 16.82it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.736, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▍      | 3900/11174 [03:51<07:12, 16.82it/s, loss=1.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▌      | 3930/11174 [03:53<07:10, 16.82it/s, loss=1.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▌      | 3930/11174 [03:53<07:10, 16.82it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▌      | 3960/11174 [03:55<07:08, 16.82it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  35%|███▌      | 3960/11174 [03:55<07:08, 16.82it/

s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 3990/11174 [03:57<07:07, 16.82it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 3990/11174 [03:57<07:07, 16.82it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 4020/11174 [03:59<07:05, 16.82it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 4020/11174 [03:59<07:05, 16.82it/s, loss=2.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 4050/11174 [04:00<07:03, 16.83it/s, loss=2.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  36%|███▌      | 4050/11174 [04:00<07:03, 16.83it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4080/11174 [04:02<07:01, 16.83it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.659, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4080/11174 [04:02<07:01, 16.83it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4110/11174 [04:04<06:59, 16.82it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4110/11174 [04:04<06:59, 16.82it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.569, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4140/11174 [04:06<06:58, 16.83it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.569, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4140/11174 [04:06<06:58, 16.83it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4170/11174 [04:07<06:56, 16.82it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  37%|███▋      | 4170/11174 [04:07<06:56, 16.82it/s, loss=1.05, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4200/11174 [04:09<06:54, 16.82it/s, loss=1.05, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4200/11174 [04:09<06:54, 16.82it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.584, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4230/11174 [04:11<06:52, 16.82it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.584, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4230/11174 [04:11<06:52, 16.82it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.733, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4260/11174 [04:13<06:51, 16.82it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.733, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4260/11174 [04:13<06:51, 16.82it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  38%|███▊      | 4290/11174 [04:14<06:49, 16.83it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  38%|███▊      | 4290/11174 [04:14<06:49, 16.83it/s, loss=1.19, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▊      | 4320/11174 [04:16<06:47, 16.83it/s, loss=1.19, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.606, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▊      | 4320/11174 [04:16<06:47, 16.83it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4350/11174 [04:18<06:45, 16.83it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4350/11174 [04:18<06:45, 16.83it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4380/11174 [04:20<06:43, 16.83it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4380/11174 [04:20<06:43, 16.83it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4410/11174 [04:22<06:41, 16.83it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  39%|███▉      | 4410/11174 [04:22<06:41, 16.83it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|███▉      | 4440/11174 [04:23<06:39, 16.84it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|███▉      | 4440/11174 [04:23<06:39, 16.84it/s, loss=2.14, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.567, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|████      | 4470/11174 [04:25<06:38, 16.83it/s, loss=2.14, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.567, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|████      | 4470/11174 [04:25<06:38, 16.83it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|████      | 4500/11174 [04:27<06:36, 16.84it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  40%|████      | 4500/11174 [04:27<06:36, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████      | 4530/11174 [04:29<06:34, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████      | 4530/11174 [04:29<06:34, 16.84it/s, loss=1.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  41%|████      | 4560/11174 [04:30<06:32, 16.83it/s, loss=1.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████      | 4560/11174 [04:30<06:32, 16.83it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.721, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████      | 4590/11174 [04:32<06:30, 16.84it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.721, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████      | 4590/11174 [04:32<06:30, 16.84it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████▏     | 4620/11174 [04:34<06:29, 16.84it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  41%|████▏     | 4620/11174 [04:34<06:29, 16.84it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4650/11174 [04:36<06:27, 16.85it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4650/11174 [04:36<06:27, 16.85it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.557, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4680/11174 [04:37<06:25, 16.84it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.557, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4680/11174 [04:37<06:25, 16.84it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4710/11174 [04:39<06:23, 16.84it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4710/11174 [04:39<06:23, 16.84it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.596, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4740/11174 [04:41<06:21, 16.85it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.596, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  42%|████▏     | 4740/11174 [04:41<06:21, 16.85it/s, loss=2.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  43%|████▎     | 4770/11174 [04:43<06:20, 16.85it/s, loss=2.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4770/11174 [04:43<06:20, 16.85it/s, loss=1.29, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4800/11174 [04:44<06:18, 16.85it/s, loss=1.29, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.666, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4800/11174 [04:44<06:18, 16.85it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4830/11174 [04:46<06:16, 16.83it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4830/11174 [04:46<06:16, 16.83it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4860/11174 [04:48<06:15, 16.83it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  43%|████▎     | 4860/11174 [04:48<06:15, 16.83it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.690, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4890/11174 [04:50<06:13, 16.84it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.690, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4890/11174 [04:50<06:13, 16.84it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4920/11174 [04:52<06:11, 16.83it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4920/11174 [04:52<06:11, 16.83it/s, loss=1.26, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.707, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4950/11174 [04:53<06:09, 16.84it/s, loss=1.26, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.707, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  44%|████▍     | 4950/11174 [04:53<06:09, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▍     | 4980/11174 [04:55<06:07, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▍     | 4980/11174 [04:55<06:07, 16.84it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▍     | 5010/11174 [04:57<06:06, 16.83it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▍     | 5010/11174 [04:57<06:06, 16.83it/s, loss=1.11, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▌     | 5040/11174 [04:59<06:04, 16.84it/s, loss=1.11, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▌     | 5040/11174 [04:59<06:04, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▌     | 5070/11174 [05:01<06:02, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  45%|████▌     | 5070/11174 [05:01<06:02, 16.84it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▌     | 5100/11174 [05:02<06:00, 16.84it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▌     | 5100/11174 [05:02<06:00, 16.84it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  46%|████▌     | 5130/11174 [05:04<05:58, 16.84it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.631, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▌     | 5130/11174 [05:04<05:58, 16.84it/s, loss=2.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▌     | 5160/11174 [05:06<05:57, 16.84it/s, loss=2.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▌     | 5160/11174 [05:06<05:57, 16.84it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▋     | 5190/11174 [05:08<05:55, 16.84it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  46%|████▋     | 5190/11174 [05:08<05:55, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.700, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5220/11174 [05:09<05:53, 16.84it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.700, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5220/11174 [05:09<05:53, 16.84it/s, loss=1.06, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5250/11174 [05:11<05:51, 16.84it/s, loss=1.06, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5250/11174 [05:11<05:51, 16.84it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_l

oss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5280/11174 [05:13<05:50, 16.84it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  47%|████▋     | 5280/11174 [05:13<05:50, 16.84it/s, loss=1.85, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5310/11174 [05:15<05:48, 16.84it/s, loss=1.85, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.603, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5310/11174 [05:15<05:48, 16.84it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5340/11174 [05:17<05:46, 16.84it/s, loss=1.89, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.570, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5340/11174 [05:17<05:46, 16.84it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.705, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5370/11174 [05:18<05:44, 16.84it/s, loss=1.72, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.705, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5370/11174 [05:18<05:44, 16.84it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5400/11174 [05:20<05:42, 16.85it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.608, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  48%|████▊     | 5400/11174 [05:20<05:42, 16.85it/s, loss=2.09, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▊     | 5430/11174 [05:22<05:40, 16.85it/s, loss=2.09, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▊     | 5430/11174 [05:22<05:40, 16.85it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5460/11174 [05:24<05:39, 16.84it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5460/11174 [05:24<05:39, 16.84it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5490/11174 [05:25<05:37, 16.85it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.679, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5490/11174 [05:25<05:37, 16.85it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5520/11174 [05:27<05:35, 16.85it/s, loss=1.81, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  49%|████▉     | 5520/11174 [05:27<05:35, 16.85it/s, loss=1.67, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|████▉     | 5550/11174 [05:29<05:33, 16.85it/s, loss=1.67, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|████▉     | 5550/11174 [05:29<05:33, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|████▉     | 5580/11174 [05:31<05:32, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.650, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|████▉     | 5580/11174 [05:31<05:32, 16.85it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|█████     | 5610/11174 [05:33<05:30, 16.85it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.592, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|█████     | 5610/11174 [05:33<05:30, 16.85it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.672, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|█████     | 5640/11174 [05:34<05:28, 16.85it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.672, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  50%|█████     | 5640/11174 [05:34<05:28, 16.85it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  51%|█████     | 5670/11174 [05:36<05:26, 16.85it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  51%|█████     | 5670/11174 [05:36<05:26, 16.85it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.707, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  51%|█████     | 5700/11174 [05:38<05:24, 16.85it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.707, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  51%|█████     | 5700/11174 [05:38<05:24, 16.85it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  51%|█████▏    | 5730/11174 [05:40<05:23, 16.85it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  51%|█████▏    | 5730/11174 [05:40<05:23, 16.85it/s, loss=2.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.619, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5760/11174 [05:41<05:21, 16.84it/s, loss=2.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.619, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5760/11174 [05:41<05:21, 16.84it/s, loss=1.25, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5790/11174 [05:43<05:19, 16.85it/s, loss=1.25, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.609, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5790/11174 [05:43<05:19, 16.85it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5820/11174 [05:45<05:17, 16.84it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5820/11174 [05:45<05:17, 16.84it/s, loss=2.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.593, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5850/11174 [05:47<05:15, 16.85it/s, loss=2.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.593, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  52%|█████▏    | 5850/11174 [05:47<05:15, 16.85it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.593, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5880/11174 [05:49<05:14, 16.84it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.593, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5880/11174 [05:49<05:14, 16.84it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5910/11174 [05:51<05:12, 16.84it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5910/11174 [05:51<05:12, 16.84it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5940/11174 [05:52<05:10, 16.84it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5940/11174 [05:52<05:10, 16.84it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5970/11174 [05:54<05:09, 16.84it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  53%|█████▎    | 5970/11174 [05:54<05:09, 16.84it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  54%|█████▎    | 6000/11174 [05:56<05:07, 16.84it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  54%|█████▎    | 6000/11174 [05:56<05:07, 16.84it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  54%|█████▍    | 6030/11174 [05:58<05:05, 16.84it/s, loss=1.55, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  54%|█████▍    | 6030/11174 [05:58<05:05, 16.84it/s, loss=1.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  54%|█████▍    | 6060/11174 [05:59<05:03, 16.84it/s, loss=1.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.670, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  54%|█████▍    | 6060/11174 [05:59<05:03, 16.84it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.560, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▍    | 6090/11174 [06:01<05:01, 16.84it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.560, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▍    | 6090/11174 [06:01<05:01, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▍    | 6120/11174 [06:03<05:00, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.636, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▍    | 6120/11174 [06:03<05:00, 16.84it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▌    | 6150/11174 [06:05<04:58, 16.84it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▌    | 6150/11174 [06:05<04:58, 16.84it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  55%|█████▌    | 6180/11174 [06:06<04:56, 16.84it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.678, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  55%|█████▌    | 6180/11174 [06:06<04:56, 16.84it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6210/11174 [06:08<04:54, 16.84it/s, loss=1.35, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.590, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6210/11174 [06:08<04:54, 16.84it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6240/11174 [06:10<04:52, 16.84it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6240/11174 [06:10<04:52, 16.84it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6270/11174 [06:12<04:51, 16.84it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.655, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▌    | 6270/11174 [06:12<04:51, 16.84it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▋    | 6300/11174 [06:13<04:49, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  56%|█████▋    | 6300/11174 [06:13<04:49, 16.85it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6330/11174 [06:15<04:47, 16.84it/s, loss=1.73, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6330/11174 [06:15<04:47, 16.84it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.619, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6360/11174 [06:17<04:45, 16.84it/s, loss=1.92, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.619, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6360/11174 [06:17<04:45, 16.84it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6390/11174 [06:19<04:44, 16.84it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6390/11174 [06:19<04:44, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6420/11174 [06:21<04:42, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  57%|█████▋    | 6420/11174 [06:21<04:42, 16.84it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6450/11174 [06:22<04:40, 16.85it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.649, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6450/11174 [06:22<04:40, 16.85it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6480/11174 [06:24<04:38, 16.85it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6480/11174 [06:24<04:38, 16.85it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6510/11174 [06:26<04:36, 16.84it/s, loss=1.75, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  58%|█████▊    | 6510/11174 [06:26<04:36, 16.84it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▊    | 6540/11174 [06:28<04:35, 16.85it/s, loss=1.48, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.688, val_f1_step=0

.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▊    | 6540/11174 [06:28<04:35, 16.85it/s, loss=1.87, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6570/11174 [06:29<04:33, 16.85it/s, loss=1.87, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6570/11174 [06:29<04:33, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6600/11174 [06:31<04:31, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.614, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6600/11174 [06:31<04:31, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6630/11174 [06:33<04:29, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  59%|█████▉    | 6630/11174 [06:33<04:29, 16.85it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|█████▉    | 6660/11174 [06:35<04:27, 16.85it/s, loss=1.58, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.689, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|█████▉    | 6660/11174 [06:35<04:27, 16.85it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|█████▉    | 6690/11174 [06:37<04:26, 16.85it/s, loss=1.38, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.598, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|█████▉    | 6690/11174 [06:37<04:26, 16.85it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|██████    | 6720/11174 [06:38<04:24, 16.85it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.674, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|██████    | 6720/11174 [06:38<04:24, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|██████    | 6750/11174 [06:40<04:22, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.664, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  60%|██████    | 6750/11174 [06:40<04:22, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6780/11174 [06:42<04:20, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6780/11174 [06:42<04:20, 16.85it/s, loss=1.91, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.587, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6810/11174 [06:44<04:19, 16.84it/s, loss=1.91, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.587, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6810/11174 [06:44<04:19, 16.84it/s, loss=1.18, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6840/11174 [06:46<04:17, 16.84it/s, loss=1.18, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████    | 6840/11174 [06:46<04:17, 16.84it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████▏   | 6870/11174 [06:48<04:15, 16.83it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  61%|██████▏   | 6870/11174 [06:48<04:15, 16.83it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  62%|██████▏   | 6900/11174 [06:49<04:13, 16.84it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  62%|██████▏   | 6900/11174 [06:49<04:13, 16.84it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  62%|██████▏   | 6930/11174 [06:51<04:12, 16.83it/s, loss=1.34, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  62%|██████▏   | 6930/11174 [06:51<04:12, 16.83it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  62%|██████▏   | 6960/11174 [06:53<04:10, 16.83it/s, loss=1.8, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  62%|██████▏   | 6960/11174 [06:53<04:10, 16.83it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 6990/11174 [06:55<04:08, 16.84it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.646, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 6990/11174 [06:55<04:08, 16.84it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7020/11174 [06:57<04:06, 16.83it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7020/11174 [06:57<04:06, 16.83it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7050/11174 [06:58<04:04, 16.84it/s, loss=1.43, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.573, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7050/11174 [06:58<04:04, 16.84it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7080/11174 [07:00<04:03, 16.84it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  63%|██████▎   | 7080/11174 [07:00<04:03, 16.84it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▎   | 7110/11174 [07:02<04:01, 16.84it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▎   | 7110/11174 [07:02<04:01, 16.84it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.561, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▍   | 7140/11174 [07:04<03:59, 16.84it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.561, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▍   | 7140/11174 [07:04<03:59, 16.84it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  64%|██████▍   | 7170/11174 [07:05<03:57, 16.84it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.692, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▍   | 7170/11174 [07:05<03:57, 16.84it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▍   | 7200/11174 [07:07<03:56, 16.83it/s, loss=1.2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  64%|██████▍   | 7200/11174 [07:07<03:56, 16.83it/s, loss=1.63, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.710, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▍   | 7230/11174 [07:09<03:54, 16.83it/s, loss=1.63, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.710, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▍   | 7230/11174 [07:09<03:54, 16.83it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▍   | 7260/11174 [07:11<03:52, 16.83it/s, loss=1.83, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.591, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▍   | 7260/11174 [07:11<03:52, 16.83it/s, loss=1.74, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▌   | 7290/11174 [07:12<03:50, 16.84it/s, loss=1.74, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  65%|██████▌   | 7290/11174 [07:12<03:50, 16.84it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7320/11174 [07:14<03:48, 16.84it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7320/11174 [07:14<03:48, 16.84it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7350/11174 [07:16<03:47, 16.84it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.615, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7350/11174 [07:16<03:47, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7380/11174 [07:18<03:45, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▌   | 7380/11174 [07:18<03:45, 16.84it/s, loss=2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]   #015Epoch 1:  66%|██████▋   | 7410/11174 [07:20<03:43, 16.84it/s, loss=2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.667, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  66%|██████▋   | 7410/11174 [07:20<03:43, 16.84it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7440/11174 [07:21<03:41, 16.84it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7440/11174 [07:21<03:41, 16.84it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.700, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7470/11174 [07:23<03:39, 16.84it/s, loss=1.86, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.700, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7470/11174 [07:23<03:39, 16.84it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7500/11174 [07:25<03:38, 16.84it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7500/11174 [07:25<03:38, 16.84it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7530/11174 [07:27<03:36, 16.84it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.640, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  67%|██████▋   | 7530/11174 [07:27<03:36, 16.84it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7560/11174 [07:28<03:34, 16.84it/s, loss=1.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.627, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7560/11174 [07:28<03:34, 16.84it/s, loss=1.28, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7590/11174 [07:30<03:32, 16.84it/s, loss=1.28, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7590/11174 [07:30<03:32, 16.84it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  68%|██████▊   | 7620/11174 [07:32<03:31, 16.84it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7620/11174 [07:32<03:31, 16.84it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.748, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7650/11174 [07:34<03:29, 16.84it/s, loss=1.44, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.748, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  68%|██████▊   | 7650/11174 [07:34<03:29, 16.84it/s, loss=1.66, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  69%|██████▊   | 7680/11174 [07:35<03:27, 16.84it/s, loss=1.66, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  69%|██████▊   | 7680/11174 [07:35<03:27, 16.84it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  69%|██████▉   | 7710/11174 [07:37<03:25, 16.84it/s, loss=1.77, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  69%|██████▉   | 7710/11174 [07:37<03:25, 16.84it/s, loss=2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]   #015Epoch 1:  69%|██████▉   | 7740/11174 [07:39<03:23, 16.85it/s, loss=2, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.643, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  69%|██████▉   | 7740/11174 [07:39<03:23, 16.85it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|██████▉   | 7770/11174 [07:41<03:22, 16.85it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.633, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|██████▉   | 7770/11174 [07:41<03:22, 16.85it/s, loss=1.5, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  70%|██████▉   | 7800/11174 [07:42<03:20, 16.85it/s, loss=1.5, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|█████�

��▉   | 7800/11174 [07:42<03:20, 16.85it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|███████   | 7830/11174 [07:44<03:18, 16.84it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|███████   | 7830/11174 [07:44<03:18, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|███████   | 7860/11174 [07:46<03:16, 16.84it/s, loss=1.59, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  70%|███████   | 7860/11174 [07:46<03:16, 16.84it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.612, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7890/11174 [07:48<03:15, 16.84it/s, loss=1.65, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.612, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7890/11174 [07:48<03:15, 16.84it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7920/11174 [07:50<03:13, 16.84it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7920/11174 [07:50<03:13, 16.84it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7950/11174 [07:52<03:11, 16.84it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.632, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████   | 7950/11174 [07:52<03:11, 16.84it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████▏  | 7980/11174 [07:53<03:09, 16.84it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  71%|███████▏  | 7980/11174 [07:53<03:09, 16.84it/s, loss=1.96, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8010/11174 [07:55<03:07, 16.84it/s, loss=1.96, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8010/11174 [07:55<03:07, 16.84it/s, loss=2.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  72%|███████▏  | 8040/11174 [07:57<03:06, 16.84it/s, loss=2.4, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.607, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8040/11174 [07:57<03:06, 16.84it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8070/11174 [07:59<03:04, 16.84it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8070/11174 [07:59<03:04, 16.84it/s, loss=1.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  72%|███████▏  | 8100/11174 [08:00<03:02, 16.84it/s, loss=1.1, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  72%|███████▏  | 8100/11174 [08:00<03:02, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8130/11174 [08:02<03:00, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8130/11174 [08:02<03:00, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8160/11174 [08:04<02:58, 16.84it/s, loss=1.46, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.644, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8160/11174 [08:04<02:58, 16.84it/s, loss=2.26, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.577, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8190/11174 [08:06<02:57, 16.85it/s, loss=2.26, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.577, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  73%|███████▎  | 8190/11174 [08:06<02:57, 16.85it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.665, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▎  | 8220/11174 [08:07<02:55, 16.85it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.665, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▎  | 8220/11174 [08:07<02:55, 16.85it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  74%|███████▍  | 8250/11174 [08:09<02:53, 16.85it/s, loss=1.9, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▍  | 8250/11174 [08:09<02:53, 16.85it/s, loss=1.24, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▍  | 8280/11174 [08:11<02:51, 16.85it/s, loss=1.24, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.634, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▍  | 8280/11174 [08:11<02:51, 16.85it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.694, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▍  | 8310/11174 [08:13<02:50, 16.85it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.694, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  74%|███████▍  | 8310/11174 [08:13<02:50, 16.85it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▍  | 8340/11174 [08:14<02:48, 16.85it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▍  | 8340/11174 [08:14<02:48, 16.85it/s, loss=1.23, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▍  | 8370/11174 [08:16<02:46, 16.85it/s, loss=1.23, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▍  | 8370/11174 [08:16<02:46, 16.85it/s, loss=1.19, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▌  | 8400/11174 [08:18<02:44, 16.85it/s, loss=1.19, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.684, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▌  | 8400/11174 [08:18<02:44, 16.85it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▌  | 8430/11174 [08:20<02:42, 16.85it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.648, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  75%|███████▌  | 8430/11174 [08:20<02:42, 16.85it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8460/11174 [08:22<02:41, 16.85it/s, loss=1.47, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8460/11174 [08:22<02:41, 16.85it/s, loss=1.39, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8490/11174 [08:23<02:39, 16.85it/s, loss=1.39, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.630, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8490/11174 [08:23<02:39, 16.85it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8520/11174 [08:25<02:37, 16.85it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  76%|███████▌  | 8520/11174 [08:25<02:37, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8550/11174 [08:27<02:35, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.641, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8550/11174 [08:27<02:35, 16.85it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8580/11174 [08:29<02:33, 16.85it/s, loss=1.33, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8580/11174 [08:29<02:33, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8610/11174 [08:30<02:32, 16.85it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8610/11174 [08:30<02:32, 16.85it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.586, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8640/11174 [08:32<02:30, 16.86it/s, loss=1.62, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.586, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  77%|███████▋  | 8640/11174 [08:32<02:30, 16.86it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8670/11174 [08:34<02:28, 16.85it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8670/11174 [08:34<02:28, 16.85it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  78%|███████▊  | 8700/11174 [08:36<02:26, 16.86it/s, loss=1.6, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.669, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8700/11174 [08:36<02:26, 16.86it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8730/11174 [08:37<02:25, 16.85it/s, loss=1.54, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.653, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8730/11174 [08:37<02:25, 16.85it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8760/11174 [08:39<02:23, 16.85it/s, loss=1.57, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.626, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  78%|███████▊  | 8760/11174 [08:39<02:23, 16.85it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▊  | 8790/11174 [08:41<02:21, 16.85it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▊  | 8790/11174 [08:41<02:21, 16.85it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8820/11174 [08:43<02:19, 16.85it/s, loss=1.76, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.661, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8820/11174 [08:43<02:19, 16.85it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8850/11174 [08:45<02:17, 16.85it/s, loss=1.36, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.594, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8850/11174 [08:45<02:17, 16.85it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.552, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8880/11174 [08:47<02:16, 16.85it/s, loss=2.16, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.552, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  79%|███████▉  | 8880/11174 [08:47<02:16, 16.85it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|███████▉  | 8910/11174 [08:48<02:14, 16.85it/s, loss=1.93, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.686, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|███████▉  | 8910/11174 [08:48<02:14, 16.85it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|████████  | 8940/11174 [08:50<02:12, 16.85it/s, loss=1.51, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.677, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|████████  | 8940/11174 [08:50<02:12, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|████████  | 8970/11174 [08:52<02:10, 16.85it/s, loss=1.69, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  80%|████████  | 8970/11174 [08:52<02:10, 16.85it/s, loss=1.31, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9000/11174 [08:54<02:09, 16.85it/s, loss=1.31, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.680, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9000/11174 [08:54<02:09, 16.85it/s, loss=2.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9030/11174 [08:55<02:07, 16.85it/s, loss=2.22, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.628, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9030/11174 [08:55<02:07, 16.85it/s, loss=1.14, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9060/11174 [08:57<02:05, 16.85it/s, loss=1.14, v_

num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.597, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████  | 9060/11174 [08:57<02:05, 16.85it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████▏ | 9090/11174 [08:59<02:03, 16.85it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.617, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  81%|████████▏ | 9090/11174 [08:59<02:03, 16.85it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.576, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9120/11174 [09:01<02:01, 16.85it/s, loss=1.52, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.576, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9120/11174 [09:01<02:01, 16.85it/s, loss=1.13, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9150/11174 [09:02<02:00, 16.86it/s, loss=1.13, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.639, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9150/11174 [09:02<02:00, 16.86it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9180/11174 [09:04<01:58, 16.85it/s, loss=1.64, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.642, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9180/11174 [09:04<01:58, 16.85it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9210/11174 [09:06<01:56, 16.85it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.660, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  82%|████████▏ | 9210/11174 [09:06<01:56, 16.85it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9240/11174 [09:08<01:54, 16.86it/s, loss=1.27, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.699, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9240/11174 [09:08<01:54, 16.86it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9270/11174 [09:10<01:52, 16.85it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.675, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9270/11174 [09:10<01:52, 16.85it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.575, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9300/11174 [09:11<01:51, 16.86it/s, loss=1.53, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.575, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9300/11174 [09:11<01:51, 16.86it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9330/11174 [09:13<01:49, 16.85it/s, loss=1.61, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.697, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  83%|████████▎ | 9330/11174 [09:13<01:49, 16.85it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9360/11174 [09:15<01:47, 16.85it/s, loss=1.71, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.698, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9360/11174 [09:15<01:47, 16.85it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9390/11174 [09:17<01:45, 16.85it/s, loss=1.37, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.635, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9390/11174 [09:17<01:45, 16.85it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9420/11174 [09:18<01:44, 16.85it/s, loss=2.07, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.611, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  84%|████████▍ | 9420/11174 [09:18<01:44, 16.85it/s, loss=1.13, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▍ | 9450/11174 [09:20<01:42, 16.85it/s, loss=1.13, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.637, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▍ | 9450/11174 [09:20<01:42, 16.85it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▍ | 9480/11174 [09:22<01:40, 16.85it/s, loss=1.56, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.610, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▍ | 9480/11174 [09:22<01:40, 16.85it/s, loss=1.24, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▌ | 9510/11174 [09:24<01:38, 16.85it/s, loss=1.24, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.624, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▌ | 9510/11174 [09:24<01:38, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▌ | 9540/11174 [09:26<01:36, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.620, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  85%|████████▌ | 9540/11174 [09:26<01:36, 16.85it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9570/11174 [09:27<01:35, 16.85it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.652, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9570/11174 [09:27<01:35, 16.85it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9600/11174 [09:29<01:33, 16.86it/s, loss=1.32, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.662, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9600/11174 [09:29<01:33, 16.86it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9630/11174 [09:31<01:31, 16.86it/s, loss=1.45, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.581, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▌ | 9630/11174 [09:31<01:31, 16.86it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▋ | 9660/11174 [09:33<01:29, 16.86it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.656, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  86%|████████▋ | 9660/11174 [09:33<01:29, 16.86it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9690/11174 [09:34<01:28, 16.86it/s, loss=1.95, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.647, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9690/11174 [09:34<01:28, 16.86it/s, loss=1.18, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9720/11174 [09:36<01:26, 16.86it/s, loss=1.18, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.657, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9720/11174 [09:36<01:26, 16.86it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9750/11174 [09:38<01:24, 16.86it/s, loss=1.78, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.663, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  87%|████████▋ | 9750/11174 [09:38<01:24, 16.86it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9780/11174 [09:40<01:22, 16.86it/s, loss=1.49, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.600, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9780/11174 [09:40<01:22, 16.86it/s, loss=1.5, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629] #015Epoch 1:  88%|████████▊ | 9810/11174 [09:42<01:20, 16.85it/s, loss=1.5, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.651, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9810/11174 [09:42<01:20, 16.85it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9840/11174 [09:43<01:19, 16.85it/s, loss=1.7, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.668, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9840/11174 [09:43<01:19, 16.85it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9870/11174 [09:45<01:17, 16.85it/s, loss=1.41, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.638, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  88%|████████▊ | 9870/11174 [09:45<01:17, 16.85it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]#015Epoch 1:  89%|████████▊ | 9900/11174 [09:47<01:15, 16.84it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/1275 [00:00<?, ?it/s]#033[A
#015Validating:   2%|▏         | 30/1275 [00:00<00:39, 31.76it/s]#033[A#015Epoch 1:  89%|████████▉ | 9930/11174 [09:48<01:13, 16.87it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:   5%|▍         | 60/1275 [00:01<00:35, 34.42it/s]#033[A#015Epoch 1:  89%|████████▉ | 9960/11174 [09:49<01:11, 16.90it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:   7%|▋         | 90/1275 [00:02<00:32, 36.97it/s]#033[A#015Epoch 1:  89%|████████▉ | 9990/11174 [09:50<01:09, 16.93it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:   9%|▉         | 120/1275 [00:02<00:29, 38.89it/s]#033[A#015Epoch 1:  90%|████████▉ | 10020/11174 [09:50<01:08, 16.96it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  12%|█▏        | 150/1275 [00:03<00:27, 40.35it/s]#033[A#015Epoch 1:  90%|████████▉ | 10050/11174 [09:51<01:06, 16.99it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]

#015Validating:  14%|█▍        | 180/1275 [00:04<00:26, 41.47it/s]#033[A#015Epoch 1:  90%|█████████ | 10080/11174 [09:52<01:04, 17.02it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  16%|█▋        | 210/1275 [00:05<00:25, 42.35it/s]#033[A#015Epoch 1:  90%|█████████ | 10110/11174 [09:52<01:02, 17.05it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  19%|█▉        | 240/1275 [00:05<00:23, 43.13it/s]#033[A#015Epoch 1:  91%|█████████ | 10140/11174 [09:53<01:00, 17.09it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  21%|██        | 270/1275 [00:06<00:23, 43.24it/s]#033[A#015Epoch 1:  91%|█████████ | 10170/11174 [09:54<00:58, 17.12it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  24%|██▎       | 300/1275 [00:07<00:22, 43.40it/s]#033[A#015Epoch 1:  91%|█████████▏| 10200/11174 [09:54<00:56, 17.15it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  26%|██▌       | 330/1275 [00:07<00:21, 43.73it/s]#033[A#015Epoch 1:  92%|█████████▏| 10230/11174 [09:55<00:54, 17.18it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  28%|██▊       | 360/1275 [00:08<00:20, 44.16it/s]#033[A#015Epoch 1:  92%|█████████▏| 10260/11174 [09:56<00:53, 17.21it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  31%|███       | 390/1275 [00:09<00:19, 44.30it/s]#033[A#015Epoch 1:  92%|█████████▏| 10290/11174 [09:56<00:51, 17.24it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  33%|███▎      | 420/1275 [00:09<00:19, 44.46it/s]#033[A#015Epoch 1:  92%|█████████▏| 10320/11174 [09:57<00:49, 17.27it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  35%|███▌      | 450/1275 [00:10<00:18, 44.53it/s]#033[A#015Epoch 1:  93%|█████████▎| 10350/11174 [09:58<00:47, 17.30it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  38%|███▊      | 480/1275 [00:11<00:17, 44.50it/s]#033[A#015Epoch 1:  93%|█████████▎| 10380/11174 [09:58<00:45, 17.33it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  40%|████      | 510/1275 [00:11<00:17, 43.46it/s]#033[A#015Epoch 1:  93%|█████████▎| 10410/11174 [09:59<00:44, 17.36it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  42%|████▏     | 540/1275 [00:12<00:16, 43.72it/s]#033[A#015Epoch 1:  93%|█████████▎| 10440/11174 [10:00<00:42, 17.39it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  45%|████▍     | 570/1275 [00:13<00:16, 43.62it/s]#033[A#015Epoch 1:  94%|█████████▎| 10470/11174 [10:00<00:40, 17.42it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  47%|████▋     | 600/1275 [00:13<00:15, 43.92it/s]#033[A#015Epoch 1:  94%|█████████▍| 10500/11174 [10:01<00:38, 17.45it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  49%|████▉     | 630/1275 [00:14<00:14, 44.10it/s]#033[A#015Epoch 1:  94%|█████████▍| 10530/11174 [10:02<00:36, 17.48it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  52%|█████▏    | 660/1275 [00:15<00:13, 43.93it/s]#033[A#015Epoch 1:  95%|█████████▍| 10560/11174 [10:03<00:35, 17.51it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  54%|█████▍    | 690/1275 [00:15<00:13, 43.89it/s]#033[A#015Epoch 1:  95%|█████████▍| 10590/11174 [10:03<00:33, 17.54it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  56%|█████▋    | 720/1275 [00:16<00:12, 43.84it/s]#033[A#015Epoch 1:  95%|█████████▌| 10620/11174 [10:04<00:31, 17.57it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  59%|█████▉    | 750/1275 [00:17<00:12, 42.70it/s]#033[A#015Epoch 1:  95%|█████████▌| 10650/11174 [10:05<00:29, 17.60it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  61%|██████    | 780/1275 [00:18<00:11, 42.78it/s]#033[A#015Epoch 1:  96%|█████████▌| 10680/11174 [10:05<00:28, 17.63it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  64%|██████▎   | 810/1275 [00:18<00:10, 42.96it/s]#033[A#015Epoch 1:  96%|█████████▌| 10710/11174 [10:06<00:26, 17.66it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  66%|██████▌   | 840/1275 [00:19<00:10, 42.94it/s]#033[A#015Epoch 1:  96%|█████████▌| 10740/11174 [10:07<00:24, 17.69it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  68%|██████▊   | 870/1275 [00:20<00:09, 42.99it/s]#033[A#015Epoch 1:  96%|█████████▋| 10770/11174 [10:07<00:22, 17.72it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  71%|███████   | 900/1275 [00:20<00:08, 43.29it/s]#033[A#015Epoch 1:  97%|█████████▋| 10800/11174 [10:08<00:21, 17.75it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  73%|███████▎  | 930/1275 [00:21<00:07, 43.63it/s]#033[A#015Epoch 1:  97%|█████████▋| 10830/11174 [10:09<00:19, 17.78it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  75%|███████▌  | 960/1275 [00:22<00:07, 43.73it/s]#033[A#015Epoch 1:  97%|█████████▋| 10860/11174 [10:09<00:17, 17.80it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  78%|███████▊  | 990/1275 [00:22<00:06, 43.91it/s]#033[A#015Epoch 1:  97%|█████████▋| 10890/11174 [10:10<00:15, 17.83it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  80%|████████  | 1020/1275 [00:23<00:05, 43.23it/s]#033[A#015Epoch 1:  98%|█████████▊| 10920/11174 [10:11<00:14, 17.86it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  82%|████████▏ | 1050/1275 [00:24<00:05, 43.42it/s]#033[A#015Epoch 1:  98%|█████████▊| 10950/11174 [10:12<00:12, 17.89it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  85%|████████▍ | 1080/1275 [00:24<00:04, 43.26it/s]#033[A#015Epoch 1:  98%|█████████▊| 10980/11174 [10:12<00:10, 17.92it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  87%|████████▋ | 1110/1275 [00:25<00:03, 43.70it/s]#033[A#015Epoch 1:  99%|█████████▊| 11010/11174 [10:13<00:09, 17.95it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  89%|████████▉ | 1140/1275 [00:26<00:03, 44.00it/s]#033[A#015Epoch 1:  99%|█████████▉| 11040/11174 [10:14<00:07, 17.98it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  92%|█████████▏| 1170/1275 [00:26<00:02, 43.41it/s]#033[A#015Epoch 1:  99%|█████████▉| 11070/11174 [10:14<00:05, 18.01it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  94%|█████████▍| 1200/1275 [00:27<00:01, 43.63it/s]#033[A#015Epoch 1:  99%|█████████▉| 11100/11174 [10:15<00:04, 18.04it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  96%|█████████▋| 1230/1275 [00:28<00:01, 43.73it/s]#033[A#015Epoch 1: 100%|█████████▉| 11130/11174 [10:16<00:02, 18.06it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating:  99%|█████████▉| 1260/1275 [00:29<00:00, 43.92it/s]#033[A#015Epoch 1: 100%|█████████▉| 11160/11174 [10:16<00:00, 18.09it/s, loss=1.42, v_num=0, val_f1_epoch=0.635, val_loss_epoch=0.147, train_f1=0.645, val_f1_step=0.496, val_loss_step=0.0629]
#015Validating: 100%|██████████| 1275/1275 [00:29<00:00, 44.18it/s]#033[A#015Epoch 1: 100%|██████████| 11174/11174 [10:17<00:00, 18.10it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]

#015                                                               #033[A#015Epoch 1:   0%|          | 0/11174 [00:00<?, ?it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]            #015Epoch 2:   0%|          | 0/11174 [00:00<?, ?it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   0%|          | 30/11174 [00:02<13:06, 14.18it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   0%|          | 30/11174 [00:02<13:06, 14.17it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 60/11174 [00:03<12:06, 15.30it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 60/11174 [00:03<12:06, 15.29it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 90/11174 [00:05<11:35, 15.94it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 90/11174 [00:05<11:35, 15.94it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 120/11174 [00:07<11:24, 16.14it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|          | 120/11174 [00:07<11:24, 16.14it/s, loss=1.69, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|▏         | 150/11174 [00:09<11:12, 16.39it/s, loss=1.69, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   1%|▏         | 150/11174 [00:09<11:12, 16.39it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 180/11174 [00:10<11:09, 16.42it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 180/11174 [00:10<11:09, 16.42it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.545, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 210/11174 [00:12<11:06, 16.44it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.545, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 210/11174 [00:12<11:06, 16.44it/s, loss=1.48, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 240/11174 [00:14<11:00, 16.56it/s, loss=1.48, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 240/11174 [00:14<11:00, 16.56it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 270/11174 [00:16<10:58, 16.56it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   2%|▏         | 270/11174 [00:16<10:58, 16.56it/s, loss=1.44, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 300/11174 [00:17<10:52, 16.67it/s, loss=1.44, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 300/11174 [00:17<10:52, 16.67it/s, loss=1.25, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 330/11174 [00:19<10:50, 16.66it/s, loss=1.25, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 330/11174 [00:19<10:50, 16.66it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 360/11174 [00:21<10:50, 16.62it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 360/11174 [00:21<10:50, 16.62it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 390/11174 [00:23<10:46, 16.69it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   3%|▎         | 390/11174 [00:23<10:46, 16.69it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 420/11174 [00:25<10:47, 16.61it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 420/11174 [00:25<10:47, 16.61it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.597, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 450/11174 [00:26<10:43, 16.67it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.597, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 450/11174 [00:26<10:43, 16.67it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 480/11174 [00:28<10:41, 16.68it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   4%|▍         | 480/11174 [00:28<10:41, 16.68it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▍         | 510/11174 [00:30<10:39, 16.67it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▍         | 510/11174 [00:30<10:39, 16.67it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▍         | 540/11174 [00:32<10:35, 16.73it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▍         | 540/11174 [00:32<10:35, 16.73it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.733, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▌         | 570/11174 [00:34<10:33, 16.74it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.733, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▌         | 570/11174 [00:34<10:33, 16.74it/s, loss=1.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.696, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▌         | 600/11174 [00:35<10:30, 16.77it/s, loss=1.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.696, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   5%|▌         | 600/11174 [00:35<10:30, 16.77it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.735, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 630/11174 [00:37<10:28, 16.77it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.735, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 630/11174 [00:37<10:28, 16.77it/s, loss=1.93, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.604, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 660/11174 [00:39<10:27, 16.75it/s, loss=1.93, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.604, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 660/11174 [00:39<10:27, 16.75it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.637, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 690/11174 [00:41<10:24, 16.78it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.637, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▌         | 690/11174 [00:41<10:24, 16.78it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▋         | 720/11174 [00:42<10:23, 16.77it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   6%|▋         | 720/11174 [00:42<10:23, 16.77it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 750/11174 [00:44<10:19, 16.81it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 750/11174 [00:44<10:19, 16.81it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 780/11174 [00:46<10:18, 16.81it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 780/11174 [00:46<10:18, 16.81it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 810/11174 [00:48<10:16, 16.81it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   7%|▋         | 810/11174 [00:48<10:16, 16.81it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:   8%|▊         | 840/11174 [00:49<10:13, 16.85it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 840/11174 [00:49<10:13, 16.85it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 870/11174 [00:51<10:12, 16.83it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 870/11174 [00:51<10:12, 16.83it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 900/11174 [00:53<10:09, 16.86it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 900/11174 [00:53<10:09, 16.86it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.643, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 930/11174 [00:55<10:07, 16.85it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.643, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   8%|▊         | 930/11174 [00:55<10:07, 16.85it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.614, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▊         | 960/11174 [00:57<10:06, 16.84it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.614, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▊         | 960/11174 [00:57<10:06, 16.84it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.603, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▉         | 990/11174 [00:58<10:04, 16.86it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.603, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▉         | 990/11174 [00:58<10:04, 16.86it/s, loss=1.56, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▉         | 1020/11174 [01:00<10:02, 16.85it/s, loss=1.56, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▉         | 1020/11174 [01:00<10:02, 16.85it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:   9%|▉         | 1050/11174 [01:02<10:00, 16.87it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:   9%|▉         | 1050/11174 [01:02<10:00, 16.87it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|▉         | 1080/11174 [01:04<09:58, 16.87it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|▉         | 1080/11174 [01:04<09:58, 16.87it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|▉         | 1110/11174 [01:05<09:56, 16.87it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|▉         | 1110/11174 [01:05<09:56, 16.87it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|█         | 1140/11174 [01:07<09:54, 16.89it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|█         | 1140/11174 [01:07<09:54, 16.89it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  10%|█         | 1170/11174 [01:09<09:52, 16.89it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  10%|█         | 1170/11174 [01:09<09:52, 16.89it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█         | 1200/11174 [01:10<09:49, 16.91it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█         | 1200/11174 [01:10<09:49, 16.91it/s, loss=1.94, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█         | 1230/11174 [01:12<09:48, 16.90it/s, loss=1.94, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█         | 1230/11174 [01:12<09:48, 16.90it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█▏        | 1260/11174 [01:14<09:46, 16.90it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  11%|█▏        | 1260/11174 [01:14<09:46, 16.90it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1290/11174 [01:16<09:44, 16.91it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1290/11174 [01:16<09:44, 16.91it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.683, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1320/11174 [01:18<09:43, 16.90it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.683, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1320/11174 [01:18<09:43, 16.90it/s, loss=1.13, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140

, train_f1=0.675, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1350/11174 [01:19<09:40, 16.92it/s, loss=1.13, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.675, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1350/11174 [01:19<09:40, 16.92it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1380/11174 [01:21<09:39, 16.91it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  12%|█▏        | 1380/11174 [01:21<09:39, 16.91it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1410/11174 [01:23<09:37, 16.89it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1410/11174 [01:23<09:37, 16.89it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.646, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  13%|█▎        | 1440/11174 [01:25<09:36, 16.88it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.646, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1440/11174 [01:25<09:36, 16.88it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1470/11174 [01:27<09:35, 16.88it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1470/11174 [01:27<09:35, 16.88it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.685, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1500/11174 [01:28<09:32, 16.89it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.685, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  13%|█▎        | 1500/11174 [01:28<09:32, 16.89it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▎        | 1530/11174 [01:30<09:31, 16.88it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▎        | 1530/11174 [01:30<09:31, 16.88it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.584, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1560/11174 [01:32<09:29, 16.88it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.584, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1560/11174 [01:32<09:29, 16.88it/s, loss=2.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.558, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1590/11174 [01:34<09:27, 16.89it/s, loss=2.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.558, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1590/11174 [01:34<09:27, 16.89it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1620/11174 [01:35<09:25, 16.89it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  14%|█▍        | 1620/11174 [01:35<09:25, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▍        | 1650/11174 [01:37<09:23, 16.91it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▍        | 1650/11174 [01:37<09:23, 16.91it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.614, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▌        | 1680/11174 [01:39<09:21, 16.90it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.614, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▌        | 1680/11174 [01:39<09:21, 16.90it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▌        | 1710/11174 [01:41<09:20, 16.90it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  15%|█▌        | 1710/11174 [01:41<09:20, 16.90it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1740/11174 [01:42<09:18, 16.90it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1740/11174 [01:42<09:18, 16.90it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.645, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1770/11174 [01:44<09:16, 16.90it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.645, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1770/11174 [01:44<09:16, 16.90it/s, loss=1.25, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1800/11174 [01:46<09:14, 16.91it/s, loss=1.25, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▌        | 1800/11174 [01:46<09:14, 16.91it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▋        | 1830/11174 [01:48<09:12, 16.90it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  16%|█▋        | 1830/11174 [01:48<09:12, 16.90it/s, loss=1.48, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.667, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1860/11174 [01:50<09:11, 16.90it/s, loss=1.48, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.667, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1860/11174 [01:50<09:11, 16.90it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.553, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  17%|█▋        | 1890/11174 [01:51<09:09, 16.91it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.553, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1890/11174 [01:51<09:09, 16.91it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1920/11174 [01:53<09:07, 16.91it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1920/11174 [01:53<09:07, 16.91it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1950/11174 [01:55<09:05, 16.92it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  17%|█▋        | 1950/11174 [01:55<09:05, 16.92it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.700, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 1980/11174 [01:57<09:03, 16.92it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.700, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 1980/11174 [01:57<09:03, 16.91it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.645, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 2010/11174 [01:58<09:01, 16.91it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.645, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 2010/11174 [01:58<09:01, 16.91it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.625, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 2040/11174 [02:00<08:59, 16.92it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.625, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  18%|█▊        | 2040/11174 [02:00<08:59, 16.92it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.717, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▊        | 2070/11174 [02:02<08:58, 16.92it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.717, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▊        | 2070/11174 [02:02<08:58, 16.92it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▉        | 2100/11174 [02:04<08:55, 16.93it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▉        | 2100/11174 [02:04<08:55, 16.93it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▉        | 2130/11174 [02:05<08:54, 16.92it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▉        | 2130/11174 [02:05<08:54, 16.92it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  19%|█▉        | 2160/11174 [02:07<08:52, 16.92it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  19%|█▉        | 2160/11174 [02:07<08:52, 16.92it/s, loss=1.76, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|█▉        | 2190/11174 [02:09<08:50, 16.93it/s, loss=1.76, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|█▉        | 2190/11174 [02:09<08:50, 16.93it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.640, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  20%|█▉        | 2220/11174 [02:11<08:49, 16.92it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.640, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|█▉        | 2220/11174 [02:11<08:49, 16.92it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.647, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|██        | 2250/11174 [02:12<08:47, 16.93it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.647, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|██        | 2250/11174 [02:12<08:47, 16.93it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|██        | 2280/11174 [02:14<08:45, 16.92it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  20%|██        | 2280/11174 [02:14<08:45, 16.92it/s, loss=1.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.689, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2310/11174 [02:16<08:43, 16.92it/s, loss=1.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.689, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2310/11174 [02:16<08:43, 16.92it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2340/11174 [02:18<08:41, 16.93it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2340/11174 [02:18<08:41, 16.93it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.602, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2370/11174 [02:20<08:40, 16.93it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.602, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██        | 2370/11174 [02:20<08:40, 16.93it/s, loss=1.06, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██▏       | 2400/11174 [02:21<08:38, 16.93it/s, loss=1.06, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  21%|██▏       | 2400/11174 [02:21<08:38, 16.93it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  22%|██▏       | 2430/11174 [02:23<08:36, 16.92it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  22%|██▏       | 2430/11174 [02:23<08:36, 16.92it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  22%|██▏       | 2460/11174 [02:25<08:35, 16.90it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  22%|██▏       | 2460/11174 [02:25<08:35, 16.90it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  22%|██▏       | 2490/11174 [02:27<08:33, 16.91it/s, loss=1.2, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  22%|██▏       | 2490/11174 [02:27<08:33, 16.91it/s, loss=1.29, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2520/11174 [02:29<08:32, 16.90it/s, loss=1.29, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2520/11174 [02:29<08:32, 16.90it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2550/11174 [02:30<08:30, 16.90it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2550/11174 [02:30<08:30, 16.90it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.589, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2580/11174 [02:32<08:28, 16.89it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.589, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2580/11174 [02:32<08:28, 16.89it/s, loss=1.86, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2610/11174 [02:34<08:27, 16.89it/s, loss=1.86, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  23%|██▎       | 2610/11174 [02:34<08:27, 16.89it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▎       | 2640/11174 [02:36<08:24, 16.90it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▎       | 2640/11174 [02:36<08:24, 16.90it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  24%|██

▍       | 2670/11174 [02:38<08:23, 16.90it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▍       | 2670/11174 [02:38<08:23, 16.90it/s, loss=1.94, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▍       | 2700/11174 [02:39<08:21, 16.91it/s, loss=1.94, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▍       | 2700/11174 [02:39<08:21, 16.91it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.650, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▍       | 2730/11174 [02:41<08:19, 16.90it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.650, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  24%|██▍       | 2730/11174 [02:41<08:19, 16.90it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▍       | 2760/11174 [02:43<08:18, 16.90it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▍       | 2760/11174 [02:43<08:18, 16.90it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▍       | 2790/11174 [02:45<08:16, 16.90it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▍       | 2790/11174 [02:45<08:16, 16.90it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▌       | 2820/11174 [02:46<08:14, 16.89it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  25%|██▌       | 2820/11174 [02:46<08:14, 16.89it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.700, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2850/11174 [02:48<08:12, 16.89it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.700, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2850/11174 [02:48<08:12, 16.89it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.636, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2880/11174 [02:50<08:11, 16.89it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.636, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2880/11174 [02:50<08:11, 16.89it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.667, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2910/11174 [02:52<08:09, 16.88it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.667, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▌       | 2910/11174 [02:52<08:09, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▋       | 2940/11174 [02:54<08:07, 16.89it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  26%|██▋       | 2940/11174 [02:54<08:07, 16.89it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 2970/11174 [02:55<08:05, 16.89it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 2970/11174 [02:55<08:05, 16.89it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.639, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3000/11174 [02:57<08:03, 16.90it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.639, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3000/11174 [02:57<08:03, 16.90it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3030/11174 [02:59<08:02, 16.89it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3030/11174 [02:59<08:02, 16.89it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3060/11174 [03:01<08:00, 16.89it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  27%|██▋       | 3060/11174 [03:01<08:00, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3090/11174 [03:02<07:58, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3090/11174 [03:02<07:58, 16.89it/s, loss=1.73, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.642, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3120/11174 [03:04<07:56, 16.89it/s, loss=1.73, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.642, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3120/11174 [03:04<07:56, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3150/11174 [03:06<07:54, 16.90it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3150/11174 [03:06<07:54, 16.90it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3180/11174 [03:08<07:53, 16.89it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  28%|██▊       | 3180/11174 [03:08<07:53, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▊       | 3210/11174 [03:10<07:51, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▊       | 3210/11174 [03:10<07:51, 16.89it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▉       | 3240/11174 [03:11<07:49, 16.90it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.653, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▉       | 3240/11174 [03:11<07:49, 16.90it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.620, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▉       | 3270/11174 [03:13<07:47, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.620, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  29%|██▉       | 3270/11174 [03:13<07:47, 16.89it/s, loss=1.69, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|██▉       | 3300/11174 [03:15<07:45, 16.90it/s, loss=1.69, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|██▉       | 3300/11174 [03:15<07:45, 16.90it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.705, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|██▉       | 3330/11174 [03:17<07:44, 16.90it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.705, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|██▉       | 3330/11174 [03:17<07:44, 16.90it/s, loss=1.21, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|███       | 3360/11174 [03:18<07:42, 16.90it/s, loss=1.21, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|███       | 3360/11174 [03:18<07:42, 16.90it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.639, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|███       | 3390/11174 [03:20<07:40, 16.90it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.639, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  30%|███       | 3390/11174 [03:20<07:40, 16.90it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.593, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3420/11174 [03:22<07:38, 16.90it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.593, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3420/11174 [03:22<07:38, 16.90it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.712, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3450/11174 [03:24<07:36, 16.90it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.712, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3450/11174 [03:24<07:36, 16.90it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3480/11174 [03:26<07:35, 16.89it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.655, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███       | 3480/11174 [03:26<07:35, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.637, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███▏      | 3510/11174 [03:27<07:33, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.637, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  31%|███▏      | 3510/11174 [03:27<07:33, 16.89it/s, loss=1.81, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.647, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3540/11174 [03:29<07:31, 16.90it/s, loss=1.81, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.647, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3540/11174 [03:29<07:31, 16.90it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.600, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3570/11174 [03:31<07:30, 16.89it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.600, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3570/11174 [03:31<07:30, 16.89it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.704, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  32%|███▏      | 3600/11174 [03:33<07:28, 16.90it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.704, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3600/11174 [03:33<07:28, 16.90it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.629, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3630/11174 [03:34<07:26, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.629, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  32%|███▏      | 3630/11174 [03:34<07:26, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.613, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3660/11174 [03:36<07:24, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.613, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3660/11174 [03:36<07:24, 16.89it/s, loss=2.04, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3690/11174 [03:38<07:22, 16.89it/s, loss=2.04, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3690/11174 [03:38<07:22, 16.89it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3720/11174 [03:40<07:21, 16.89it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  33%|███▎      | 3720/11174 [03:40<07:21, 16.89it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▎      | 3750/11174 [03:41<07:19, 16.90it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▎      | 3750/11174 [03:41<07:19, 16.90it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.656, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▍      | 3780/11174 [03:43<07:17, 16.89it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.656, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▍      | 3780/11174 [03:43<07:17, 16.89it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  34%|███▍      | 3810/11174 [03:45<07:15, 16.89it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▍      | 3810/11174 [03:45<07:15, 16.89it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▍      | 3840/11174 [03:47<07:14, 16.90it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  34%|███▍      | 3840/11174 [03:47<07:14, 16.90it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▍      | 3870/11174 [03:49<07:12, 16.89it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▍      | 3870/11174 [03:49<07:12, 16.89it/s, loss=1.44, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▍      | 3900/11174 [03:50<07:10, 16.90it/s, loss=1.44, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▍      | 3900/11174 [03:50<07:10, 16.90it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.704, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  35%|███▌      | 3930/11174 [03:52<07:08, 16.89it/s, loss=1.7, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.704, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▌      | 3930/11174 [03:52<07:08, 16.89it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.710, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▌      | 3960/11174 [03:54<07:07, 16.89it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.710, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  35%|███▌      | 3960/11174 [03:54<07:07, 16.89it/s, loss=1.

16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 3990/11174 [03:56<07:05, 16.90it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 3990/11174 [03:56<07:05, 16.90it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 4020/11174 [03:57<07:03, 16.89it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 4020/11174 [03:57<07:03, 16.89it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 4050/11174 [03:59<07:01, 16.90it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  36%|███▌      | 4050/11174 [03:59<07:01, 16.90it/s, loss=1.91, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4080/11174 [04:01<06:59, 16.90it/s, loss=1.91, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4080/11174 [04:01<06:59, 16.90it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4110/11174 [04:03<06:58, 16.89it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4110/11174 [04:03<06:58, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4140/11174 [04:05<06:56, 16.90it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4140/11174 [04:05<06:56, 16.90it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4170/11174 [04:06<06:54, 16.90it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  37%|███▋      | 4170/11174 [04:06<06:54, 16.90it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.770, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4200/11174 [04:08<06:52, 16.90it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.770, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4200/11174 [04:08<06:52, 16.90it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  38%|███▊      | 4230/11174 [04:10<06:50, 16.90it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4230/11174 [04:10<06:50, 16.90it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4260/11174 [04:12<06:49, 16.90it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.610, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4260/11174 [04:12<06:49, 16.90it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.698, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  38%|███▊      | 4290/11174 [04:13<06:47, 16.91it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.698, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  38%|███▊      | 4290/11174 [04:13<06:47, 16.91it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.626, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▊      | 4320/11174 [04:15<06:45, 16.90it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.626, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▊      | 4320/11174 [04:15<06:45, 16.90it/s, loss=1.73, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4350/11174 [04:17<06:43, 16.91it/s, loss=1.73, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4350/11174 [04:17<06:43, 16.91it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.620, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4380/11174 [04:19<06:41, 16.90it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.620, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4380/11174 [04:19<06:41, 16.90it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4410/11174 [04:20<06:40, 16.90it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  39%|███▉      | 4410/11174 [04:20<06:40, 16.90it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|███▉      | 4440/11174 [04:22<06:38, 16.90it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|███▉      | 4440/11174 [04:22<06:38, 16.90it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|████      | 4470/11174 [04:24<06:37, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|████      | 4470/11174 [04:24<06:37, 16.88it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|████      | 4500/11174 [04:26<06:35, 16.89it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  40%|████      | 4500/11174 [04:26<06:35, 16.89it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4530/11174 [04:28<06:33, 16.89it/s, loss=1.46, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4530/11174 [04:28<06:33, 16.89it/s, loss=2.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4560/11174 [04:30<06:31, 16.89it/s, loss=2.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.606, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4560/11174 [04:30<06:31, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4590/11174 [04:31<06:29, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████      | 4590/11174 [04:31<06:29, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████▏     | 4620/11174 [04:33<06:28, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  41%|████▏     | 4620/11174 [04:33<06:28, 16.89it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.683, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4650/11174 [04:35<06:26, 16.89it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.683, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4650/11174 [04:35<06:26, 16.89it/s, loss=1.84, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4680/11174 [04:37<06:24, 16.89it/s, loss=1.84, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4680/11174 [04:37<06:24, 16.89it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  42%|████▏     | 4710/11174 [04:38<06:22, 16.89it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4710/11174 [04:38<06:22, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4740/11174 [04:40<06:20, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  42%|████▏     | 4740/11174 [04:40<06:20, 16.89it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4770/11174 [04:42<06:19, 16.88it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4770/11174 [04:42<06:19, 16.88it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.594, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4800/11174 [04:44<06:17, 16.89it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.594, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4800/11174 [04:44<06:17, 16.89it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4830/11174 [04:46<06:15, 16.89it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4830/11174 [04:46<06:15, 16.89it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4860/11174 [04:47<06:14, 16.88it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  43%|████▎     | 4860/11174 [04:47<06:14, 16.88it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.613, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4890/11174 [04:49<06:12, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.613, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4890/11174 [04:49<06:12, 16.89it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4920/11174 [04:51<06:10, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4920/11174 [04:51<06:10, 16.88it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4950/11174 [04:53<06:08, 16.89it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  44%|████▍     | 4950/11174 [04:53<06:08, 16.89it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  45%|████▍     | 4980/11174 [04:54<06:06, 16.89it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▍     | 4980/11174 [04:54<06:06, 16.89it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.668, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▍     | 5010/11174 [04:56<06:05, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.668, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▍     | 5010/11174 [04:56<06:05, 16.88it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.654, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  45%|████▌     | 5040/11174 [04:58<06:03, 16.89it/s, loss=1.8, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.654, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▌     | 5040/11174 [04:58<06:03, 16.89it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▌     | 5070/11174 [05:00<06:01, 16.89it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  45%|████▌     | 5070/11174 [05:00<06:01, 16.89it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.682, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5100/11174 [05:01<05:59, 16.89it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.682, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5100/11174 [05:01<05:59, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5130/11174 [05:03<05:57, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5130/11174 [05:03<05:57, 16.88it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5160/11174 [05:05<05:56, 16.88it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▌     | 5160/11174 [05:05<05:56, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.641, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▋     | 5190/11174 [05:07<05:54, 16.89it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.641, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  46%|████▋     | 5190/11174 [05:07<05:54, 16.89it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.754, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5220/11174 [05:09<05:52, 16.88it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.754, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5220/11174 [05:09<05:52, 16.88it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5250/11174 [05:10<05:50, 16.89it/s, loss=1.24, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.661, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5250/11174 [05:10<05:50, 16.89it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch

=0.140, train_f1=0.675, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5280/11174 [05:12<05:49, 16.89it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.675, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  47%|████▋     | 5280/11174 [05:12<05:49, 16.89it/s, loss=2.04, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5310/11174 [05:14<05:47, 16.88it/s, loss=2.04, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5310/11174 [05:14<05:47, 16.88it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.738, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  48%|████▊     | 5340/11174 [05:16<05:45, 16.89it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.738, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5340/11174 [05:16<05:45, 16.89it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.691, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5370/11174 [05:18<05:43, 16.89it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.691, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5370/11174 [05:18<05:43, 16.89it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.693, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  48%|████▊     | 5400/11174 [05:19<05:41, 16.89it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.693, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  48%|████▊     | 5400/11174 [05:19<05:41, 16.89it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.695, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▊     | 5430/11174 [05:21<05:40, 16.88it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.695, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▊     | 5430/11174 [05:21<05:40, 16.88it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5460/11174 [05:23<05:38, 16.88it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.631, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5460/11174 [05:23<05:38, 16.88it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5490/11174 [05:25<05:36, 16.88it/s, loss=1.89, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.662, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5490/11174 [05:25<05:36, 16.88it/s, loss=1.14, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.692, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5520/11174 [05:27<05:35, 16.87it/s, loss=1.14, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.692, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  49%|████▉     | 5520/11174 [05:27<05:35, 16.87it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|████▉     | 5550/11174 [05:28<05:33, 16.87it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.678, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|████▉     | 5550/11174 [05:28<05:33, 16.87it/s, loss=1.79, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|████▉     | 5580/11174 [05:30<05:31, 16.87it/s, loss=1.79, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|████▉     | 5580/11174 [05:30<05:31, 16.87it/s, loss=1.15, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|█████     | 5610/11174 [05:32<05:29, 16.87it/s, loss=1.15, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|█████     | 5610/11174 [05:32<05:29, 16.87it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.656, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|█████     | 5640/11174 [05:34<05:27, 16.88it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.656, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  50%|█████     | 5640/11174 [05:34<05:27, 16.88it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████     | 5670/11174 [05:36<05:26, 16.87it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████     | 5670/11174 [05:36<05:26, 16.87it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.705, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████     | 5700/11174 [05:37<05:24, 16.88it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.705, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████     | 5700/11174 [05:37<05:24, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████▏    | 5730/11174 [05:39<05:22, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.632, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  51%|█████▏    | 5730/11174 [05:39<05:22, 16.88it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5760/11174 [05:41<05:20, 16.87it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5760/11174 [05:41<05:20, 16.87it/s, loss=1.29, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5790/11174 [05:43<05:18, 16.88it/s, loss=1.29, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5790/11174 [05:43<05:18, 16.88it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.640, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5820/11174 [05:44<05:17, 16.88it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.640, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5820/11174 [05:44<05:17, 16.88it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5850/11174 [05:46<05:15, 16.88it/s, loss=1.43, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  52%|█████▏    | 5850/11174 [05:46<05:15, 16.88it/s, loss=1.42, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5880/11174 [05:48<05:13, 16.88it/s, loss=1.42, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5880/11174 [05:48<05:13, 16.88it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.595, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5910/11174 [05:50<05:11, 16.87it/s, loss=1.63, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.595, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5910/11174 [05:50<05:11, 16.87it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5940/11174 [05:51<05:10, 16.88it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5940/11174 [05:51<05:10, 16.88it/s, loss=1.21, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5970/11174 [05:53<05:08, 16.88it/s, loss=1.21, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  53%|█████▎    | 5970/11174 [05:53<05:08, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  54%|█████▎    | 6000/11174 [05:55<05:06, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  54%|█████▎    | 6000/11174 [05:55<05:06, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.689, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  54%|█████▍    | 6030/11174 [05:57<05:04, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.689, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  54%|█████▍    | 6030/11174 [05:57<05:04, 16.88it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  54%|█████▍    | 6060/11174 [05:59<05:02, 16.88it/s, loss=1.6, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  54%|█████▍    | 6060/11174 [05:59<05:02, 16.88it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▍    | 6090/11174 [06:00<05:01, 16.88it/s, loss=1.71, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▍    | 6090/11174 [06:00<05:01, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▍    | 6120/11174 [06:02<04:59, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▍    | 6120/11174 [06:02<04:59, 16.88it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▌    | 6150/11174 [06:04<04:57, 16.89it/s, loss=1.38, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▌    | 6150/11174 [06:04<04:57, 16.89it/s, loss=1.13, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.617, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▌    | 6180/11174 [06:05<04:55, 16.89it/s, loss=1.13, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.617, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  55%|█████▌    | 6180/11174 [06:05<04:55, 16.89it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.694, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  56%|█████▌    | 6210/11174 [06:07<04:54, 16.88it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.694, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▌    | 6210/11174 [06:07<04:54, 16.88it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▌    | 6240/11174 [06:09<04:52, 16.89it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▌    | 6240/11174 [06:09<04:52, 16.89it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▌    | 6270/11174 [06:11<04:50, 16.89it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▌    | 6270/11174 [06:11<04:50, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▋    | 6300/11174 [06:13<04:48, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  56%|█████▋    | 6300/11174 [06:13<04:48, 16.89it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6330/11174 [06:14<04:46, 16.88it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.634, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6330/11174 [06:14<04:46, 16.88it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.668, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6360/11174 [06:16<04:45, 16.88it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.668, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6360/11174 [06:16<04:45, 16.88it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.711, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6390/11174 [06:18<04:43, 16.89it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.711, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6390/11174 [06:18<04:43, 16.89it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.643, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6420/11174 [06:20<04:41, 16.88it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.643, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  57%|█████▋    | 6420/11174 [06:20<04:41, 16.88it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.744, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6450/11174 [06:22<04:39, 16.88it/s, loss=1.61, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.744, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6450/11174 [06:22<04:39, 16.88it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.671, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6480/11174 [06:23<04:38, 16.88it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.671, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6480/11174 [06:23<04:38, 16.88it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6510/11174 [06:25<04:36, 16.87it/s, loss=1.68, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.649, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  58%|█████▊    | 6510/11174 [06:25<04:36, 16.87it/s, loss=1.42, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▊    | 6540/11174 [06:27<04:34, 16.87it/s, loss=1.42, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, 

val_loss_step=0.0676]#015Epoch 2:  59%|█████▊    | 6540/11174 [06:27<04:34, 16.87it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6570/11174 [06:29<04:32, 16.87it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6570/11174 [06:29<04:32, 16.87it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6600/11174 [06:31<04:31, 16.88it/s, loss=1.87, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6600/11174 [06:31<04:31, 16.88it/s, loss=1.76, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6630/11174 [06:32<04:29, 16.88it/s, loss=1.76, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  59%|█████▉    | 6630/11174 [06:32<04:29, 16.88it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|█████▉    | 6660/11174 [06:34<04:27, 16.87it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|█████▉    | 6660/11174 [06:34<04:27, 16.87it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.592, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|█████▉    | 6690/11174 [06:36<04:25, 16.88it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.592, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|█████▉    | 6690/11174 [06:36<04:25, 16.88it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|██████    | 6720/11174 [06:38<04:23, 16.88it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|██████    | 6720/11174 [06:38<04:23, 16.88it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.595, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|██████    | 6750/11174 [06:39<04:22, 16.88it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.595, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  60%|██████    | 6750/11174 [06:39<04:22, 16.88it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6780/11174 [06:41<04:20, 16.88it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6780/11174 [06:41<04:20, 16.88it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6810/11174 [06:43<04:18, 16.88it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6810/11174 [06:43<04:18, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6840/11174 [06:45<04:16, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████    | 6840/11174 [06:45<04:16, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.721, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████▏   | 6870/11174 [06:47<04:15, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.721, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  61%|██████▏   | 6870/11174 [06:47<04:15, 16.88it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6900/11174 [06:48<04:13, 16.88it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6900/11174 [06:48<04:13, 16.88it/s, loss=1.15, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6930/11174 [06:50<04:11, 16.88it/s, loss=1.15, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6930/11174 [06:50<04:11, 16.88it/s, loss=1.11, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.673, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6960/11174 [06:52<04:09, 16.88it/s, loss=1.11, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.673, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  62%|██████▏   | 6960/11174 [06:52<04:09, 16.88it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 6990/11174 [06:54<04:07, 16.88it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 6990/11174 [06:54<04:07, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.697, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7020/11174 [06:55<04:06, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.697, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7020/11174 [06:55<04:06, 16.88it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7050/11174 [06:57<04:04, 16.88it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.658, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7050/11174 [06:57<04:04, 16.88it/s, loss=1.74, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7080/11174 [06:59<04:02, 16.88it/s, loss=1.74, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.666, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  63%|██████▎   | 7080/11174 [06:59<04:02, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▎   | 7110/11174 [07:01<04:00, 16.88it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.676, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▎   | 7110/11174 [07:01<04:00, 16.88it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.642, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7140/11174 [07:02<03:58, 16.89it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.642, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7140/11174 [07:02<03:58, 16.89it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7170/11174 [07:04<03:57, 16.89it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7170/11174 [07:04<03:57, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7200/11174 [07:06<03:55, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  64%|██████▍   | 7200/11174 [07:06<03:55, 16.89it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▍   | 7230/11174 [07:08<03:53, 16.89it/s, loss=1.59, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▍   | 7230/11174 [07:08<03:53, 16.89it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.674, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▍   | 7260/11174 [07:09<03:51, 16.89it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.674, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▍   | 7260/11174 [07:09<03:51, 16.89it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.593, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▌   | 7290/11174 [07:11<03:49, 16.89it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.593, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  65%|██████▌   | 7290/11174 [07:11<03:49, 16.89it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7320/11174 [07:13<03:48, 16.88it/s, loss=1.52, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.687, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7320/11174 [07:13<03:48, 16.88it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7350/11174 [07:15<03:46, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.623, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7350/11174 [07:15<03:46, 16.89it/s, loss=1.82, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7380/11174 [07:17<03:44, 16.89it/s, loss=1.82, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▌   | 7380/11174 [07:17<03:44, 16.89it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.693, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▋   | 7410/11174 [07:18<03:42, 16.89it/s, loss=1.85, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.693, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  66%|██████▋   | 7410/11174 [07:18<03:42, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7440/11174 [07:20<03:41, 16.89it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7440/11174 [07:20<03:41, 16.89it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7470/11174 [07:22<03:39, 16.89it/s, loss=1.53, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7470/11174 [07:22<03:39, 16.89it/s, loss=0.975, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7500/11174 [07:24<03:37, 16.88it/s, loss=0.975, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.651, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7500/11174 [07:24<03:37, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  67%|██████▋   | 7530/11174 [07:26<03:35, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  67%|██████▋   | 7530/11174 [07:26<03:35, 16.88it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.711, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7560/11174 [07:27<03:34, 16.88it/s, loss=1.57, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.711, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7560/11174 [07:27<03:34, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7590/11174 [07:29<03:32, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7590/11174 [07:29<03:32, 16.88it/s, loss=1.17, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7620/11174 [07:31<03:30, 16.88it/s, loss=1.17, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.624, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7620/11174 [07:31<03:30, 16.88it/s, loss=1.08, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.682, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7650/11174 [07:33<03:28, 16.88it/s, loss=1.08, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.682, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  68%|██████▊   | 7650/11174 [07:33<03:28, 16.88it/s, loss=1.96, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.592, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▊   | 7680/11174 [07:34<03:26, 16.88it/s, loss=1.96, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.592, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▊   | 7680/11174 [07:34<03:26, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.731, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▉   | 7710/11174 [07:36<03:25, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.731, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▉   | 7710/11174 [07:36<03:25, 16.88it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.611, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▉   | 7740/11174 [07:38<03:23, 16.88it/s, loss=1.31, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.611, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  69%|██████▉   | 7740/11174 [07:38<03:23, 16.88it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  70%|██████▉   | 7770/11174 [07:40<03:21, 16.88it/s, loss=1.4, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|██████▉   | 7770/11174 [07:40<03:21, 16.88it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|██████▉   | 7800/11174 [07:41<03:19, 16.89it/s, loss=1.49, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|██�

�███▉   | 7800/11174 [07:41<03:19, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|███████   | 7830/11174 [07:43<03:18, 16.89it/s, loss=1.58, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|███████   | 7830/11174 [07:43<03:18, 16.89it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|███████   | 7860/11174 [07:45<03:16, 16.89it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.715, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  70%|███████   | 7860/11174 [07:45<03:16, 16.89it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7890/11174 [07:47<03:14, 16.89it/s, loss=1.07, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7890/11174 [07:47<03:14, 16.89it/s, loss=1.01, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7920/11174 [07:48<03:12, 16.89it/s, loss=1.01, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.635, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7920/11174 [07:48<03:12, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7950/11174 [07:50<03:10, 16.89it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.664, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████   | 7950/11174 [07:50<03:10, 16.89it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████▏  | 7980/11174 [07:52<03:09, 16.89it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  71%|███████▏  | 7980/11174 [07:52<03:09, 16.89it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8010/11174 [07:54<03:07, 16.89it/s, loss=1.67, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8010/11174 [07:54<03:07, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.719, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8040/11174 [07:55<03:05, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.719, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8040/11174 [07:55<03:05, 16.89it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8070/11174 [07:57<03:03, 16.89it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8070/11174 [07:57<03:03, 16.89it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.617, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8100/11174 [07:59<03:01, 16.89it/s, loss=1.23, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.617, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  72%|███████▏  | 8100/11174 [07:59<03:01, 16.89it/s, loss=0.98, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8130/11174 [08:01<03:00, 16.89it/s, loss=0.98, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8130/11174 [08:01<03:00, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8160/11174 [08:03<02:58, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.663, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8160/11174 [08:03<02:58, 16.89it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8190/11174 [08:04<02:56, 16.89it/s, loss=1.39, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  73%|███████▎  | 8190/11174 [08:04<02:56, 16.89it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▎  | 8220/11174 [08:06<02:54, 16.89it/s, loss=1.55, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▎  | 8220/11174 [08:06<02:54, 16.89it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8250/11174 [08:08<02:53, 16.89it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8250/11174 [08:08<02:53, 16.89it/s, loss=1.14, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8280/11174 [08:10<02:51, 16.89it/s, loss=1.14, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.660, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8280/11174 [08:10<02:51, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.600, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8310/11174 [08:12<02:49, 16.89it/s, loss=1.27, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.600, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  74%|███████▍  | 8310/11174 [08:12<02:49, 16.89it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▍  | 8340/11174 [08:13<02:47, 16.89it/s, loss=1.62, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.672, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▍  | 8340/11174 [08:13<02:47, 16.89it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▍  | 8370/11174 [08:15<02:46, 16.89it/s, loss=1.16, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▍  | 8370/11174 [08:15<02:46, 16.89it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  75%|███████▌  | 8400/11174 [08:17<02:44, 16.89it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▌  | 8400/11174 [08:17<02:44, 16.89it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.723, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▌  | 8430/11174 [08:19<02:42, 16.89it/s, loss=1.77, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.723, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  75%|███████▌  | 8430/11174 [08:19<02:42, 16.89it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  76%|███████▌  | 8460/11174 [08:20<02:40, 16.89it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.679, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  76%|███████▌  | 8460/11174 [08:20<02:40, 16.89it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  76%|███████▌  | 8490/11174 [08:22<02:38, 16.89it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.609, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  76%|███████▌  | 8490/11174 [08:22<02:38, 16.89it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.685, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  76%|███████▌  | 8520/11174 [08:24<02:37, 16.89it/s, loss=1.47, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.685, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  76%|███████▌  | 8520/11174 [08:24<02:37, 16.89it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  77%|███████▋  | 8550/11174 [08:26<02:35, 16.89it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8550/11174 [08:26<02:35, 16.89it/s, loss=1.17, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8580/11174 [08:28<02:33, 16.89it/s, loss=1.17, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.576, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8580/11174 [08:28<02:33, 16.89it/s, loss=2.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8610/11174 [08:29<02:31, 16.89it/s, loss=2.12, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.608, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8610/11174 [08:29<02:31, 16.89it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.737, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8640/11174 [08:31<02:30, 16.89it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.737, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  77%|███████▋  | 8640/11174 [08:31<02:30, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8670/11174 [08:33<02:28, 16.89it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.670, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8670/11174 [08:33<02:28, 16.89it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8700/11174 [08:35<02:26, 16.89it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.684, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8700/11174 [08:35<02:26, 16.89it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8730/11174 [08:36<02:24, 16.89it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.619, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8730/11174 [08:36<02:24, 16.89it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.588, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8760/11174 [08:38<02:22, 16.88it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.588, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  78%|███████▊  | 8760/11174 [08:38<02:22, 16.88it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.694, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▊  | 8790/11174 [08:40<02:21, 16.89it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.694, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▊  | 8790/11174 [08:40<02:21, 16.89it/s, loss=1.88, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8820/11174 [08:42<02:19, 16.89it/s, loss=1.88, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.630, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8820/11174 [08:42<02:19, 16.89it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8850/11174 [08:44<02:17, 16.89it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8850/11174 [08:44<02:17, 16.89it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8880/11174 [08:45<02:15, 16.88it/s, loss=1.95, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.598, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  79%|███████▉  | 8880/11174 [08:45<02:15, 16.88it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  80%|███████▉  | 8910/11174 [08:47<02:14, 16.88it/s, loss=1.36, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.681, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  80%|███████▉  | 8910/11174 [08:47<02:14, 16.88it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.706, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  80%|████████  | 8940/11174 [08:49<02:12, 16.88it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.706, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  80%|████████  | 8940/11174 [08:49<02:12, 16.88it/s, loss=1.74, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  80%|████████  | 8970/11174 [08:51<02:10, 16.88it/s, loss=1.74, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  80%|████████  | 8970/11174 [08:51<02:10, 16.88it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9000/11174 [08:53<02:08, 16.88it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.621, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9000/11174 [08:53<02:08, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9030/11174 [08:54<02:07, 16.88it/s, loss=1.78, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9030/11174 [08:54<02:07, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9060/11174 [08:56<02:05, 16.88it/s, los

s=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.648, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████  | 9060/11174 [08:56<02:05, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.626, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████▏ | 9090/11174 [08:58<02:03, 16.88it/s, loss=1.41, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.626, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  81%|████████▏ | 9090/11174 [08:58<02:03, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9120/11174 [09:00<02:01, 16.88it/s, loss=1.26, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.714, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9120/11174 [09:00<02:01, 16.88it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  82%|████████▏ | 9150/11174 [09:02<01:59, 16.88it/s, loss=1.3, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.605, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9150/11174 [09:02<01:59, 16.88it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9180/11174 [09:03<01:58, 16.88it/s, loss=1.22, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.622, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9180/11174 [09:03<01:58, 16.88it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9210/11174 [09:05<01:56, 16.87it/s, loss=1.34, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.680, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  82%|████████▏ | 9210/11174 [09:05<01:56, 16.87it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9240/11174 [09:07<01:54, 16.87it/s, loss=1.18, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.638, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9240/11174 [09:07<01:54, 16.87it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.650, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9270/11174 [09:09<01:52, 16.87it/s, loss=1.45, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.650, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9270/11174 [09:09<01:52, 16.87it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  83%|████████▎ | 9300/11174 [09:11<01:51, 16.88it/s, loss=1.5, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.688, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9300/11174 [09:11<01:51, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.690, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9330/11174 [09:12<01:49, 16.88it/s, loss=1.35, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.690, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  83%|████████▎ | 9330/11174 [09:12<01:49, 16.88it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9360/11174 [09:14<01:47, 16.87it/s, loss=1.65, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.615, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9360/11174 [09:14<01:47, 16.87it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9390/11174 [09:16<01:45, 16.88it/s, loss=1.28, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.618, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9390/11174 [09:16<01:45, 16.88it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9420/11174 [09:18<01:43, 16.87it/s, loss=1.19, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.708, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  84%|████████▍ | 9420/11174 [09:18<01:43, 16.87it/s, loss=1.93, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.718, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▍ | 9450/11174 [09:19<01:42, 16.88it/s, loss=1.93, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.718, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▍ | 9450/11174 [09:19<01:42, 16.88it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▍ | 9480/11174 [09:21<01:40, 16.87it/s, loss=1.54, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.665, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▍ | 9480/11174 [09:21<01:40, 16.87it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▌ | 9510/11174 [09:23<01:38, 16.87it/s, loss=1.33, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.616, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▌ | 9510/11174 [09:23<01:38, 16.87it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.586, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▌ | 9540/11174 [09:25<01:36, 16.87it/s, loss=1.64, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.586, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  85%|████████▌ | 9540/11174 [09:25<01:36, 16.87it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9570/11174 [09:27<01:35, 16.87it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.709, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9570/11174 [09:27<01:35, 16.87it/s, loss=1.96, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.746, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9600/11174 [09:29<01:33, 16.87it/s, loss=1.96, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.746, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9600/11174 [09:29<01:33, 16.87it/s, loss=1.86, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9630/11174 [09:30<01:31, 16.87it/s, loss=1.86, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.652, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▌ | 9630/11174 [09:30<01:31, 16.87it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▋ | 9660/11174 [09:32<01:29, 16.87it/s, loss=1.75, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.659, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  86%|████████▋ | 9660/11174 [09:32<01:29, 16.87it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.641, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9690/11174 [09:34<01:27, 16.87it/s, loss=1.51, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.641, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9690/11174 [09:34<01:27, 16.87it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.636, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9720/11174 [09:36<01:26, 16.87it/s, loss=1.66, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.636, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9720/11174 [09:36<01:26, 16.87it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9750/11174 [09:37<01:24, 16.87it/s, loss=1.37, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.633, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  87%|████████▋ | 9750/11174 [09:37<01:24, 16.87it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676] #015Epoch 2:  88%|████████▊ | 9780/11174 [09:39<01:22, 16.87it/s, loss=1.1, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.644, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9780/11174 [09:39<01:22, 16.87it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9810/11174 [09:41<01:20, 16.87it/s, loss=1.32, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.669, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9810/11174 [09:41<01:20, 16.87it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.713, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9840/11174 [09:43<01:19, 16.87it/s, loss=1.72, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.713, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9840/11174 [09:43<01:19, 16.87it/s, loss=1.03, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9870/11174 [09:45<01:17, 16.87it/s, loss=1.03, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.628, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  88%|████████▊ | 9870/11174 [09:45<01:17, 16.87it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]#015Epoch 2:  89%|████████▊ | 9900/11174 [09:47<01:15, 16.86it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating: 0it [00:00, ?it/s]#033[A
#015Validating:   0%|          | 0/1275 [00:00<?, ?it/s]#033[A
#015Validating:   2%|▏         | 30/1275 [00:00<00:38, 32.22it/s]#033[A#015Epoch 2:  89%|████████▉ | 9930/11174 [09:47<01:13, 16.89it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:   5%|▍         | 60/1275 [00:01<00:34, 34.94it/s]#033[A#015Epoch 2:  89%|████████▉ | 9960/11174 [09:48<01:11, 16.92it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:   7%|▋         | 90/1275 [00:02<00:32, 37.02it/s]#033[A#015Epoch 2:  89%|████████▉ | 9990/11174 [09:49<01:09, 16.95it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:   9%|▉         | 120/1275 [00:03<00:29, 38.62it/s]#033[A#015Epoch 2:  90%|████████▉ | 10020/11174 [09:50<01:07, 16.98it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  12%|█▏        | 150/1275 [00:03<00:27, 40.21it/s]#033[A#015Epoch 2:  90%|████████▉ | 10050/11174 [09:50<01:06, 17.01it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  14%|█▍        | 180/1275 [00:04<00:26, 41.25it/s]#033[A#015Epoch 2:  90%|█████████ | 10080/11174 [09:51<01:04, 17.04it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  16%|█▋        | 210/1275 [00:05<00:25, 42.19it/s]#033[A#015Epoch 2:  90%|█████████ | 10110/11174 [09:52<01:02, 17.07it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  19%|█▉        | 240/1275 [00:05<00:24, 42.75it/s]#033[A#015Epoch 2:  91%|█████████ | 10140/11174 [09:52<01:00, 17.11it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  21%|██        | 270/1275 [00:06<00:23, 43.25it/s]#033[A#015Epoch 2:  91%|█████████ | 10170/11174 [09:53<00:58, 17.14it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  24%|██▎       | 300/1275 [00:07<00:22, 43.55it/s]#033[A#015Epoch 2:  91%|█████████▏| 10200/11174 [09:54<00:56, 17.17it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  26%|██▌       | 330/1275 [00:07<00:21, 43.37it/s]#033[A#015Epoch 2:  92%|█████████▏| 10230/11174 [09:54<00:54, 17.20it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]

#015Validating:  28%|██▊       | 360/1275 [00:08<00:21, 43.19it/s]#033[A#015Epoch 2:  92%|█████████▏| 10260/11174 [09:55<00:53, 17.23it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  31%|███       | 390/1275 [00:09<00:20, 43.60it/s]#033[A#015Epoch 2:  92%|█████████▏| 10290/11174 [09:56<00:51, 17.26it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  33%|███▎      | 420/1275 [00:09<00:19, 43.95it/s]#033[A#015Epoch 2:  92%|█████████▏| 10320/11174 [09:56<00:49, 17.29it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  35%|███▌      | 450/1275 [00:10<00:18, 43.98it/s]#033[A#015Epoch 2:  93%|█████████▎| 10350/11174 [09:57<00:47, 17.32it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  38%|███▊      | 480/1275 [00:11<00:17, 44.32it/s]#033[A#015Epoch 2:  93%|█████████▎| 10380/11174 [09:58<00:45, 17.35it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  40%|████      | 510/1275 [00:11<00:17, 44.38it/s]#033[A#015Epoch 2:  93%|█████████▎| 10410/11174 [09:58<00:43, 17.38it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  42%|████▏     | 540/1275 [00:12<00:16, 44.49it/s]#033[A#015Epoch 2:  93%|█████████▎| 10440/11174 [09:59<00:42, 17.41it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  45%|████▍     | 570/1275 [00:13<00:16, 43.35it/s]#033[A#015Epoch 2:  94%|█████████▎| 10470/11174 [10:00<00:40, 17.44it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  47%|████▋     | 600/1275 [00:13<00:15, 43.36it/s]#033[A#015Epoch 2:  94%|█████████▍| 10500/11174 [10:00<00:38, 17.47it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  49%|████▉     | 630/1275 [00:14<00:14, 43.39it/s]#033[A#015Epoch 2:  94%|█████████▍| 10530/11174 [10:01<00:36, 17.50it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  52%|█████▏    | 660/1275 [00:15<00:14, 43.60it/s]#033[A#015Epoch 2:  95%|█████████▍| 10560/11174 [10:02<00:35, 17.53it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  54%|█████▍    | 690/1275 [00:15<00:13, 43.62it/s]#033[A#015Epoch 2:  95%|█████████▍| 10590/11174 [10:03<00:33, 17.56it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  56%|█████▋    | 720/1275 [00:16<00:12, 43.82it/s]#033[A#015Epoch 2:  95%|█████████▌| 10620/11174 [10:03<00:31, 17.59it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  59%|█████▉    | 750/1275 [00:17<00:12, 43.66it/s]#033[A#015Epoch 2:  95%|█████████▌| 10650/11174 [10:04<00:29, 17.62it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  61%|██████    | 780/1275 [00:18<00:11, 43.41it/s]#033[A#015Epoch 2:  96%|█████████▌| 10680/11174 [10:05<00:27, 17.65it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  64%|██████▎   | 810/1275 [00:18<00:10, 43.47it/s]#033[A#015Epoch 2:  96%|█████████▌| 10710/11174 [10:05<00:26, 17.68it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  66%|██████▌   | 840/1275 [00:19<00:09, 43.74it/s]#033[A#015Epoch 2:  96%|█████████▌| 10740/11174 [10:06<00:24, 17.71it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  68%|██████▊   | 870/1275 [00:20<00:09, 43.03it/s]#033[A#015Epoch 2:  96%|█████████▋| 10770/11174 [10:07<00:22, 17.74it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  71%|███████   | 900/1275 [00:20<00:08, 42.75it/s]#033[A#015Epoch 2:  97%|█████████▋| 10800/11174 [10:07<00:21, 17.77it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  73%|███████▎  | 930/1275 [00:21<00:07, 43.24it/s]#033[A#015Epoch 2:  97%|█████████▋| 10830/11174 [10:08<00:19, 17.80it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  75%|███████▌  | 960/1275 [00:22<00:07, 43.66it/s]#033[A#015Epoch 2:  97%|█████████▋| 10860/11174 [10:09<00:17, 17.82it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  78%|███████▊  | 990/1275 [00:22<00:06, 43.84it/s]#033[A#015Epoch 2:  97%|█████████▋| 10890/11174 [10:09<00:15, 17.85it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  80%|████████  | 1020/1275 [00:23<00:05, 43.79it/s]#033[A#015Epoch 2:  98%|█████████▊| 10920/11174 [10:10<00:14, 17.88it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  82%|████████▏ | 1050/1275 [00:24<00:05, 44.01it/s]#033[A#015Epoch 2:  98%|█████████▊| 10950/11174 [10:11<00:12, 17.91it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  85%|████████▍ | 1080/1275 [00:24<00:04, 44.21it/s]#033[A#015Epoch 2:  98%|█████████▊| 10980/11174 [10:11<00:10, 17.94it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  87%|████████▋ | 1110/1275 [00:25<00:03, 44.26it/s]#033[A#015Epoch 2:  99%|█████████▊| 11010/11174 [10:12<00:09, 17.97it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  89%|████████▉ | 1140/1275 [00:26<00:03, 44.34it/s]#033[A#015Epoch 2:  99%|█████████▉| 11040/11174 [10:13<00:07, 18.00it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  92%|█████████▏| 1170/1275 [00:26<00:02, 44.38it/s]#033[A#015Epoch 2:  99%|█████████▉| 11070/11174 [10:14<00:05, 18.03it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  94%|█████████▍| 1200/1275 [00:27<00:01, 44.30it/s]#033[A#015Epoch 2:  99%|█████████▉| 11100/11174 [10:14<00:04, 18.06it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  96%|█████████▋| 1230/1275 [00:28<00:01, 43.63it/s]#033[A#015Epoch 2: 100%|█████████▉| 11130/11174 [10:15<00:02, 18.09it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating:  99%|█████████▉| 1260/1275 [00:29<00:00, 43.65it/s]#033[A#015Epoch 2: 100%|█████████▉| 11160/11174 [10:16<00:00, 18.11it/s, loss=1.02, v_num=0, val_f1_epoch=0.651, val_loss_epoch=0.140, train_f1=0.657, val_f1_step=0.495, val_loss_step=0.0676]
#015Validating: 100%|██████████| 1275/1275 [00:29<00:00, 43.98it/s]#033[A#015Epoch 2: 100%|██████████| 11174/11174 [10:16<00:00, 18.12it/s, loss=1.68, v_num=0, val_f1_epoch=0.664, val_loss_epoch=0.136, train_f1=0.569, val_f1_step=0.494, val_loss_step=0.072] 
#015                                                               #033[A#015Epoch 2: 100%|██████████| 11174/11174 [10:18<00:00, 18.05it/s, loss=1.68, v_num=0, val_f1_epoch=0.664, val_loss_epoch=0.136, train_f1=0.569, val_f1_step=0.494, val_loss_step=0.072]

2021-10-05 19:47:58.164450: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/core/decorators.py:65: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `forward`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."
INFO:root:reading, preprocessing data
2021/10/05 19:48:03 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
DEBUG:filelock:Attempting to acquire lock 140290714995064 on /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda.lock
DEBUG:filelock:Lock 140290714995064 acquired on /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda.lock
#015Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 525/525 [00:00<00:00, 489kB/s]
DEBUG:filelock:Attempting to release lock 140290714995064 on /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda.lock
DEBUG:filelock:Lock 140290714995064 released on /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda.lock
DEBUG:filelock:Attempting to acquire lock 140290715144880 on /root/.cache/huggingface/transformers/3046dbd571c6315fcfeb5a2bf3cb9cd4171cafef23ab51cff552baa94fad032c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140290715144880 acquired on /root/.cache/huggingface/transformers/3046dbd571c6315fcfeb5a2bf3cb9cd4171cafef23ab51cff552baa94fad032c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
#015Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 232k/232k [00:00<00:00, 33.6MB/s]
DEBUG:filelock:Attempting to release lock 140290715144880 on /root/.cache/huggingface/transformers/3046dbd571c6315fcfeb5a2bf3cb9cd4171cafef23ab51cff552baa94fad032c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140290715144880 released on /root/.cache/huggingface/transformers/3046dbd571c6315fcfeb5a2bf3cb9cd4171cafef23ab51cff552baa94fad032c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Attempting to acquire lock 140290498155016 on /root/.cache/huggingface/transformers/5b9e9aa22907609566b731f4ac500e12a8f652024928a0d35bd8cb4de48d04dc.182b458c89816670d9a6a9910ed233b7a10984e821dac6b9a056a9512b4e1ad0.lock
DEBUG:filelock:Lock 140290498155016 acquired on /root/.cache/huggingface/transformers/5b9e9aa22907609566b731f4ac500e12a8f652024928a0d35bd8cb4de48d04dc.182b458c89816670d9a6a9910ed233b7a10984e821dac6b9a056a9512b4e1ad0.lock
2021-10-05 20:56:33,569 sagemaker-training-toolkit INFO     Reporting training SUCCESS
#015Downloading:   0%|          | 0.00/51.0M [00:00<?, ?B/s]#015Downloading:   7%|▋         | 3.60M/51.0M [00:00<00:01, 35.5MB/s]#015Downloading:  15%|█▌        | 7.77M/51.0M [00:00<00:01, 37.2MB/s]#015Downloading:  22%|██▏       | 11.2M/51.0M [00:00<00:01, 36.2MB/s]#015Downloading:  31%|███       | 15.6M/51.0M [00:00<00:00, 38.3MB/s]#015Downloading:  40%|████      | 20.6M/51.0M [00:00<00:00, 41.1MB/s]#015Downloading:  47%|████▋     | 24.2M/51.0M [00:00<00:00, 30.0MB/s]#015Downloading:  53%|█████▎    | 27.2M/51.0M [00:00<00:00, 29.7MB/s]#015Downloading:  59%|█████▉    | 30.3M/51.0M [00:00<00:00, 29.5MB/s]#015Downloading:  67%|██████▋   | 34.1M/51.0M [00:01<00:00, 31.2MB/s]#015Downloading:  75%|███████▌  | 38.3M/51.0M [00:01<00:00, 33.9MB/s]#015Downloading:  82%|████████▏ | 41.8M/51.0M [00:01<00:00, 32.7MB/s]#015Downloading:  90%|████████▉ | 45.7M/51.0M [00:01<00:00, 34.4MB/s]#015Downloading:  96%|█████████▋| 49.3M/51.0M [00:01<00:00, 33.2MB/s]#015Downloading: 100%|██████████| 51.0M/51.0M [00:01<00:00, 34.1MB/s]
DEBUG:filelock:Attempting to release lock 140290498155016 on /root/.cache/huggingface/transformers/5b9e9aa22907609566b731f4ac500e12a8f652024928a0d35bd8cb4de48d04dc.182b458c89816670d9a6a9910ed233b7a10984e821dac6b9a056a9512b4e1ad0.lock
DEBUG:filelock:Lock 140290498155016 released on /root/.cache/huggingface/transformers/5b9e9aa22907609566b731f4ac500e12a8f652024928a0d35bd8cb4de48d04dc.182b458c89816670d9a6a9910ed233b7a10984e821dac6b9a056a9512b4e1ad0.lock
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/trainer/deprecated_api.py:152: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
  "`Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5."
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/trainer/callback_hook.py:101: LightningDeprecationWarning: The signature of `Callback.on_train_epoch_end` has changed in v1.3. `outputs` parameter has been removed. Support for the old signature will be removed in v1.5
  "The signature of `Callback.on_train_epoch_end` has changed in v1.3."
Epoch 0, global step 3023: val_f1 reached 0.62877 (best 0.62877), saving model to "/opt/ml/model/model_subpillars_1d.ckpt" as top 1
Epoch 1, global step 6047: val_f1 reached 0.67361 (best 0.67361), saving model to "/opt/ml/model/model_subpillars_1d.ckpt" as top 1
Epoch 2, global step 9071: val_f1 reached 0.68902 (best 0.68902), saving model to "/opt/ml/model/model_subpillars_1d.ckpt" as top 1
FIT Profiler Report

Action                             #011|  Mean duration (s)#011|Num calls      #011|  Total time (s) #011|  Percentage %   #011|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              #011|  -              #011|_              #011|  799.07         #011|  100 %          #011|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 #011|  198.16         #011|3              #011|  594.49         #011|  74.398         #011|
run_training_batch                 #011|  0.056313       #011|9072           #011|  510.87         #011|  63.933         #011|
optimizer_step_and_closure_0       #011|  0.055901       #011|9072           #011|  507.14         #011|  63.466         #011|
training_step_and_backward         #011|  0.045896       #011|9072           #011|  416.37         #011|  52.106         #011|
backward                           #011|  0.022882       #011|9072           #011|  207.58         #011|  25.978         #011|
model_forward                      #011|  0.021314       #011|9072           #011|  193.36         #011|  24.198         #011|
training_step                      #011|  0.021015       #011|9072           #011|  190.65         #011|  23.859         #011|
evaluation_step_and_end            #011|  0.025824       #011|1193           #011|  30.807         #011|  3.8554         #011|
validation_step                    #011|  0.025602       #011|1193           #011|  30.543         #011|  3.8223         #011|
get_train_batch                    #011|  0.0028665      #011|9072           #011|  26.005         #011|  3.2544         #011|
on_validation_end                  #011|  0.8872         #011|4              #011|  3.5488         #011|  0.44411        #011|
cache_result                       #011|  2.2815e-05     #011|39912          #011|  0.91058        #011|  0.11395        #011|
on_train_batch_end                 #011|  8.52e-05       #011|9072           #011|  0.77294        #011|  0.096729       #011|
on_train_start                     #011|  0.63383        #011|1              #011|  0.63383        #011|  0.079321       #011|
on_batch_start                     #011|  2.7359e-05     #011|9072           #011|  0.2482         #011|  0.031062       #011|
on_after_backward                  #011|  2.7334e-05     #011|9072           #011|  0.24797        #011|  0.031032       #011|
on_before_zero_grad                #011|  2.3184e-05     #011|9072           #011|  0.21033        #011|  0.026321       #011|
on_batch_end                       #011|  2.2474e-05     #011|9072           #011|  0.20389        #011|  0.025516       #011|
on_train_batch_start               #011|  1.903e-05      #011|9072           #011|  0.17264        #011|  0.021605       #011|
training_step_end                  #011|  1.4079e-05     #011|9072           #011|  0.12773        #011|  0.015985       #011|
on_validation_batch_end            #011|  5.1591e-05     #011|1193           #011|  0.061548       #011|  0.0077024      #011|
on_validation_batch_start          #011|  2.4211e-05     #011|1193           #011|  0.028883       #011|  0.0036146      #011|
validation_step_end                #011|  1.4192e-05     #011|1193           #011|  0.016931       #011|  0.0021188      #011|
on_validation_epoch_end            #011|  0.00088144     #011|4              #011|  0.0035258      #011|  0.00044123     #011|
on_validation_start                #011|  0.00053908     #011|4              #011|  0.0021563      #011|  0.00026985     #011|
on_train_epoch_end                 #011|  0.00049127     #011|3              #011|  0.0014738      #011|  0.00018444     #011|
on_train_epoch_start               #011|  0.00046617     #011|3              #011|  0.0013985      #011|  0.00017502     #011|
on_train_end                       #011|  0.00072905     #011|1              #011|  0.00072905     #011|  9.1236e-05     #011|
on_epoch_start                     #011|  2.4172e-05     #011|7              #011|  0.0001692      #011|  2.1175e-05     #011|
on_epoch_end                       #011|  2.3508e-05     #011|7              #011|  0.00016456     #011|  2.0594e-05     #011|
on_validation_epoch_start          #011|  1.9807e-05     #011|4              #011|  7.9228e-05     #011|  9.915e-06      #011|
on_fit_start                       #011|  4.6399e-05     #011|1              #011|  4.6399e-05     #011|  5.8066e-06     #011|
on_before_accelerator_backend_setup#011|  2.2127e-05     #011|1              #011|  2.2127e-05     #011|  2.7691e-06     #011|
on_val_dataloader                  #011|  1.5671e-05     #011|1              #011|  1.5671e-05     #011|  1.9612e-06     #011|
on_train_dataloader                #011|  1.502e-05      #011|1              #011|  1.502e-05      #011|  1.8797e-06     #011|

#015  0%|          | 0/396 [00:00<?, ?it/s]#015  0%|          | 1/396 [00:00<01:40,  3.95it/s]#015  1%|▏         | 5/396 [00:00<01:12,  5.38it/s]#015  3%|▎         | 11/396 [00:00<00:52,  7.35it/s]#015  4%|▍         | 16/396 [00:00<00:38,  9.87it/s]#015  5%|▌         | 21/396 [00:00<00:28, 12.96it/s]#015  7%|▋         | 26/396 [00:00<00:22, 16.65it/s]#015  8%|▊         | 31/396 [00:00<00:17, 20.66it/s]#015  9%|▉         | 36/396 [00:01<00:14, 24.35it/s]#015 11%|█         | 42/396 [00:01<00:12, 28.78it/s]#015 12%|█▏        | 48/396 [00:01<00:10, 33.01it/s]#015 14%|█▎        | 54/396 [00:01<00:09, 36.80it/s]#015 15%|█▌        | 60/396 [00:01<00:08, 39.99it/s]#015 17%|█▋        | 66/396 [00:01<00:07, 42.60it/s]#015 18%|█▊        | 71/396 [00:01<00:07, 44.45it/s]#015 19%|█▉        | 76/396 [00:01<00:06, 45.95it/s]#015 21%|██        | 82/396 [00:01<00:06, 47.09it/s]#015 22%|██▏       | 88/396 [00:02<00:06, 47.99it/s]#015 24%|██▎       | 94/396 [00:02<00:06, 48.59it/s]#015 25%|██▌       | 99/396 [00:02<00:06, 48.95it/s]#015 26%|██▋       | 104/396 [00:02<00:05, 49.24it/s]#015 28%|██▊       | 110/396 [00:02<00:05, 49.46it/s]#015 29%|██▉       | 115/396 [00:02<00:05, 49.55it/s]#015 30%|███       | 120/396 [00:02<00:05, 49.67it/s]#015 32%|███▏      | 125/396 [00:02<00:05, 49.25it/s]#015 33%|███▎      | 131/396 [00:02<00:05, 49.62it/s]#015 34%|███▍      | 136/396 [00:03<00:05, 49.70it/s]#015 36%|███▌      | 142/396 [00:03<00:05, 49.84it/s]#015 37%|███▋      | 148/396 [00:03<00:04, 49.94it/s]#015 39%|███▊      | 153/396 [00:03<00:04, 49.91it/s]#015 40%|████      | 159/396 [00:03<00:04, 50.04it/s]#015 42%|████▏     | 165/396 [00:03<00:04, 50.12it/s]#015 43%|████▎     | 171/396 [00:03<00:04, 50.17it/s]#015 45%|████▍     | 177/396 [00:03<00:04, 49.98it/s]#015 46%|████▌     | 183/396 [00:03<00:04, 50.12it/s]#015 48%|████▊     | 189/396 [00:04<00:04, 50.14it/s]#015 49%|████▉     | 195/396 [00:04<00:04, 50.05it/s]#015 51%|█████     | 201/396 [00:04<00:03, 49.98it/s]#015 52%|█████▏    | 206/396 [00:04<00:03, 48.18it/s]#015 53%|█████▎    | 211/396 [00:04<00:03, 46.75it/s]#015 55%|█████▍    | 216/396 [00:04<00:03, 47.64it/s]#015 56%|█████▌    | 221/396 [00:04<00:03, 47.32it/s]#015 57%|█████▋    | 226/396 [00:04<00:03, 47.56it/s]#015 58%|█████▊    | 231/396 [00:04<00:03, 48.13it/s]#015 60%|█████▉    | 236/396 [00:05<00:03, 48.19it/s]#015 61%|██████    | 241/396 [00:05<00:03, 48.66it/s]#015 62%|██████▏   | 246/396 [00:05<00:03, 47.89it/s]#015 64%|██████▎   | 252/396 [00:05<00:02, 48.63it/s]#015 65%|██████▍   | 257/396 [00:05<00:02, 48.78it/s]#015 66%|██████▌   | 262/396 [00:05<00:02, 49.13it/s]#015 68%|██████▊   | 268/396 [00:05<00:02, 49.42it/s]#015 69%|██████▉   | 273/396 [00:05<00:02, 49.28it/s]#015 70%|███████   | 278/396 [00:05<00:02, 49.31it/s]#015 71%|███████▏  | 283/396 [00:06<00:02, 46.96it/s]#015 73%|███████▎  | 288/396 [00:06<00:02, 46.76it/s]#015 74%|███████▍  | 293/396 [00:06<00:02, 46.21it/s]#015 75%|███████▌  | 298/396 [00:06<00:02, 45.34it/s]#015 77%|███████▋  | 303/396 [00:06<00:02, 44.25it/s]#015 78%|███████▊  | 308/396 [00:06<00:01, 45.71it/s]#015 79%|███████▉  | 313/396 [00:06<00:01, 46.90it/s]#015 80%|████████  | 318/396 [00:06<00:01, 46.67it/s]#015 82%|████████▏ | 323/396 [00:06<00:01, 45.89it/s]#015 83%|████████▎ | 329/396 [00:07<00:01, 47.18it/s]#015 84%|████████▍ | 334/396 [00:07<00:01, 47.96it/s]#015 86%|████████▌ | 340/396 [00:07<00:01, 48.60it/s]#015 87%|████████▋ | 346/396 [00:07<00:01, 49.03it/s]#015 89%|████████▊ | 351/396 [00:07<00:00, 49.23it/s]#015 90%|████████▉ | 356/396 [00:07<00:00, 49.31it/s]#015 91%|█████████ | 361/396 [00:07<00:00, 49.46it/s]#015 92%|█████████▏| 366/396 [00:07<00:00, 49.49it/s]#015 94%|█████████▎| 371/396 [00:07<00:00, 49.33it/s]#015 95%|█████████▌| 377/396 [00:07<00:00, 49.67it/s]#015 97%|█████████▋| 383/396 [00:08<00:00, 49.91it/s]#015 98%|█████████▊| 389/396 [00:08<00:00, 50.04it/s]#015100%|█████████▉| 395/396 [00:08<00:00, 50.31it/s]#015397it [00:08, 47.04it/s]                         
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/trainer/deprecated_api.py:152: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
  "`Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5."
Epoch 0, global step 9898: val_f1 reached 0.63512 (best 0.63512), saving model to "/opt/ml/model/model_subpillars_2d.ckpt" as top 1
Epoch 1, global step 19797: val_f1 reached 0.65099 (best 0.65099), saving model to "/opt/ml/model/model_subpillars_2d.ckpt" as top 1
Epoch 2, global step 29696: val_f1 reached 0.66412 (best 0.66412), saving model to "/opt/ml/model/model_subpillars_2d.ckpt" as top 1
FIT Profiler Report

Action                             #011|  Mean duration (s)#011|Num calls      #011|  Total time (s) #011|  Percentage %   #011|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              #011|  -              #011|_              #011|  2434.5         #011|  100 %          #011|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 #011|  617.41         #011|3              #011|  1852.2         #011|  76.084         #011|
run_training_batch                 #011|  0.053403       #011|29697          #011|  1585.9         #011|  65.143         #011|
optimizer_step_and_closure_0       #011|  0.052993       #011|29697          #011|  1573.7         #011|  64.644         #011|
training_step_and_backward         #011|  0.042912       #011|29697          #011|  1274.4         #011|  52.347         #011|
backward                           #011|  0.022885       #011|29697          #011|  679.61         #011|  27.916         #011|
model_forward                      #011|  0.018267       #011|29697          #011|  542.49         #011|  22.284         #011|
training_step                      #011|  0.017968       #011|29697          #011|  533.61         #011|  21.919         #011|
get_train_batch                    #011|  0.0027663      #011|29697          #011|  82.152         #011|  3.3745         #011|
evaluation_step_and_end            #011|  0.019727       #011|3827           #011|  75.495         #011|  3.1011         #011|
validation_step                    #011|  0.019506       #011|3827           #011|  74.65          #011|  3.0664         #011|
on_validation_end                  #011|  1.8118         #011|4              #011|  7.2472         #011|  0.29769        #011|
cache_result                       #011|  2.2793e-05     #011|130314         #011|  2.9703         #011|  0.12201        #011|
on_train_batch_end                 #011|  8.4963e-05     #011|29697          #011|  2.5231         #011|  0.10364        #011|
on_after_backward                  #011|  2.7644e-05     #011|29697          #011|  0.82094        #011|  0.033721       #011|
on_batch_start                     #011|  2.6648e-05     #011|29697          #011|  0.79138        #011|  0.032507       #011|
on_before_zero_grad                #011|  2.3494e-05     #011|29697          #011|  0.69769        #011|  0.028659       #011|
on_batch_end                       #011|  2.2281e-05     #011|29697          #011|  0.66167        #011|  0.027179       #011|
on_train_batch_start               #011|  1.9007e-05     #011|29697          #011|  0.56444        #011|  0.023185       #011|
on_train_start                     #011|  0.47551        #011|1              #011|  0.47551        #011|  0.019532       #011|
training_step_end                  #011|  1.4337e-05     #011|29697          #011|  0.42577        #011|  0.017489       #011|
on_validation_batch_end            #011|  5.096e-05      #011|3827           #011|  0.19502        #011|  0.0080109      #011|
on_validation_batch_start          #011|  2.3973e-05     #011|3827           #011|  0.091744       #011|  0.0037686      #011|
validation_step_end                #011|  1.4179e-05     #011|3827           #011|  0.054264       #011|  0.002229       #011|
on_validation_epoch_end            #011|  0.00080858     #011|4              #011|  0.0032343      #011|  0.00013285     #011|
on_validation_start                #011|  0.00050866     #011|4              #011|  0.0020347      #011|  8.3577e-05     #011|
on_train_epoch_start               #011|  0.00048585     #011|3              #011|  0.0014575      #011|  5.9871e-05     #011|
on_train_epoch_end                 #011|  0.00029731     #011|3              #011|  0.00089192     #011|  3.6637e-05     #011|
on_train_end                       #011|  0.00062631     #011|1              #011|  0.00062631     #011|  2.5727e-05     #011|
on_epoch_start                     #011|  2.9438e-05     #011|7              #011|  0.00020607     #011|  8.4645e-06     #011|
on_epoch_end                       #011|  2.4174e-05     #011|7              #011|  0.00016922     #011|  6.9509e-06     #011|
on_validation_epoch_start          #011|  2.4502e-05     #011|4              #011|  9.8007e-05     #011|  4.0258e-06     #011|
on_fit_start                       #011|  3.3522e-05     #011|1              #011|  3.3522e-05     #011|  1.377e-06      #011|
on_before_accelerator_backend_setup#011|  2.114e-05      #011|1              #011|  2.114e-05      #011|  8.6836e-07     #011|
on_train_dataloader                #011|  1.6903e-05     #011|1              #011|  1.6903e-05     #011|  6.9432e-07     #011|
on_val_dataloader                  #011|  1.4778e-05     #011|1              #011|  1.4778e-05     #011|  6.0703e-07     #011|

#015  0%|          | 0/1274 [00:00<?, ?it/s]#015  0%|          | 1/1274 [00:00<06:04,  3.49it/s]#015  0%|          | 5/1274 [00:00<04:26,  4.76it/s]#015  1%|          | 10/1274 [00:00<03:13,  6.52it/s]#015  1%|▏         | 16/1274 [00:00<02:22,  8.84it/s]#015  2%|▏         | 22/1274 [00:00<01:46, 11.76it/s]#015  2%|▏         | 28/1274 [00:00<01:21, 15.29it/s]#015  3%|▎         | 33/1274 [00:00<01:04, 19.21it/s]#015  3%|▎         | 38/1274 [00:01<00:53, 23.26it/s]#015  3%|▎         | 44/1274 [00:01<00:44, 27.80it/s]#015  4%|▍         | 50/1274 [00:01<00:37, 32.29it/s]#015  4%|▍         | 55/1274 [00:01<00:34, 35.70it/s]#015  5%|▍         | 60/1274 [00:01<00:31, 38.78it/s]#015  5%|▌         | 65/1274 [00:01<00:29, 40.91it/s]#015  5%|▌         | 70/1274 [00:01<00:28, 42.68it/s]#015  6%|▌         | 75/1274 [00:01<00:27, 44.38it/s]#015  6%|▋         | 80/1274 [00:01<00:26, 45.38it/s]#015  7%|▋         | 85/1274 [00:02<00:25, 46.49it/s]#015  7%|▋         | 90/1274 [00:02<00:25, 46.28it/s]#015  7%|▋         | 95/1274 [00:02<00:25, 46.75it/s]#015  8%|▊         | 100/1274 [00:02<00:24, 47.15it/s]#015  8%|▊         | 105/1274 [00:02<00:25, 46.68it/s]#015  9%|▊         | 110/1274 [00:02<00:25, 46.04it/s]#015  9%|▉         | 115/1274 [00:02<00:25, 46.11it/s]#015  9%|▉         | 120/1274 [00:02<00:25, 46.11it/s]#015 10%|▉         | 126/1274 [00:02<00:24, 47.61it/s]#015 10%|█         | 132/1274 [00:03<00:23, 48.71it/s]#015 11%|█         | 138/1274 [00:03<00:22, 49.69it/s]#015 11%|█▏        | 144/1274 [00:03<00:22, 50.26it/s]#015 12%|█▏        | 150/1274 [00:03<00:22, 49.63it/s]#015 12%|█▏        | 155/1274 [00:03<00:22, 49.16it/s]#015 13%|█▎        | 160/1274 [00:03<00:22, 48.58it/s]#015 13%|█▎        | 165/1274 [00:03<00:23, 47.36it/s]#015 13%|█▎        | 170/1274 [00:03<00:23, 47.96it/s]#015 14%|█▍        | 176/1274 [00:03<00:22, 48.29it/s]#015 14%|█▍        | 181/1274 [00:04<00:22, 48.06it/s]#015 15%|█▍        | 186/1274 [00:04<00:22, 47.71it/s]#015 15%|█▌        | 192/1274 [00:04<00:22, 48.68it/s]#015 15%|█▌        | 197/1274 [00:04<00:22, 47.77it/s]#015 16%|█▌        | 202/1274 [00:04<00:22, 47.11it/s]#015 16%|█▌        | 207/1274 [00:04<00:22, 46.44it/s]#015 17%|█▋        | 212/1274 [00:04<00:22, 47.07it/s]#015 17%|█▋        | 217/1274 [00:04<00:22, 47.28it/s]#015 17%|█▋        | 222/1274 [00:04<00:21, 47.99it/s]#015 18%|█▊        | 227/1274 [00:05<00:21, 47.85it/s]#015 18%|█▊        | 232/1274 [00:05<00:22, 47.14it/s]#015 19%|█▊        | 237/1274 [00:05<00:22, 47.10it/s]#015 19%|█▉        | 243/1274 [00:05<00:21, 48.01it/s]#015 20%|█▉        | 249/1274 [00:05<00:20, 49.11it/s]#015 20%|██        | 255/1274 [00:05<00:20, 49.71it/s]#015 20%|██        | 261/1274 [00:05<00:20, 50.33it/s]#015 21%|██        | 267/1274 [00:05<00:19, 50.82it/s]#015 21%|██▏       | 273/1274 [00:05<00:19, 50.94it/s]#015 22%|██▏       | 279/1274 [00:06<00:19, 50.87it/s]#015 22%|██▏       | 285/1274 [00:06<00:19, 50.32it/s]#015 23%|██▎       | 291/1274 [00:06<00:19, 49.19it/s]#015 23%|██▎       | 297/1274 [00:06<00:19, 49.71it/s]#015 24%|██▎       | 302/1274 [00:06<00:19, 48.89it/s]#015 24%|██▍       | 308/1274 [00:06<00:19, 49.35it/s]#015 25%|██▍       | 314/1274 [00:06<00:19, 50.17it/s]#015 25%|██▌       | 320/1274 [00:06<00:18, 50.67it/s]#015 26%|██▌       | 326/1274 [00:06<00:18, 50.92it/s]#015 26%|██▌       | 332/1274 [00:07<00:18, 51.07it/s]#015 27%|██▋       | 338/1274 [00:07<00:18, 51.24it/s]#015 27%|██▋       | 344/1274 [00:07<00:18, 51.43it/s]#015 27%|██▋       | 350/1274 [00:07<00:17, 51.61it/s]#015 28%|██▊       | 356/1274 [00:07<00:17, 51.87it/s]#015 28%|██▊       | 362/1274 [00:07<00:17, 51.98it/s]#015 29%|██▉       | 368/1274 [00:07<00:17, 52.09it/s]#015 29%|██▉       | 374/1274 [00:07<00:17, 51.74it/s]#015 30%|██▉       | 380/1274 [00:08<00:17, 50.54it/s]#015 30%|███       | 386/1274 [00:08<00:17, 50.48it/s]#015 31%|███       | 392/1274 [00:08<00:17, 50.91it/s]#015 31%|███       | 398/1274 [00:08<00:17, 51.24it/s]#015 32%|███▏      | 404/1274 [00:08<00:16, 51.29it/s]#015 32%|███▏      | 410/1274 [00:08<00:16, 51.56it/s]#015 33%|███▎      | 416/1274 [00:08<00:16, 51.37it/s]#015 33%|███▎      | 422/1274 [00:08<00:17, 49.97it/s]#015 34%|███▎      | 428/1274 [00:08<00:16, 50.40it/s]#015 34%|███▍      | 434/1274 [00:09<00:16, 50.53it/s]#015 35%|███▍      | 440/1274 [00:09<00:16, 50.69it/s]#015 35%|███▌      | 446/1274 [00:09<00:16, 50.90it/s]#015 35%|███▌      | 452/1274 [00:09<00:16, 51.29it/s]#015 36%|███▌      | 458/1274 [00:09<00:16, 50.41it/s]#015 36%|███▋      | 464/1274 [00:09<00:16, 49.80it/s]#015 37%|███▋      | 469/1274 [00:09<00:16, 49.70it/s]#015 37%|███▋      | 475/1274 [00:09<00:15, 50.32it/s]#015 38%|███▊      | 481/1274 [00:10<00:15, 50.85it/s]#015 38%|███▊      | 487/1274 [00:10<00:15, 50.13it/s]#015 39%|███▊      | 493/1274 [00:10<00:15, 50.15it/s]#015 39%|███▉      | 499/1274 [00:10<00:15, 50.24it/s]#015 40%|███▉      | 505/1274 [00:10<00:15, 49.79it/s]#015 40%|████      | 510/1274 [00:10<00:15, 48.82it/s]#015 41%|████      | 516/1274 [00:10<00:15, 49.35it/s]#015 41%|████      | 522/1274 [00:10<00:15, 49.77it/s]#015 41%|████▏     | 527/1274 [00:10<00:15, 49.45it/s]#015 42%|████▏     | 532/1274 [00:11<00:15, 48.89it/s]#015 42%|████▏     | 537/1274 [00:11<00:14, 49.21it/s]#015 43%|████▎     | 542/1274 [00:11<00:15, 47.70it/s]#015 43%|████▎     | 547/1274 [00:11<00:15, 47.94it/s]#015 43%|████▎     | 553/1274 [00:11<00:14, 48.94it/s]#015 44%|████▍     | 558/1274 [00:11<00:14, 49.21it/s]#015 44%|████▍     | 564/1274 [00:11<00:14, 49.90it/s]#015 45%|████▍     | 570/1274 [00:11<00:13, 50.39it/s]#015 45%|████▌     | 576/1274 [00:11<00:13, 50.84it/s]#015 46%|████▌     | 582/1274 [00:12<00:13, 50.59it/s]#015 46%|████▌     | 588/1274 [00:12<00:13, 50.20it/s]#015 47%|████▋     | 594/1274 [00:12<00:13, 49.07it/s]#015 47%|████▋     | 600/1274 [00:12<00:13, 49.83it/s]#015 48%|████▊     | 606/1274 [00:12<00:13, 50.28it/s]#015 48%|████▊     | 612/1274 [00:12<00:13, 50.58it/s]#015 49%|████▊     | 618/1274 [00:12<00:12, 50.73it/s]#015 49%|████▉     | 624/1274 [00:12<00:12, 51.12it/s]#015 49%|████▉     | 630/1274 [00:13<00:12, 50.81it/s]#015 50%|████▉     | 636/1274 [00:13<00:12, 50.22it/s]#015 50%|█████     | 642/1274 [00:13<00:12, 50.70it/s]#015 51%|█████     | 648/1274 [00:13<00:12, 50.51it/s]#015 51%|█████▏    | 654/1274 [00:13<00:12, 49.97it/s]#015 52%|█████▏    | 660/1274 [00:13<00:12, 49.32it/s]#015 52%|█████▏    | 666/1274 [00:13<00:12, 50.04it/s]#015 53%|█████▎    | 672/1274 [00:13<00:12, 49.49it/s]#015 53%|█████▎    | 677/1274 [00:13<00:12, 48.71it/s]#015 54%|█████▎    | 682/1274 [00:14<00:12, 49.07it/s]#015 54%|█████▍    | 687/1274 [00:14<00:12, 48.81it/s]#015 54%|█████▍    | 692/1274 [00:14<00:12, 48.42it/s]#015 55%|█████▍    | 698/1274 [00:14<00:11, 49.47it/s]#015 55%|█████▌    | 703/1274 [00:14<00:11, 49.30it/s]#015 56%|█████▌    | 708/1274 [00:14<00:11, 48.83it/s]#015 56%|█████▌    | 714/1274 [00:14<00:11, 49.76it/s]#015 57%|█████▋    | 720/1274 [00:14<00:10, 50.44it/s]#015 57%|█████▋    | 726/1274 [00:14<00:10, 50.51it/s]#015 57%|█████▋    | 732/1274 [00:15<00:10, 50.46it/s]#015 58%|█████▊    | 738/1274 [00:15<00:10, 50.06it/s]#015 58%|█████▊    | 744/1274 [00:15<00:10, 49.52it/s]#015 59%|█████▉    | 750/1274 [00:15<00:10, 50.29it/s]#015 59%|█████▉    | 756/1274 [00:15<00:10, 50.62it/s]#015 60%|█████▉    | 762/1274 [00:15<00:10, 50.45it/s]#015 60%|██████    | 768/1274 [00:15<00:09, 50.64it/s]#015 61%|██████    | 774/1274 [00:15<00:09, 50.62it/s]#015 61%|██████    | 780/1274 [00:16<00:09, 51.16it/s]#015 62%|██████▏   | 786/1274 [00:16<00:09, 51.29it/s]#015 62%|██████▏   | 792/1274 [00:16<00:09, 51.16it/s]#015 63%|██████▎   | 798/1274 [00:16<00:09, 49.10it/s]#015 63%|██████▎   | 803/1274 [00:16<00:09, 48.74it/s]#015 64%|██████▎   | 809/1274 [00:16<00:09, 49.64it/s]#015 64%|██████▍   | 814/1274 [00:16<00:09, 49.33it/s]#015 64%|██████▍   | 819/1274 [00:16<00:09, 48.53it/s]#015 65%|██████▍   | 824/1274 [00:16<00:09, 48.70it/s]#015 65%|██████▌   | 830/1274 [00:17<00:08, 49.92it/s]#015 66%|██████▌   | 836/1274 [00:17<00:08, 50.46it/s]#015 66%|██████▌   | 842/1274 [00:17<00:08, 49.96it/s]#015 67%|██████▋   | 848/1274 [00:17<00:08, 49.44it/s]#015 67%|██████▋   | 854/1274 [00:17<00:08, 49.48it/s]#015 68%|██████▊   | 860/1274 [00:17<00:08, 49.64it/s]#015 68%|██████▊   | 866/1274 [00:17<00:08, 49.97it/s]#015 68%|██████▊   | 872/1274 [00:17<00:08, 50.18it/s]#015 69%|██████▉   | 878/1274 [00:17<00:07, 49.68it/s]#015 69%|██████▉   | 883/1274 [00:18<00:07, 49.25it/s]#015 70%|██████▉   | 888/1274 [00:18<00:07, 48.34it/s]#015 70%|███████   | 893/1274 [00:18<00:08, 47.30it/s]#015 70%|███████   | 898/1274 [00:18<00:07, 47.41it/s]#015 71%|███████   | 903/1274 [00:18<00:07, 47.62it/s]#015 71%|███████▏  | 909/1274 [00:18<00:07, 48.57it/s]#015 72%|███████▏  | 915/1274 [00:18<00:07, 49.58it/s]#015 72%|███████▏  | 921/1274 [00:18<00:07, 50.24it/s]#015 73%|███████▎  | 927/1274 [00:18<00:06, 50.85it/s]#015 73%|███████▎  | 933/1274 [00:19<00:06, 50.47it/s]#015 74%|███████▎  | 939/1274 [00:19<00:06, 50.98it/s]#015 74%|███████▍  | 945/1274 [00:19<00:06, 51.28it/s]#015 75%|███████▍  | 951/1274 [00:19<00:06, 51.07it/s]#015 75%|███████▌  | 957/1274 [00:19<00:06, 51.22it/s]#015 76%|███████▌  | 963/1274 [00:19<00:06, 51.26it/s]#015 76%|███████▌  | 969/1274 [00:19<00:05, 51.31it/s]#015 77%|███████▋  | 975/1274 [00:19<00:05, 51.45it/s]#015 77%|███████▋  | 981/1274 [00:20<00:05, 51.51it/s]#015 77%|███████▋  | 987/1274 [00:20<00:05, 51.30it/s]#015 78%|███████▊  | 993/1274 [00:20<00:05, 51.46it/s]#015 78%|███████▊  | 999/1274 [00:20<00:05, 51.51it/s]#015 79%|███████▉  | 1005/1274 [00:20<00:05, 51.63it/s]#015 79%|███████▉  | 1011/1274 [00:20<00:05, 51.60it/s]#015 80%|███████▉  | 1017/1274 [00:20<00:05, 51.10it/s]#015 80%|████████  | 1023/1274 [00:20<00:04, 51.33it/s]#015 81%|████████  | 1029/1274 [00:20<00:04, 51.61it/s]#015 81%|████████  | 1035/1274 [00:21<00:04, 51.71it/s]#015 82%|████████▏ | 1041/1274 [00:21<00:04, 51.82it/s]#015 82%|████████▏ | 1047/1274 [00:21<00:04, 51.50it/s]#015 83%|████████▎ | 1053/1274 [00:21<00:04, 49.34it/s]#015 83%|████████▎ | 1058/1274 [00:21<00:04, 47.67it/s]#015 83%|████████▎ | 1063/1274 [00:21<00:04, 47.96it/s]#015 84%|████████▍ | 1068/1274 [00:21<00:04, 47.65it/s]#015 84%|████████▍ | 1073/1274 [00:21<00:04, 47.70it/s]#015 85%|████████▍ | 1079/1274 [00:21<00:03, 48.81it/s]#015 85%|████████▌ | 1085/1274 [00:22<00:03, 49.82it/s]#015 86%|████████▌ | 1091/1274 [00:22<00:03, 50.41it/s]#015 86%|████████▌ | 1097/1274 [00:22<00:03, 50.75it/s]#015 87%|████████▋ | 1103/1274 [00:22<00:03, 51.25it/s]#015 87%|████████▋ | 1109/1274 [00:22<00:03, 51.07it/s]#015 88%|████████▊ | 1115/1274 [00:22<00:03, 50.77it/s]#015 88%|████████▊ | 1121/1274 [00:22<00:02, 51.04it/s]#015 88%|████████▊ | 1127/1274 [00:22<00:02, 51.23it/s]#015 89%|████████▉ | 1133/1274 [00:23<00:02, 51.38it/s]#015 89%|████████▉ | 1139/1274 [00:23<00:02, 51.36it/s]#015 90%|████████▉ | 1145/1274 [00:23<00:02, 51.43it/s]#015 90%|█████████ | 1151/1274 [00:23<00:02, 51.53it/s]#015 91%|█████████ | 1157/1274 [00:23<00:02, 51.45it/s]#015 91%|█████████▏| 1163/1274 [00:23<00:02, 51.73it/s]#015 92%|█████████▏| 1169/1274 [00:23<00:02, 51.77it/s]#015 92%|█████████▏| 1175/1274 [00:23<00:01, 51.38it/s]#015 93%|█████████▎| 1181/1274 [00:23<00:01, 51.47it/s]#015 93%|█████████▎| 1187/1274 [00:24<00:01, 51.57it/s]#015 94%|█████████▎| 1193/1274 [00:24<00:01, 51.55it/s]#015 94%|█████████▍| 1199/1274 [00:24<00:01, 51.71it/s]#015 95%|█████████▍| 1205/1274 [00:24<00:01, 51.43it/s]#015 95%|█████████▌| 1211/1274 [00:24<00:01, 51.28it/s]#015 96%|█████████▌| 1217/1274 [00:24<00:01, 51.41it/s]#015 96%|█████████▌| 1223/1274 [00:24<00:00, 51.25it/s]#015 96%|█████████▋| 1229/1274 [00:24<00:00, 51.43it/s]#015 97%|█████████▋| 1235/1274 [00:25<00:00, 51.55it/s]#015 97%|█████████▋| 1241/1274 [00:25<00:00, 51.67it/s]#015 98%|█████████▊| 1247/1274 [00:25<00:00, 51.76it/s]#015 98%|█████████▊| 1253/1274 [00:25<00:00, 51.87it/s]#015 99%|█████████▉| 1259/1274 [00:25<00:00, 51.94it/s]#015 99%|█████████▉| 1265/1274 [00:25<00:00, 52.03it/s]#015100%|█████████▉| 1271/1274 [00:25<00:00, 52.12it/s]#0151275it [00:25, 49.32it/s]                          

